# Create a search index in Azure AI Search using the Azure SDK for Python

This Jupyter Notebook steps through creating, loading, and querying an index in Azure AI Search index by calling the azure-search-documents library in the Azure SDK for Python. 

In [9]:
# Uncomment the next line if you need to install the azure-search-documents client library:
%pip install azure-search-documents --pre --upgrade
%pip show azure-search-documents
%pip install python-dotenv




Name: azure-search-documentsNote: you may need to restart the kernel to use updated packages.

Version: 11.4.0
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: c:\Users\manuchadha\OneDrive - Microsoft\Manu_Microsoft\demo-data\aoai-openai\aoai-code\.venv\Lib\site-packages
Requires: azure-common, azure-core, isodate
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [71]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient

from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (  
    CorsOptions,
    SearchIndex,  
    SearchField,  
    SemanticSearch,
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,
    ComplexField,
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,  
    SemanticField ,
    VectorSearchProfile,
    VectorSearchAlgorithmConfiguration,
    HnswParameters,
    VectorSearch, 
    HnswAlgorithmConfiguration
)

In this step, initialize the search client used to make each request. Provide the name and admin API key of your search service. If you get ConnectionError "Failed to establish a new connection", verify that the api-key is a primary or secondary admin key, and not a query key.

In [72]:
# Set the service endpoint and API key from the environment

service_name = "aisearchmcuksouth"
admin_key = "sguckbZFk6EbbP8Azrosy1temBCqumUdCSW73ZLJ5JAzSeBNqRps"

index_name = "hotel-finder-vector-semantic"

# Create an SDK client
endpoint = "https://aisearchmcuksouth.search.windows.net".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))


In [73]:
# Delete the index if it exists
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)


Index hotel-finder-vector-semantic Deleted


Execute this for semantic ranking part of test

In [75]:


# Specify the index schema
name = index_name

fields = [
        SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
        SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

        SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),

        SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
        SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Rating", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

        ComplexField(name="Address", fields=[
            SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
            SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        ]),
        SearchField(name="HotelNameVector",
    type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
    searchable=True, 
    vector_search_dimensions=1536,
    vector_search_profile_name="my-vector-profile"),
    SearchField(name="DescriptionVector",
    type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
    searchable=True, 
    vector_search_dimensions=1536,
    vector_search_profile_name="my-vector-profile")
    ]

hnswParameters = HnswParameters(m=4,ef_construction= 400, ef_search= 500, metric= "cosine")
vectorSearchAlgorithm = HnswAlgorithmConfiguration(name = "my-hnsw-vector-config-1",parameters=hnswParameters)
vectorSearchProfile = VectorSearchProfile(name="my-vector-profile",algorithm_configuration_name="my-hnsw-vector-config-1")

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")]
    )
)
vector_search = VectorSearch(profiles = [vectorSearchProfile],algorithms=[vectorSearchAlgorithm])
semantic_search = SemanticSearch(configurations=[semantic_config])
suggester = [{'name': 'sg', 'source_fields': ['Tags', 'Address/City', 'Address/Country']}]
scoring_profiles = []

execute for semantic search

In [76]:
index = SearchIndex(
    name=name,
    fields=fields,
    vector_search=vector_search,
    semantic_search = semantic_search,
    scoring_profiles=scoring_profiles,
    suggesters=suggester)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

Index hotel-finder-vector-semantic created


Next, provide documents that conform to the index schema. Specify an upload action for each document.

In [56]:
documents = [
    {
    "@search.action": "upload",
    "HotelId": "1",
    "HotelName": "Secret Point Motel",
    "Description": "The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.",

    "Description_fr": "L'hôtel est idéalement situé sur la principale artère commerciale de la ville en plein cœur de New York. A quelques minutes se trouve la place du temps et le centre historique de la ville, ainsi que d'autres lieux d'intérêt qui font de New York l'une des villes les plus attractives et cosmopolites de l'Amérique.",
    "Category": "Boutique",
    "Tags": [ "pool", "air conditioning", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1970-01-18T00:00:00Z",
    "Rating": "3.60",
    "Address": {
        "StreetAddress": "677 5th Ave",
        "City": "New York",
        "StateProvince": "NY",
        "PostalCode": "10022",
        "Country": "USA"
        },
    "HotelNameVector":[
                0.0040753875,
                0.0040819664,
                0.007854742,
                -0.008874412,
                -0.017301485,
                0.030655863,
                0.001705479,
                -0.012249188,
                0.009604626,
                0.0042924783,
                0.0012622519,
                0.026564028,
                -0.004144462,
                0.00006748113,
                -0.024708888,
                0.0071574203,
                0.017538311,
                -0.010887436,
                0.0032366274,
                0.008446808,
                -0.0038418504,
                0.00578909,
                0.0035425283,
                -0.021064393,
                0.0023715533,
                -0.017340956,
                0.0055884453,
                -0.0012211362,
                0.005660809,
                0.0060982797,
                0.019077683,
                -0.022709021,
                -0.009209916,
                -0.005466743,
                -0.02253798,
                0.000696911,
                -0.01073613,
                -0.023537913,
                -0.0007248697,
                -0.009551998,
                0.020314442,
                -0.018761914,
                -0.00033776544,
                -0.019775005,
                -0.0019850656,
                0.005647652,
                0.017446212,
                0.00045021687,
                -0.026340358,
                0.018590873,
                0.005571999,
                0.013367535,
                -0.0052957013,
                -0.020893352,
                -0.0020163136,
                -0.01065061,
                -0.0028073797,
                0.016933087,
                0.0052562305,
                -0.014472725,
                0.009867767,
                -0.020209186,
                -0.0344714,
                0.0188277,
                -0.00048393174,
                -0.002460363,
                0.001087099,
                0.008584957,
                -0.008959932,
                0.008532329,
                0.025156226,
                0.02377474,
                -0.003044206,
                -0.034892425,
                0.025998276,
                -0.0106571885,
                -0.005798958,
                -0.0015295038,
                0.020130245,
                0.02019603,
                0.017972492,
                -0.013446477,
                -0.021774871,
                0.025327269,
                0.0107163945,
                -0.002912636,
                -0.0065456186,
                0.020998608,
                -0.014170113,
                -0.018577715,
                0.018682972,
                0.027603433,
                0.004387867,
                0.010453254,
                -0.032497846,
                0.015485816,
                -0.0029225035,
                0.00525952,
                0.00795342,
                -0.021064393,
                -0.019893417,
                0.012058411,
                -0.017104128,
                -0.008532329,
                -0.026327202,
                -0.0065324614,
                -0.005331883,
                -0.030340094,
                0.0010246031,
                0.008716527,
                -0.020538112,
                -0.00919018,
                0.004581933,
                -0.021538045,
                -0.007769222,
                0.007854742,
                -0.0012186692,
                0.021222277,
                -0.021774871,
                -0.011413717,
                0.02739292,
                0.024669416,
                -0.0012778759,
                -0.0028156028,
                0.015051634,
                -0.009709883,
                -0.01997236,
                -0.009400693,
                0.017525153,
                0.013683303,
                0.02412998,
                -0.0005891879,
                0.0058581643,
                -0.0039832885,
                -0.016091038,
                0.0075323954,
                -0.008788891,
                0.00026416834,
                -0.032760985,
                -0.009670411,
                0.028024457,
                0.030813746,
                -0.008427073,
                0.0064502303,
                0.025998276,
                0.0015385493,
                -0.0033649085,
                -0.009545419,
                0.011611072,
                -0.018932955,
                0.007920528,
                -0.02266955,
                0.012222874,
                0.0051575527,
                -0.020932823,
                0.025800921,
                -0.024577318,
                0.025393054,
                -0.029708557,
                -0.028050771,
                0.000114918366,
                0.03681335,
                0.007736329,
                0.008992825,
                0.02836654,
                0.02934016,
                0.020340757,
                0.012288659,
                -0.0056048916,
                -0.004891123,
                0.008512594,
                0.008821784,
                -0.032682043,
                0.020485483,
                -0.0150253195,
                0.0032711646,
                -0.013110973,
                -0.002593578,
                -0.0131372865,
                -0.01913031,
                -0.015998939,
                -0.0048088916,
                0.019525021,
                0.030050639,
                0.006354842,
                -0.010315105,
                0.0052989908,
                -0.014972691,
                0.0007400825,
                0.0036708093,
                0.015630543,
                0.011670279,
                0.0023797764,
                -0.0046411394,
                -0.6727975,
                -0.013038609,
                -0.022682706,
                -0.009078345,
                0.034366142,
                0.017393583,
                0.029103333,
                0.004581933,
                -0.029155962,
                0.0060686767,
                -0.008045519,
                0.021643301,
                0.008209982,
                -0.002527793,
                -0.0057134368,
                -0.0017729087,
                -0.0072297836,
                -0.011433452,
                0.008690213,
                -0.0017202806,
                -0.0104203615,
                0.023353714,
                -0.007440296,
                0.033208326,
                0.042839266,
                -0.011775535,
                0.006578511,
                0.005002958,
                -0.027103467,
                -0.00054396066,
                -0.0034899001,
                0.017354112,
                0.009801982,
                -0.010051965,
                0.04775999,
                -0.020446014,
                -0.012361023,
                0.03628707,
                -0.00455233,
                0.03857639,
                -0.019103996,
                -0.009670411,
                0.013393849,
                0.0025606854,
                0.0054470073,
                -0.0022301152,
                0.021432789,
                -0.020893352,
                -0.0020837435,
                -0.033945117,
                0.037234373,
                0.015248989,
                -0.011176891,
                0.01179527,
                -0.012545221,
                -0.01542003,
                0.019419765,
                -0.012775469,
                -0.009170445,
                0.0028534292,
                0.005854875,
                0.001566508,
                -0.032971498,
                -0.024379963,
                -0.012735997,
                0.01948555,
                -0.009163866,
                -0.007492924,
                -0.0110124275,
                0.0070455857,
                0.0006812871,
                0.027208723,
                -0.033787232,
                0.006239718,
                -0.0013625742,
                0.025248326,
                0.013630675,
                -0.019275038,
                -0.012255766,
                -0.013709618,
                -0.0013296816,
                -0.012097882,
                -0.030655863,
                -0.019314509,
                0.022261681,
                -0.0024110244,
                -0.022380095,
                0.009690147,
                0.021445947,
                0.0026906112,
                -0.0028287598,
                0.0034339828,
                0.009795403,
                0.0070126927,
                0.005289123,
                0.0021544625,
                0.003512925,
                0.011716329,
                0.035181876,
                -0.05836455,
                -0.014143799,
                -0.026564028,
                0.008427073,
                0.017354112,
                0.020867037,
                -0.022695864,
                0.016419964,
                -0.020064458,
                0.028550738,
                -0.014472725,
                0.012137353,
                -0.0020228922,
                -0.014867435,
                -0.015064791,
                -0.021222277,
                -0.033918805,
                0.0051312386,
                -0.0040096026,
                0.0075060814,
                -0.013446477,
                0.004223404,
                -0.0040589413,
                0.006246296,
                -0.005456875,
                0.022406409,
                0.01626208,
                0.011341353,
                -0.007920528,
                0.007052164,
                -0.006335106,
                -0.0065850895,
                -0.001564041,
                0.015077948,
                -0.006305503,
                0.00064716104,
                -0.0005443718,
                0.013341221,
                -0.017564625,
                0.015143733,
                -0.0077034365,
                -0.044681247,
                0.00397671,
                -0.009650676,
                0.006578511,
                -0.01369646,
                -0.025327269,
                0.002138016,
                -0.009703304,
                -0.0121899815,
                -0.009532263,
                -0.00075488415,
                -0.014709551,
                -0.030366408,
                0.019314509,
                -0.013189915,
                -0.013249122,
                0.0006874544,
                -0.009723039,
                -0.0053483294,
                -0.024393119,
                0.0031659086,
                0.01320965,
                -0.0108940145,
                -0.0010665412,
                0.013334642,
                -0.015196361,
                -0.0020771648,
                0.020314442,
                0.0022416278,
                -0.013630675,
                -0.0030162474,
                -0.0011907106,
                -0.005726594,
                -0.011933419,
                -0.033392522,
                0.008578379,
                0.0007688635,
                -0.012657056,
                -0.0016495617,
                0.008124461,
                -0.007519238,
                0.009519106,
                -0.013551733,
                0.019603964,
                0.020011831,
                -0.009973023,
                0.0327873,
                0.012183403,
                -0.017025188,
                0.0026412723,
                0.00085438415,
                0.0036214704,
                0.001155351,
                0.011440031,
                0.010367733,
                0.008808627,
                0.05536475,
                0.0066377176,
                0.009380957,
                0.037892226,
                -0.0058088256,
                0.014683237,
                0.004848363,
                -0.0041806437,
                0.0046641645,
                -0.02973487,
                0.00091770233,
                -0.05731199,
                0.035681844,
                0.004489834,
                0.019603964,
                -0.034445085,
                -0.0036576523,
                -0.016354179,
                0.0039043464,
                0.019643435,
                0.006835073,
                0.030103268,
                0.0130780805,
                0.010670345,
                -0.012104461,
                0.02280112,
                0.007361354,
                0.0027383054,
                -0.0012458055,
                0.009098081,
                0.019288195,
                0.0003677799,
                0.007966577,
                0.010091436,
                -0.019459236,
                -0.018761914,
                0.014525353,
                -0.00030816213,
                -0.005558842,
                -0.004026049,
                -0.0056871227,
                -0.02394578,
                0.0084731225,
                -0.0062989243,
                0.015525286,
                0.007631073,
                0.008012626,
                -0.030892689,
                -0.021932757,
                0.02986644,
                0.04162882,
                0.0026922557,
                0.010315105,
                0.013814873,
                -0.033813547,
                -0.003141239,
                -0.009545419,
                0.041655134,
                -0.0021330824,
                0.0068942797,
                0.012268923,
                0.024906244,
                0.0110716345,
                0.024037879,
                -0.015406873,
                -0.017459368,
                -0.00450628,
                -0.0033879334,
                0.014722708,
                0.0038879002,
                0.011893949,
                -0.010203271,
                0.0088875685,
                0.022327468,
                -0.007736329,
                0.01731464,
                0.0059272386,
                0.0060423627,
                -0.005394379,
                -0.009841452,
                -0.008874412,
                0.034392457,
                0.0012277147,
                0.0060094697,
                -0.027287664,
                -0.019775005,
                0.032892555,
                -0.012597849,
                -0.00941385,
                -0.005173999,
                0.0004202024,
                -0.0013009006,
                -0.0130780805,
                0.009926974,
                0.0150253195,
                -0.014999006,
                0.0007421383,
                -0.020511799,
                -0.017814608,
                -0.009940131,
                -0.0072758333,
                -0.031155828,
                0.015143733,
                0.01970922,
                0.0111308405,
                -0.0074074036,
                -0.01418327,
                -0.031418968,
                0.011538709,
                0.0010517394,
                -0.030024325,
                -0.021248592,
                0.0007030784,
                -0.029761184,
                0.0032284043,
                0.0037497515,
                -0.011597916,
                -0.0066969246,
                -0.016380493,
                -0.01184132,
                -0.020827567,
                -0.02081441,
                0.04599695,
                0.008828362,
                -0.024998343,
                -0.010959799,
                -0.019288195,
                0.0015040122,
                0.10273003,
                0.0030951896,
                0.024603631,
                0.00512466,
                -0.00006388351,
                0.019775005,
                -0.026143003,
                -0.03657652,
                0.0029767763,
                -0.01126899,
                -0.015367402,
                0.006164065,
                0.00056780776,
                0.0004325371,
                0.026129846,
                0.0068613873,
                -0.022880062,
                -0.011775535,
                0.013828031,
                -0.0201434,
                -0.035155565,
                0.007091635,
                -0.0146174515,
                0.026392987,
                -0.008552064,
                -0.0149069065,
                0.01060456,
                0.01453851,
                0.009696726,
                -0.014999006,
                -0.0011783758,
                0.011676857,
                0.01528846,
                -0.0029718424,
                0.00095635105,
                0.0078021144,
                -0.0015558179,
                -0.007762643,
                0.016143667,
                0.008538907,
                0.046470605,
                0.0050128256,
                0.0063482635,
                -0.0035293712,
                0.0082955025,
                0.014262212,
                0.011170312,
                0.0218275,
                0.0016709418,
                0.015827898,
                0.030208524,
                -0.007986313,
                0.0055259494,
                -0.013466212,
                0.0000157653,
                0.014341154,
                0.01188737,
                -0.004302346,
                0.0024274706,
                -0.003835272,
                -0.026537715,
                -0.030550607,
                0.007091635,
                -0.009571734,
                -0.023195831,
                -0.019248724,
                -0.00517071,
                0.0077758003,
                -0.005871321,
                -0.0022810989,
                0.012084725,
                -0.0028863219,
                -0.022393253,
                -0.00064428296,
                0.024722045,
                0.02328793,
                -0.002800801,
                -0.01851193,
                -0.0036116026,
                0.03531345,
                -0.005792379,
                -0.014288526,
                -0.014867435,
                -0.029682243,
                0.023340559,
                0.017827766,
                0.05360171,
                -0.024827302,
                -0.0053384616,
                0.027945515,
                0.02744555,
                -0.00746661,
                0.011209783,
                -0.01188737,
                -0.0040556523,
                -0.01905137,
                0.0007055453,
                0.035944983,
                -0.0054766107,
                0.013512262,
                0.046233777,
                -0.034076687,
                0.017748823,
                -0.023695797,
                0.024972029,
                -0.006854809,
                -0.0006187912,
                -0.010551932,
                -0.00039512184,
                0.01834089,
                0.012334708,
                -0.012045254,
                -0.022630079,
                0.0086507425,
                -0.009696726,
                0.022116955,
                -0.011801849,
                -0.012874146,
                -0.014459567,
                -0.013854344,
                0.008203403,
                -0.021274906,
                0.018840857,
                0.026458772,
                -0.0061870897,
                -0.015262146,
                0.010005916,
                -0.005134528,
                0.008775734,
                -0.015012163,
                -0.02103808,
                0.021998541,
                -0.0020524955,
                -0.01493322,
                -0.005289123,
                -0.028077086,
                -0.0013428386,
                0.019169781,
                0.019827632,
                -0.021367004,
                -0.02359054,
                0.00419709,
                -0.011328196,
                0.009505949,
                -0.012439964,
                -0.016367337,
                -0.023077417,
                -0.008644164,
                -0.004223404,
                0.035418704,
                0.00397671,
                0.035760786,
                0.00216433,
                0.0072297836,
                -0.003190578,
                -0.042260356,
                -0.02902439,
                -0.011334775,
                0.0011660411,
                -0.0018370493,
                0.0050753215,
                0.0002598512,
                0.012025518,
                0.015209517,
                -0.013012296,
                0.005637784,
                0.002549173,
                0.014499038,
                -0.03344515,
                0.014683237,
                0.009775667,
                0.030629547,
                0.0027185697,
                0.013630675,
                0.012525485,
                0.02739292,
                0.011591337,
                0.0044733877,
                -0.017459368,
                -0.026971895,
                -0.0051542637,
                -0.01224261,
                -0.00035832328,
                0.0005645185,
                -0.020169714,
                0.011387403,
                0.0076113376,
                0.002637983,
                -0.017340956,
                -0.0029323713,
                0.011617651,
                -0.011703172,
                0.022156425,
                -0.0034339828,
                0.0010944998,
                0.0006878656,
                -0.0037661977,
                -0.0003677799,
                -0.0035787101,
                0.0023172805,
                -0.0020705864,
                -0.010315105,
                0.024840457,
                0.010571668,
                0.011893949,
                0.019025054,
                -0.018656658,
                -0.03362935,
                -0.016498907,
                -0.02836654,
                -0.016512062,
                -0.011893949,
                -0.03852376,
                -0.036602836,
                0.03155054,
                -0.0003180299,
                -0.0061344616,
                0.019472392,
                -0.012472857,
                -0.023090575,
                -0.002593578,
                -0.021130178,
                0.020288128,
                0.0017877105,
                0.024932558,
                0.028050771,
                -0.008453387,
                -0.019656591,
                0.04241824,
                -0.022116955,
                0.003884611,
                0.026511401,
                0.012275502,
                0.00083053706,
                -0.008150775,
                -0.00437471,
                0.017222542,
                -0.0060028913,
                -0.022590607,
                0.010900593,
                0.013893816,
                0.0022827433,
                -0.024116822,
                0.00071623543,
                0.0075323954,
                0.014972691,
                -0.0044964124,
                0.013242543,
                -0.023669483,
                -0.00091770233,
                -0.040365744,
                0.013354378,
                0.0081770895,
                0.017577782,
                0.018406674,
                0.015314774,
                0.00079353293,
                -0.007874478,
                0.0024192475,
                -0.008545486,
                -0.021616988,
                0.037629083,
                -0.032682043,
                0.007052164,
                -0.012532064,
                0.02655087,
                -0.01038089,
                0.0032695201,
                -0.025379896,
                0.021630146,
                -0.025800921,
                -0.026590342,
                0.0045128586,
                -0.018367203,
                0.02130122,
                0.0019505286,
                -0.010545353,
                -0.018525088,
                -0.008315238,
                -0.00042390282,
                0.011407139,
                0.019827632,
                -0.010407205,
                -0.023051104,
                -0.007223205,
                -0.010571668,
                -0.0046904786,
                0.0017021898,
                -0.017301485,
                -0.01935398,
                -0.01493322,
                -0.025103599,
                -0.004614826,
                -0.012828097,
                0.0010295371,
                0.003772776,
                0.012268923,
                0.041918274,
                -0.033418838,
                0.014841122,
                0.01603841,
                0.037076488,
                -0.021577517,
                0.010492725,
                0.031839993,
                -0.0033484623,
                0.023801053,
                -0.0078086928,
                -0.00998618,
                -0.017840922,
                -0.0153805595,
                0.014525353,
                -0.0023041235,
                0.013025452,
                0.025471995,
                0.001071475,
                -0.015314774,
                -0.0036773877,
                -0.016933087,
                0.0062101143,
                -0.022603765,
                -0.000930037,
                -0.004249718,
                -0.006147619,
                0.00901256,
                0.015525286,
                -0.011209783,
                -0.0037925118,
                -0.0066804783,
                0.0010903883,
                -0.007104792,
                0.00025944004,
                -0.014880592,
                -0.01785408,
                -0.00583185,
                -0.023919467,
                -0.006920594,
                0.024406277,
                -0.02514307,
                -0.004364842,
                0.012874146,
                0.0154989725,
                -0.024629947,
                -0.0018255368,
                -0.01948555,
                -0.023893153,
                0.0018699418,
                -0.0032316935,
                -0.00039388836,
                -0.018077748,
                -0.011630808,
                0.035181876,
                0.0034668753,
                -0.018038278,
                -0.0003704524,
                0.0047431067,
                -0.031524226,
                0.0139069725,
                -0.0009530618,
                -0.0006331817,
                0.044970702,
                0.0009061899,
                -0.022985319,
                0.0104795685,
                0.038629018,
                0.018946113,
                0.0017137022,
                0.00614104,
                0.011387403,
                -0.011143998,
                0.0025376608,
                0.004410892,
                -0.024722045,
                0.003062297,
                0.008716527,
                0.0187356,
                -0.0039832885,
                0.007295569,
                -0.021761715,
                -0.024116822,
                0.0070718997,
                -0.007078478,
                0.004272743,
                -0.011584758,
                0.0019028344,
                -0.026261417,
                0.00033159807,
                0.0090059815,
                0.01815669,
                -0.030971631,
                0.008775734,
                0.011236097,
                0.019893417,
                -0.0028616523,
                -0.03070849,
                -0.021064393,
                -0.003106702,
                -0.016564691,
                0.0065653543,
                0.026682442,
                -0.008065254,
                -0.00684823,
                0.00009816372,
                -0.008124461,
                -0.009084924,
                -0.000286782,
                -0.052022867,
                -0.021774871,
                0.0041477513,
                -0.021788029,
                0.0007762643,
                -0.00050078915,
                0.014301684,
                -0.009703304,
                0.011084791,
                0.008933618,
                0.01603841,
                0.0045490405,
                0.0059107924,
                0.01303203,
                -0.01338727,
                0.022656392,
                -0.038760588,
                -0.016617319,
                -0.0021988673,
                -0.009117817,
                -0.0005673966,
                -0.01622261,
                0.0045918007,
                -0.0033254374,
                0.014235898,
                -0.0149069065,
                0.0032728093,
                0.016498907,
                -0.013282014,
                0.028471796,
                0.23408975,
                0.01528846,
                -0.019722376,
                0.032945186,
                0.0121899815,
                0.027208723,
                0.019498708,
                -0.005223338,
                0.000102429476,
                0.021538045,
                -0.015564757,
                -0.00684823,
                0.005539106,
                0.00093990477,
                0.017696194,
                -0.040471002,
                -0.022564294,
                -0.024761517,
                -0.023853682,
                -0.024222078,
                0.010578246,
                0.018669814,
                0.00016569626,
                0.013801716,
                0.014551667,
                -0.004953619,
                -0.0297875,
                -0.0008831651,
                0.004588512,
                0.040444687,
                0.0011939998,
                -0.00008490391,
                0.00910466,
                0.002310702,
                0.017748823,
                0.007854742,
                0.008433651,
                0.0037037018,
                0.021024922,
                0.014341154,
                -0.016604163,
                -0.020695996,
                0.011216362,
                0.0037234374,
                -0.017064659,
                -0.020393385,
                -0.017393583,
                0.013880659,
                0.006384445,
                0.014472725,
                -0.026485085,
                -0.00905861,
                -0.0087099485,
                -0.016801517,
                -0.011183469,
                -0.0109532215,
                0.010630874,
                -0.0012893882,
                -0.016025253,
                -0.01727517,
                0.0046378504,
                0.020077616,
                -0.019393452,
                0.008275767,
                -0.01678836,
                0.013643832,
                0.013203072,
                0.009216494,
                0.009861188,
                -0.024866773,
                0.006756131,
                -0.011565023,
                0.012426808,
                0.0014217807,
                -0.04620746,
                -0.029708557,
                0.022972161,
                0.00066319614,
                0.028971763,
                0.023235302,
                -0.037760653,
                -0.013025452,
                -0.005275966,
                -0.026182475,
                0.020393385,
                -0.037365943,
                0.02359054,
                -0.0017252145,
                -0.004828627,
                0.01731464,
                -0.010538775,
                -0.035023995,
                -0.015972625,
                -0.0017202806,
                -0.011545287,
                0.006420627,
                0.004068809,
                0.01622261,
                -0.010900593,
                0.013985915,
                0.009762511,
                -0.013420163,
                0.0044865445,
                -0.0104203615,
                -0.025432523,
                0.002813958,
                0.009782246,
                0.027340293,
                -0.0003075454,
                -0.015854212,
                -0.007716594,
                -0.009052032,
                0.010440097,
                -0.002208735,
                -0.012492592,
                -0.0028073797,
                0.029050706,
                -0.01926188,
                -0.00093085936,
                -0.008558643,
                0.0017235699,
                -0.024906244,
                -0.00049174373,
                -0.013604362,
                0.026366673,
                -0.008861255,
                0.014314841,
                -0.01418327,
                0.032892555,
                -0.020722311,
                0.017761981,
                -0.016288394,
                0.013512262,
                0.0093020145,
                -0.017025188,
                0.028708624,
                0.004312214,
                -0.017104128,
                0.0034142474,
                0.01250575,
                0.016696261,
                0.015801584,
                0.02231431,
                -0.0038155364,
                -0.0020689417,
                -0.013499105,
                0.013426742,
                0.021932757,
                -0.018682972,
                0.009913816,
                0.0073021473,
                -0.02111702,
                0.016933087,
                -0.004785867,
                0.015248989,
                -0.020419698,
                -0.023695797,
                -0.012578113,
                -0.008479701,
                0.031313714,
                -0.0203276,
                -0.008683635,
                0.018327732,
                -0.0030902557,
                -0.021564359,
                -0.002907702,
                -0.1693572,
                0.024590475,
                0.0008436941,
                -0.00494704,
                0.011597916,
                0.010492725,
                0.021024922,
                0.01763041,
                -0.018314576,
                0.0022728755,
                -0.015854212,
                -0.0064304946,
                -0.023090575,
                0.00010751754,
                0.02147226,
                0.008466544,
                -0.0072297836,
                0.0050522964,
                -0.012512328,
                -0.005341751,
                0.005279255,
                0.0017120575,
                0.0037760655,
                0.0028649417,
                0.008953353,
                0.01731464,
                0.0007010226,
                0.023182673,
                0.006900858,
                0.009749354,
                -0.00005463248,
                -0.006667321,
                0.011440031,
                0.013203072,
                0.027498176,
                -0.006012759,
                -0.005601602,
                0.004940462,
                0.0007182912,
                0.0069074365,
                -0.008966511,
                0.023722112,
                0.01772251,
                -0.014209584,
                -0.0018386939,
                0.012512328,
                -0.00808499,
                0.004167487,
                0.009683568,
                0.002187355,
                0.0060917013,
                -0.027287664,
                0.009150709,
                0.006124594,
                0.0093020145,
                -0.0018074459,
                -0.005512792,
                0.023445813,
                0.000063163985,
                0.016946245,
                -0.029050706,
                -0.026814012,
                0.0028879663,
                0.00041711872,
                0.0010616073,
                -0.042312983,
                -0.001409446,
                -0.009486213,
                -0.035550274,
                0.016538378,
                -0.0011430163,
                -0.027050838,
                -0.006127883,
                -0.022590607,
                -0.008052098,
                0.005404247,
                -0.00998618,
                0.0067166598,
                -0.011828163,
                -0.008052098,
                -0.018051434,
                0.031839993,
                -0.027103467,
                -0.0026790986,
                0.00038607637,
                -0.00032789767,
                0.011078212,
                0.0021347268,
                0.010407205,
                0.006410759,
                0.008499436,
                -0.026577186,
                0.027682375,
                -0.025169384,
                0.033155695,
                0.026919268,
                0.008216561,
                0.032050505,
                0.0019077682,
                -0.011880791,
                -0.020906508,
                0.001962041,
                -0.025156226,
                0.004348396,
                0.0037925118,
                -0.008242874,
                0.03760277,
                0.017038343,
                0.029129647,
                -0.009137552,
                -0.0139069725,
                -0.0026790986,
                0.017406741,
                0.019209253,
                -0.010729551,
                0.0027021235,
                -0.0010542064,
                -0.0037431729,
                0.022774806,
                0.018709285,
                0.035813414,
                -0.0068679657,
                0.0023008343,
                0.004920726,
                -0.0033912226,
                -0.025156226,
                -0.06994273,
                -0.019735534,
                0.011946577,
                0.003858297,
                -0.03247153,
                0.027366607,
                -0.00009615933,
                0.032892555,
                -0.0034964788,
                0.022945847,
                0.0055785775,
                -0.014735865,
                -0.014314841,
                -0.001794289,
                0.021051235,
                -0.013683303,
                0.0014217807,
                -0.025432523,
                -0.020867037,
                0.010551932,
                -0.012308395,
                -0.015156889,
                -0.018959269,
                0.0085915355,
                0.0010172023,
                -0.00017546123,
                -0.026366673,
                0.038997415,
                0.015209517,
                0.017735666,
                -0.0014456279,
                -0.0038648753,
                0.02426155,
                -0.023748426,
                0.010722973,
                0.0033484623,
                -0.0027514624,
                0.017564625,
                0.017656725,
                -0.009880924,
                -0.022985319,
                0.004723371,
                0.0043582637,
                -0.022787962,
                -0.009953287,
                0.013814873,
                -0.008663899,
                0.024735201,
                0.0037267266,
                0.0075981803,
                0.004437206,
                0.006795602,
                -0.023366872,
                0.0005242251,
                0.008953353,
                -0.00022284707,
                0.016077882,
                0.003353396,
                -0.02063021,
                -0.013801716,
                -0.017196229,
                0.023235302,
                0.008334974,
                -0.0019110575,
                0.013920129,
                0.008966511,
                0.0042694537,
                -0.012821518,
                0.026708756,
                -0.010801915,
                -0.0076508084,
                0.00063647097,
                -0.02907702,
                0.026208788,
                -0.014814807,
                0.0017383716,
                -0.032629415,
                -0.02407735,
                -0.0017334376,
                -0.016472593,
                -0.0139069725,
                -0.017288327,
                0.00013527063,
                -0.025814079,
                0.025787763,
                0.009124395,
                0.033603035,
                -0.0104795685,
                -0.0024537847,
                -0.02907702,
                -0.0016840989,
                0.025958804,
                0.007078478,
                -0.013946444,
                -0.0054864786,
                -0.0057331724,
                -0.01727517,
                -0.0091836015,
                -0.026090376,
                0.0016750534,
                -0.02894545,
                -0.033208326,
                -0.081731424,
                0.00062372506,
                0.018893484,
                -0.013459634,
                0.010815073,
                -0.023656325,
                -0.0078810565,
                0.0046510072,
                -0.0021051236,
                -0.0036181812,
                -0.02553778,
                -0.004588512,
                0.0131372865,
                -0.001705479,
                -0.02739292,
                -0.013051766,
                0.008137618,
                -0.015209517,
                0.004272743,
                0.0025837102,
                0.01489375,
                0.0036313382,
                0.022853747,
                -0.009861188,
                -0.009052032,
                0.018143535,
                -0.027945515,
                0.026787698,
                -0.015564757,
                0.018314576,
                0.003733305,
                -0.0233274,
                0.01082823,
                0.04970723,
                -0.044839133,
                -0.0025212143,
                -0.017301485,
                0.009111238,
                -0.0021528178,
                -0.012893882,
                -0.02699821,
                -0.022906376,
                -0.00865732,
                -0.007091635,
                -0.017393583,
                0.014683237,
                -0.015827898,
                0.0034274044,
                0.019169781,
                -0.021814343,
                0.013328063,
                0.019893417,
                -0.026590342,
                -0.003808958,
                -0.015564757,
                0.011288725,
                0.0059765773,
                0.0054009575,
                -0.013946444,
                -0.0100717,
                0.017696194,
                0.017459368,
                0.027945515,
                -0.002239983,
                0.0187356,
                0.009111238,
                -0.002244917,
                0.0124597,
                0.009255965,
                -0.022327468,
                -0.0423393,
                0.007413982,
                0.023340559,
                0.005233206,
                -0.002478454,
                -0.0327873,
                0.016367337,
                -0.018972427,
                -0.019893417,
                0.018235633,
                0.039023727,
                0.017511997,
                -0.010321684,
                -0.011716329,
                0.027813945,
                0.0042957678,
                -0.017577782,
                0.009933552,
                0.0044470737,
                0.011453188,
                0.00018296484,
                0.0172357,
                0.0031872888,
                0.014091171,
                -0.007361354,
                0.025774607,
                0.0042957678,
                0.0018074459,
                0.0063219494,
                0.008196825,
                -0.002001512,
                -0.0002148295,
                0.0033369497,
                -0.010959799,
                -0.01678836,
                0.015235832,
                -0.032418903,
                -0.0312874,
                0.012117618,
                -0.000042554744,
                0.014262212,
                0.0064535197,
                -0.00905861,
                0.022353781,
                -0.0022778094,
                -0.0048845448,
                0.016854146,
                -0.017617254,
                -0.003808958,
                0.0008301259,
                -0.0023485285,
                0.016696261,
                0.007295569,
                0.002174198,
                0.02377474,
                0.0076244944,
                0.000007047487,
                0.0005188801,
                -0.014170113,
                0.00074830564,
                -0.006335106,
                -0.018090906,
                0.0040819664,
                -0.008492858,
                -0.017933022,
                0.019998673,
                -0.011111106,
                0.01922241,
                -0.016683104,
                0.05889083,
                0.020617055,
                -0.008190246,
                0.020827567,
                -0.0076113376,
                0.020064458,
                -0.0023353714,
                0.006305503,
                -0.023537913,
                -0.017696194,
                0.016906774,
                -0.018367203,
                0.018814541,
                -0.004282611,
                -0.0028961895,
                -0.032629415,
                -0.00089549983,
                0.009407271,
                0.010137485,
                -0.02593249,
                0.021143336,
                -0.010032229,
                0.017419897,
                0.009427006,
                -0.008341552,
                0.0034833218,
                -0.011650544,
                -0.00049708877,
                -0.003106702,
                -0.027366607,
                0.0093020145,
                0.01970922,
                -0.012867568,
                -0.015709484,
                0.016143667,
                -0.021985384,
                -0.010624296,
                -0.0069074365,
                0.01877507,
                0.0065653543,
                -0.019985517,
                0.0131964935,
                -0.01847246,
                -0.015722642,
                0.047654737,
                0.010788758,
                -0.0019850656,
                0.0033649085,
                -0.022709021
            ]
     ,
    "DescriptionVector": [
                -0.02472702,
                0.013989609,
                -0.009879193,
                -0.025204524,
                -0.012718412,
                0.006581827,
                -0.010627715,
                0.0010235709,
                0.0006347914,
                -0.013628253,
                0.018635605,
                0.01136333,
                -0.0014615527,
                0.004323357,
                -0.028856793,
                -0.020068118,
                0.020068118,
                -0.014867186,
                -0.012350605,
                -0.025243241,
                -0.02484317,
                0.014183192,
                -0.025746558,
                -0.0038426253,
                -0.016970789,
                0.029915048,
                0.00791755,
                0.0026811266,
                0.008620903,
                -0.021810368,
                -0.0067560514,
                -0.024507625,
                -0.02637893,
                -0.017293427,
                0.0135379145,
                0.0063108103,
                -0.0065947324,
                0.0034006105,
                0.005352574,
                -0.0035974202,
                0.016364228,
                -0.023565521,
                0.0059075123,
                0.0015591509,
                -0.033399545,
                0.015757667,
                0.025062565,
                -0.029269772,
                -0.040110428,
                0.012428038,
                -0.0089241825,
                0.027282318,
                -0.015499557,
                -0.015280163,
                -0.023875255,
                -0.0088854665,
                -0.0015817356,
                -0.0087241465,
                -0.0068141264,
                -0.050538104,
                0.0063527534,
                -0.041478414,
                -0.035722543,
                0.012034419,
                -0.01546084,
                -0.022326589,
                -0.035799976,
                -0.0017825781,
                0.018545264,
                0.008530564,
                0.030482892,
                0.002416563,
                0.016919166,
                -0.0006033341,
                0.025346486,
                -0.013266898,
                -0.0011727912,
                0.032573592,
                0.017125655,
                0.0033812523,
                -0.011950533,
                -0.043620735,
                -0.016983693,
                0.023565521,
                0.032883324,
                0.022287872,
                -0.017370861,
                0.029037472,
                -0.018364588,
                0.011524649,
                0.014776846,
                0.00024903665,
                0.0035264397,
                -0.003111849,
                -0.036754984,
                0.00050291285,
                0.0016712678,
                0.026430551,
                -0.000017959374,
                0.006381791,
                0.0073238956,
                0.0034490062,
                -0.015499557,
                -0.0055074403,
                -0.020416569,
                0.0008348273,
                -0.011614989,
                -0.050976895,
                0.0078014005,
                -0.0015615707,
                -0.0044556386,
                0.030534515,
                -0.0050428407,
                -0.017848365,
                -0.02735975,
                -0.011421406,
                0.01122137,
                -0.01688045,
                -0.012266719,
                -0.01922926,
                0.015564085,
                0.029502071,
                0.0043975636,
                -0.0014147701,
                0.018480737,
                0.0030150574,
                0.0007448918,
                -0.01951318,
                -0.027075829,
                -0.005988172,
                -0.0078078536,
                -0.00018087924,
                -0.002072953,
                0.018958243,
                0.013292709,
                -0.0004488709,
                -0.010305076,
                0.02495932,
                -0.023836538,
                -0.020623058,
                0.011898911,
                0.019384125,
                -0.007994983,
                0.017435389,
                0.0013760536,
                0.028908417,
                -0.011550461,
                0.008956446,
                0.0370131,
                0.0025552975,
                0.013305615,
                -0.029166527,
                -0.002919879,
                0.008530564,
                -0.006430187,
                0.04163328,
                0.0035425716,
                0.013163654,
                -0.007401329,
                -0.005813947,
                -0.002534326,
                0.010317981,
                0.015680235,
                -0.015060768,
                0.016699772,
                0.009982438,
                0.020635962,
                0.009433952,
                -0.009814666,
                0.00028614007,
                -0.01780965,
                0.0029214923,
                -0.0035361187,
                0.0018342002,
                0.03809716,
                0.01224736,
                0.012066683,
                -0.0013179786,
                -0.031076547,
                -0.009182294,
                -0.016286796,
                -0.0074981204,
                0.023229977,
                0.044756424,
                -0.011744044,
                0.007291632,
                0.01734505,
                0.0015777026,
                -0.002158452,
                -0.037658375,
                0.0104470365,
                0.020752113,
                0.016390039,
                -0.01271196,
                -0.657873,
                0.002032623,
                -0.02661123,
                -0.02516581,
                0.019500276,
                0.0034683647,
                0.031334657,
                0.03115398,
                -0.005323536,
                -0.016093211,
                -0.014738129,
                0.00016283165,
                0.003916832,
                -0.0061333594,
                -0.025152903,
                -0.004762145,
                0.009233916,
                -0.0009638827,
                -0.023281598,
                -0.0024988358,
                0.022455644,
                0.013925081,
                -0.01951318,
                -0.001882596,
                0.04212369,
                -0.005597779,
                0.007936909,
                -0.018984053,
                -0.019590614,
                -0.0033747994,
                -0.0025714294,
                0.0027440412,
                0.025759464,
                -0.018777564,
                0.05038324,
                0.003587741,
                -0.023836538,
                0.0135379145,
                0.0047976356,
                0.04707942,
                0.013744403,
                -0.0006331782,
                0.033554412,
                -0.01853236,
                0.037451886,
                -0.00543646,
                0.043336812,
                -0.0008727373,
                -0.02223625,
                -0.0016059335,
                0.028185705,
                -0.040084615,
                -0.02244274,
                0.008220831,
                -0.008820939,
                -0.010382509,
                0.022636322,
                -0.02223625,
                -0.010982617,
                -0.01022119,
                0.0016712678,
                0.009156482,
                -0.011485933,
                -0.028082462,
                -0.00838215,
                0.033889957,
                -0.0080337,
                0.026766095,
                0.0035361187,
                -0.004594373,
                -0.0071819345,
                0.047931187,
                -0.022636322,
                -0.0016954656,
                0.0089241825,
                0.025372297,
                0.025540069,
                -0.0109697115,
                -0.019887442,
                0.013937986,
                0.003949096,
                0.015112391,
                -0.006297905,
                -0.016131928,
                0.0025746557,
                -0.014751036,
                -0.031515334,
                0.014867186,
                0.034999833,
                -0.009530744,
                -0.0029715013,
                0.008091776,
                -0.020339135,
                -0.033502787,
                -0.021294145,
                0.019926159,
                0.0033909313,
                0.022778284,
                0.008433772,
                -0.01830006,
                -0.018467832,
                -0.021061845,
                -0.0047556926,
                0.0051138215,
                -0.000076425014,
                -0.004494355,
                0.008136945,
                0.022068478,
                0.021861989,
                -0.021113468,
                -0.011911816,
                -0.015125296,
                -0.025643313,
                -0.019100204,
                -0.006833485,
                -0.033502787,
                0.018029043,
                0.008588639,
                0.029811803,
                -0.015654424,
                -0.007814306,
                0.013370142,
                0.017758027,
                -0.010905184,
                0.0225847,
                0.013525009,
                0.025101282,
                -0.015202729,
                -0.02579818,
                -0.0114923855,
                0.012950712,
                -0.0055848737,
                0.02223625,
                0.0020487548,
                0.030405458,
                0.019242164,
                0.027617862,
                -0.032676835,
                0.0031150754,
                -0.018519454,
                -0.028211517,
                -0.0067818626,
                -0.0027408146,
                -0.015525368,
                -0.0142993415,
                -0.021874895,
                0.01217638,
                -0.013292709,
                -0.046485767,
                -0.006685071,
                -0.032160614,
                -0.024481814,
                -0.048782952,
                0.02544973,
                -0.012647432,
                -0.005326763,
                -0.0054913084,
                -0.025049658,
                -0.004023303,
                -0.0030731324,
                0.018622698,
                0.012879732,
                -0.025462637,
                0.016325511,
                -0.0014026712,
                0.005646175,
                -0.010956806,
                0.002826314,
                0.012460302,
                -0.028572872,
                -0.0034812703,
                0.0036587215,
                -0.008814486,
                0.032573592,
                -0.0009638827,
                0.0033264037,
                -0.006491488,
                -0.018945336,
                -0.010543829,
                0.002150386,
                0.0006807674,
                -0.010285717,
                -0.019306691,
                -0.015499557,
                0.029785993,
                0.0042523765,
                0.021074751,
                0.012647432,
                -0.023320315,
                -0.007956267,
                -0.00093645847,
                0.0026730606,
                -0.04302708,
                -0.0069883512,
                -0.002124575,
                0.007459404,
                0.017370861,
                0.00040833943,
                -0.0045298454,
                0.0041943016,
                0.023023488,
                -0.010885825,
                0.008517658,
                -0.0034070632,
                -0.0004960568,
                -0.025978858,
                0.012486112,
                -0.00027262958,
                0.032238048,
                0.009330708,
                0.01052447,
                -0.032909133,
                -0.019435747,
                -0.007736873,
                0.02617244,
                0.030508703,
                -0.0027908236,
                0.02023589,
                -0.0024875433,
                0.030792626,
                0.011053598,
                0.011208464,
                -0.0010308302,
                -0.0028585778,
                -0.044369258,
                -0.008214378,
                0.025346486,
                0.0053461213,
                -0.018880809,
                -0.020584341,
                -0.014041231,
                0.0029602088,
                0.0050751045,
                0.00516867,
                -0.017099844,
                0.039542582,
                0.0045040343,
                -0.027514618,
                0.0026004668,
                0.007033521,
                0.025746558,
                -0.005020256,
                0.031928312,
                -0.014854279,
                -0.009582366,
                -0.0012816817,
                0.010505112,
                0.023759104,
                -0.0025520711,
                -0.012189285,
                -0.012989429,
                -0.005481629,
                0.00047669848,
                0.0056010056,
                -0.003295753,
                0.009195199,
                0.023991404,
                -0.0028747097,
                0.017448293,
                0.036961474,
                -0.012144116,
                -0.008214378,
                0.020506907,
                0.0034199688,
                -0.006359206,
                -0.0058010416,
                0.021668406,
                -0.012957165,
                0.018171005,
                0.012292529,
                0.017332144,
                0.011950533,
                0.022455644,
                -0.0006299518,
                -0.016867545,
                -0.021177996,
                -0.011189106,
                -0.019926159,
                -0.015706046,
                0.0072400095,
                -0.0015486651,
                -0.024417287,
                0.015873818,
                0.01783546,
                0.008943541,
                -0.013473387,
                -0.011098767,
                0.023513898,
                0.013473387,
                0.066076376,
                -0.0086854305,
                -0.008388603,
                -0.0109439,
                -0.01075677,
                0.0032893002,
                -0.01355082,
                0.029166527,
                0.0022278193,
                0.013834742,
                -0.04707942,
                -0.0045266193,
                -0.009750138,
                -0.01725471,
                -0.026998395,
                -0.0041265474,
                -0.011182653,
                -0.0024391476,
                -0.008595091,
                0.0011348812,
                -0.009782402,
                0.0078078536,
                -0.023733294,
                0.0005275141,
                -0.012395774,
                -0.010588998,
                -0.004794409,
                -0.02070049,
                0.008666072,
                0.010214737,
                -0.02686934,
                0.015357596,
                -0.0098082125,
                -0.018596888,
                -0.0050718784,
                0.09772077,
                -0.021836178,
                0.0023649407,
                0.013079768,
                -0.004965408,
                -0.006917371,
                -0.007162576,
                -0.0032973662,
                -0.0026311176,
                0.01902277,
                -0.014867186,
                -0.03928447,
                0.024920603,
                0.018158099,
                0.00938233,
                0.006330169,
                -0.0218878,
                -0.021861989,
                0.02484317,
                0.004158811,
                0.008104681,
                0.0014196098,
                -0.014841374,
                0.008330528,
                -0.009937268,
                0.0068980125,
                0.04067827,
                -0.0034167424,
                0.0070722373,
                -0.019797103,
                0.009975985,
                0.013195918,
                -0.008401508,
                0.012428038,
                -0.005513893,
                0.015899628,
                0.0049428227,
                0.0061010956,
                -0.004165264,
                -0.0035522506,
                0.041555848,
                0.0071496707,
                -0.004852484,
                -0.005078331,
                0.0040523405,
                0.0124538485,
                0.029373016,
                0.009362971,
                -0.016635245,
                -0.012511924,
                0.025062565,
                0.017138561,
                -0.014776846,
                -0.005455818,
                0.00233429,
                0.008027248,
                -0.0118924575,
                -0.011001975,
                -0.003923285,
                -0.0030924906,
                -0.00015879868,
                -0.0335286,
                0.0018212948,
                0.0004254796,
                -0.01948737,
                -0.01783546,
                -0.010311529,
                -0.0035845146,
                -0.026198251,
                -0.0034554591,
                0.014931713,
                0.009369425,
                -0.0049976716,
                -0.017370861,
                0.0463309,
                -0.0057332874,
                0.001259097,
                0.00118731,
                -0.0029924728,
                0.052319072,
                0.014673602,
                -0.014015419,
                -0.017641878,
                0.011640799,
                -0.013525009,
                0.021526445,
                -0.00021072332,
                -0.010279265,
                -0.010672884,
                0.023088016,
                0.0028053424,
                -0.011085861,
                0.015099485,
                0.014867186,
                -0.011369783,
                -0.016893355,
                -0.0092468215,
                0.0025649767,
                0.004768598,
                -0.017435389,
                -0.0024310816,
                -0.0040684724,
                0.015654424,
                -0.015473746,
                0.022339495,
                0.006343074,
                0.0014873638,
                -0.00040712956,
                0.0011647253,
                0.011614989,
                0.0110923145,
                -0.015215635,
                0.0012582904,
                0.0045266193,
                -0.025669124,
                0.013925081,
                0.00915003,
                0.034638476,
                -0.0056171375,
                -0.0015123683,
                -0.011724685,
                -0.027953405,
                0.011608535,
                0.029398827,
                0.026688663,
                -0.014015419,
                0.022571795,
                0.0020697264,
                -0.01627389,
                -0.0017341824,
                -0.009291991,
                -0.0006791542,
                0.0058333054,
                -0.024210798,
                -0.011621442,
                -0.0012897477,
                -0.01213121,
                -0.00008560005,
                -0.0344578,
                0.010440584,
                -0.021190902,
                0.0024520531,
                0.000386763,
                -0.012537735,
                -0.025372297,
                -0.017770933,
                -0.011234275,
                -0.009608177,
                -0.012911996,
                0.02423661,
                0.013525009,
                0.025101282,
                -0.03043127,
                -0.005497761,
                -0.014815563,
                -0.016467473,
                -0.017964516,
                -0.042923834,
                -0.009595271,
                0.04165909,
                0.02148773,
                0.013641159,
                0.012692601,
                0.009072596,
                -0.00022040248,
                0.009156482,
                0.0019116335,
                0.00077836553,
                -0.011873099,
                0.024572153,
                0.02070049,
                0.024868982,
                -0.015615707,
                -0.009369425,
                -0.028418005,
                0.029450448,
                0.010789034,
                0.01294426,
                -0.024856076,
                -0.01876466,
                0.013886364,
                0.0158351,
                -0.029889237,
                0.014183192,
                -0.033709276,
                -0.012924901,
                0.025733652,
                -0.009117766,
                -0.002016491,
                -0.002460119,
                0.003313498,
                -0.015770573,
                0.019810008,
                0.023139639,
                -0.0014680055,
                0.008704789,
                -0.0021213486,
                -0.031902503,
                0.008336981,
                0.023604238,
                0.017654782,
                -0.018803377,
                -0.004665354,
                -0.016325511,
                0.020829545,
                0.0042426973,
                -0.0028376062,
                -0.010556734,
                -0.01546084,
                0.009627535,
                -0.0018100024,
                -0.010234095,
                -0.013679875,
                -0.030147348,
                0.024223704,
                0.00013006368,
                -0.017370861,
                0.0087306,
                -0.034406178,
                -0.006439866,
                0.015112391,
                -0.019474464,
                0.022971867,
                -0.012298983,
                0.017512823,
                -0.0081627555,
                -0.014492924,
                -0.008143398,
                0.0050170296,
                -0.016648151,
                -0.0050654258,
                0.0034651381,
                0.0005404196,
                0.021152185,
                -0.0102082845,
                -0.0037135698,
                0.01331852,
                -0.004103963,
                -0.027695294,
                0.041039627,
                0.010214737,
                0.0022568568,
                -0.028882604,
                -0.021616785,
                -0.0016478765,
                0.011756949,
                -0.0023875255,
                -0.008866108,
                -0.020829545,
                -0.00800789,
                -0.026740285,
                0.027875973,
                -0.02081664,
                0.027953405,
                0.021500634,
                -0.023075111,
                0.011956985,
                -0.023939783,
                -0.01040832,
                -0.00091871334,
                -0.0041781696,
                -0.0024972225,
                -0.021810368,
                0.0072658206,
                0.00003067587,
                0.0018454926,
                -0.0038103615,
                0.005868796,
                0.014892996,
                0.020726303,
                -0.031437904,
                -0.008220831,
                0.013963797,
                0.0027666257,
                0.016260983,
                0.009924362,
                -0.018235533,
                -0.007375518,
                -0.008775769,
                -0.0033651204,
                -0.0037845504,
                0.016402945,
                0.014312247,
                -0.019397032,
                0.0018261343,
                -0.02070049,
                -0.0077175144,
                0.014015419,
                0.007911098,
                -0.0012776487,
                -0.008711241,
                -0.016196456,
                0.014609074,
                -0.006433413,
                0.015602801,
                0.010176021,
                0.019822914,
                0.026533796,
                -0.0042523765,
                0.0063624326,
                0.016906261,
                0.03804554,
                -0.018635605,
                -0.017641878,
                -0.0014042844,
                -0.012344152,
                0.030224781,
                0.003523213,
                -0.0080337,
                -0.003011831,
                -0.00082192174,
                0.006743146,
                -0.0016454568,
                0.010840656,
                -0.0004726655,
                0.007762684,
                -0.0056010056,
                0.01844202,
                -0.024752831,
                0.003739381,
                -0.01797742,
                -0.004091057,
                0.02081664,
                -0.012505471,
                0.023204166,
                0.014118664,
                -0.0074852146,
                0.011389142,
                -0.021642596,
                -0.012369962,
                -0.020106835,
                0.023746198,
                0.0034651381,
                -0.0036425896,
                -0.0039103795,
                -0.019126015,
                -0.021616785,
                0.02195233,
                -0.0010590611,
                -0.026920963,
                -0.014531641,
                0.02435276,
                0.009188746,
                0.01946156,
                -0.0071109543,
                0.0069496348,
                0.0081240395,
                -0.0026875793,
                -0.044369258,
                -0.03587741,
                0.00014942199,
                0.031541146,
                -0.0026504758,
                -0.045504943,
                -0.028082462,
                0.012789393,
                -0.045530755,
                -0.0004928304,
                -0.003136047,
                0.024559248,
                0.029192338,
                0.007433593,
                -0.0066399015,
                0.011866647,
                -0.0039942656,
                0.015899628,
                -0.011995702,
                0.017332144,
                0.022029761,
                -0.030224781,
                0.031205602,
                0.011802119,
                -0.014118664,
                0.004768598,
                0.010027607,
                0.0041943016,
                -0.017783837,
                0.023023488,
                -0.0017938705,
                -0.006281773,
                0.006265641,
                0.020455286,
                0.008136945,
                0.015060768,
                0.012705507,
                -0.020894073,
                -0.0050912364,
                0.030250592,
                -0.011956985,
                -0.010350245,
                0.02321707,
                -0.009846929,
                0.0068205795,
                0.009059691,
                -0.015512463,
                0.0012695828,
                0.013170106,
                -0.025307769,
                0.012466755,
                0.014338058,
                -0.008982258,
                0.0035006285,
                -0.0037038908,
                -0.004558883,
                -0.003923285,
                0.008646714,
                -0.035387,
                -0.006556016,
                -0.012111852,
                -0.0342255,
                -0.0065624686,
                -0.022262061,
                0.033683468,
                -0.0054493654,
                -0.0015785092,
                0.01804195,
                -0.030121537,
                -0.0024891566,
                -0.002474638,
                0.026133724,
                0.008311169,
                0.011769855,
                -0.011305256,
                0.0038716628,
                -0.02244274,
                -0.0077239675,
                0.008511205,
                -0.00647213,
                -0.010195379,
                0.0005577614,
                0.0023891386,
                -0.03569673,
                -0.024339853,
                0.005678439,
                -0.0051235007,
                -0.0043298095,
                0.20225567,
                0.022984771,
                -0.01557699,
                0.006672166,
                0.0063624326,
                0.024275325,
                0.011660158,
                0.0101308515,
                -0.020894073,
                0.003655495,
                -0.033554412,
                0.0096856095,
                0.0033393092,
                -0.0042136596,
                0.004836352,
                -0.039800692,
                -0.020726303,
                -0.04276897,
                0.0073238956,
                -0.014131569,
                -0.005910739,
                0.0062688673,
                0.00440079,
                -0.0120924935,
                0.016093211,
                -0.004407243,
                -0.012034419,
                0.005597779,
                0.028108273,
                0.037426073,
                0.0029505298,
                0.0034651381,
                0.018519454,
                0.00032364682,
                0.0027795313,
                0.00066463544,
                -0.0031247546,
                0.014996241,
                -0.0027666257,
                0.0026908056,
                -0.014157381,
                0.0042878664,
                -0.007788495,
                0.009253275,
                0.012686148,
                0.029811803,
                -0.009717873,
                -0.0038200407,
                -0.002934398,
                0.016441662,
                -0.017874178,
                -0.017357955,
                0.008808033,
                0.0055009876,
                0.0050008977,
                0.011840835,
                0.032238048,
                0.0037167964,
                0.019693859,
                0.012866826,
                0.0050008977,
                0.023436466,
                -0.025462637,
                -0.004149132,
                -0.0015825422,
                0.010324434,
                -0.018454926,
                0.015047863,
                0.029708559,
                0.022984771,
                0.0012445783,
                -0.01879047,
                -0.02493351,
                0.0034651381,
                0.011318161,
                -0.004123321,
                0.039981373,
                -0.000058780715,
                0.0153834075,
                0.012034419,
                -0.008930636,
                -0.003471591,
                -0.0007848183,
                0.0072980844,
                0.023204166,
                -0.007220651,
                0.023139639,
                0.008278905,
                0.017874178,
                0.0018454926,
                -0.000017417442,
                -0.021152185,
                0.0070786905,
                -0.0130152395,
                -0.013925081,
                0.01601578,
                0.0039394167,
                0.010505112,
                -0.0021891028,
                0.007459404,
                0.015473746,
                -0.016570717,
                0.012866826,
                -0.03567092,
                -0.014144475,
                -0.00059970445,
                0.010672884,
                0.015357596,
                0.0038619838,
                -0.008040153,
                0.0014446141,
                -0.016390039,
                0.00665926,
                -0.011847288,
                0.01830006,
                0.01922926,
                0.043672357,
                -0.041968826,
                0.002877936,
                -0.024210798,
                0.009195199,
                -0.018713037,
                -0.0067818626,
                0.015602801,
                0.0046395427,
                -0.021126373,
                -0.015370501,
                0.0124538485,
                0.010027607,
                -0.035541866,
                0.019771291,
                0.010847108,
                0.012873279,
                0.021784557,
                -0.011460122,
                -0.00010168156,
                0.022378212,
                -0.027462995,
                -0.017009506,
                0.0022649227,
                0.009588818,
                -0.00037103434,
                -0.0041943016,
                -0.0052235182,
                -0.001397025,
                -0.0057590986,
                -0.018958243,
                -0.0018890488,
                -0.006749599,
                0.0073690647,
                -0.0044685444,
                -0.017357955,
                0.0035070812,
                -0.041065436,
                0.013950892,
                -0.021500634,
                -0.025669124,
                -0.003523213,
                0.0076271757,
                0.009575913,
                0.020055214,
                -0.019087298,
                0.024159176,
                -0.0051428587,
                -0.011989249,
                0.013208823,
                -0.16230011,
                0.03019897,
                0.018119382,
                -0.005965587,
                0.019074392,
                -0.0024794773,
                0.0118537415,
                -0.0060172095,
                -0.010621262,
                0.008478941,
                0.021668406,
                -0.024804452,
                -0.027669484,
                -0.017848365,
                0.0023972045,
                -0.018390398,
                -0.023617143,
                -0.0002365344,
                0.0007001257,
                0.0039845863,
                0.0114536695,
                -0.0111310305,
                0.034328744,
                -0.011272992,
                -0.0023955915,
                0.013692781,
                -0.00283438,
                0.021913612,
                0.018880809,
                -0.027075829,
                -0.0116537055,
                -0.011550461,
                0.029992482,
                0.016957883,
                0.017964516,
                0.015951252,
                0.002418176,
                0.000050739956,
                -0.008582186,
                0.010072776,
                0.016131928,
                -0.0032441309,
                0.009537196,
                0.0041394527,
                -0.004149132,
                0.01366697,
                0.015977062,
                -0.0045685624,
                0.007788495,
                -0.013628253,
                0.030250592,
                -0.011737592,
                0.019590614,
                0.0042265654,
                0.009279085,
                0.014996241,
                -0.006039794,
                0.017861271,
                0.026559606,
                -0.013383048,
                -0.015215635,
                0.009763043,
                -0.012763582,
                -0.019797103,
                0.0017406351,
                -0.030379647,
                0.014686508,
                -0.005794589,
                -0.015654424,
                0.026211157,
                -0.0021600653,
                -0.010150209,
                -0.0015373728,
                0.001756767,
                0.022881526,
                0.007033521,
                -0.016338417,
                0.0054171016,
                0.0076400815,
                -0.011447216,
                -0.00903388,
                0.029269772,
                -0.053170837,
                -0.00048032816,
                -0.0011735978,
                0.005930097,
                0.00026758836,
                -0.009827571,
                -0.01890662,
                -0.006530205,
                0.012757129,
                -0.028882604,
                -0.018971147,
                -0.005068652,
                0.02756624,
                0.023242883,
                0.032238048,
                0.013118484,
                -0.0031102358,
                -0.0016841734,
                0.008446678,
                0.0071819345,
                -0.034173876,
                0.012537735,
                0.032521967,
                0.040988006,
                0.0108922785,
                0.017938705,
                0.013963797,
                -0.0053912904,
                -0.0061914343,
                -0.030405458,
                0.009956626,
                0.025385203,
                -0.01844202,
                0.012034419,
                0.01780965,
                -0.032780077,
                0.0028150214,
                0.003974907,
                0.019964876,
                -0.016699772,
                -0.03946515,
                0.04607279,
                -0.0038297197,
                -0.025204524,
                -0.091939084,
                -0.026314402,
                0.027617862,
                -0.0069238236,
                -0.021823272,
                0.046563197,
                -0.005036388,
                -0.005823626,
                0.030173158,
                0.018106477,
                -0.017177278,
                -0.016054496,
                -0.0017083712,
                -0.0035554771,
                0.0370131,
                -0.0061494913,
                0.012499019,
                0.0057010236,
                -0.028392194,
                -0.013847647,
                -0.023333222,
                -0.028418005,
                -0.014028325,
                -0.014673602,
                -0.016609434,
                0.013370142,
                -0.032005746,
                0.0017632198,
                0.022752471,
                0.032160614,
                0.0067883157,
                -0.0071367654,
                0.0013413698,
                -0.0022697623,
                0.0074077817,
                0.016196456,
                -0.0033489882,
                -0.0026069197,
                0.006717335,
                -0.033657655,
                -0.010924542,
                0.017654782,
                -0.0026569287,
                -0.03518051,
                0.022597605,
                -0.018016137,
                -0.010356698,
                0.008014342,
                -0.0019842272,
                0.001363148,
                0.011866647,
                0.008782222,
                -0.017616065,
                -0.013421765,
                0.005339668,
                0.01795161,
                -0.025978858,
                0.001167145,
                -0.014428397,
                -0.013628253,
                -0.007646534,
                -0.0058429847,
                -0.026069196,
                0.0010195379,
                0.010498659,
                0.0052654613,
                -0.019822914,
                0.010627715,
                0.017099844,
                -0.02174584,
                -0.021539351,
                -0.0039007002,
                -0.01052447,
                0.004420148,
                -0.02640474,
                -0.0068463907,
                -0.012640979,
                -0.008059512,
                0.02368167,
                -0.02128124,
                -0.032651022,
                -0.032521967,
                0.005604232,
                -0.003094104,
                -0.010576093,
                0.02400431,
                -0.0067689572,
                -0.011389142,
                -0.0038361726,
                -0.02172003,
                0.023359032,
                0.03799392,
                -0.0115311025,
                -0.012602262,
                0.007969173,
                0.019939063,
                0.0142993415,
                -0.01850655,
                -0.004984766,
                0.019306691,
                -0.031747635,
                -0.014415491,
                -0.07268402,
                -0.0014139635,
                0.008879013,
                0.0013760536,
                0.013705687,
                -0.030508703,
                0.015848007,
                -0.0045427512,
                0.0096856095,
                -0.007343254,
                -0.030792626,
                -0.020571435,
                0.023229977,
                0.016390039,
                -0.0071367654,
                -0.015318879,
                0.01774512,
                0.0135379145,
                0.011137484,
                0.007227104,
                0.0031053962,
                -0.00547195,
                0.029915048,
                0.011815025,
                -0.02461087,
                0.003723249,
                -0.0070077097,
                0.027230695,
                -0.007175482,
                0.00961463,
                0.024920603,
                -0.020597246,
                0.012234455,
                0.038226217,
                0.0028214743,
                -0.04209788,
                -0.009963079,
                0.030379647,
                0.010059871,
                0.0074723093,
                -0.037297018,
                -0.014338058,
                0.026456363,
                -0.019384125,
                0.0043362626,
                0.02177165,
                0.019900346,
                0.008504753,
                0.017203089,
                -0.0066205435,
                0.014415491,
                0.009517838,
                -0.00080296677,
                -0.020494003,
                -0.0049879923,
                0.0015631838,
                0.008704789,
                -0.013808931,
                -0.015009146,
                -0.02102313,
                0.03019897,
                0.012111852,
                0.005010577,
                -0.0018890488,
                0.028624494,
                -0.025101282,
                -0.0104470365,
                -0.0072400095,
                0.0086854305,
                -0.0026569287,
                -0.02177165,
                -0.0020697264,
                0.04777632,
                0.00028412358,
                0.005794589,
                -0.013266898,
                0.00800789,
                -0.008491847,
                -0.031257223,
                0.007220651,
                0.02852125,
                -0.005078331,
                -0.023707483,
                0.010253454,
                0.0056010056,
                -0.014131569,
                -0.03541281,
                0.011776308,
                -0.004578241,
                0.0036329103,
                -0.0033264037,
                -0.0077175144,
                0.008304717,
                0.0160674,
                -0.00344578,
                0.007459404,
                0.02377201,
                -0.005078331,
                0.001167145,
                0.017564444,
                -0.0016970789,
                -0.011524649,
                -0.010376057,
                -0.022571795,
                -0.026030479,
                -0.0068980125,
                -0.021061845,
                -0.011763402,
                -0.014712319,
                0.018932432,
                0.021307051,
                0.038019728,
                0.025462637,
                0.008549922,
                -0.034380365,
                0.00012905542,
                -0.006497941,
                0.022262061,
                -0.019900346,
                0.011466575,
                -0.010595451,
                0.019822914,
                -0.0038458519,
                -0.008214378,
                0.0043459414,
                0.012286076,
                -0.012324793,
                -0.0069560874,
                -0.0060043037,
                -0.005194481,
                0.0070141624,
                0.012002155,
                -0.012763582,
                0.01331852,
                -0.0058074943,
                0.016002873,
                0.024920603,
                0.015099485,
                -0.01818391,
                0.04635671,
                0.019590614,
                0.000987274,
                0.009414594,
                -0.008156303,
                0.028185705,
                -0.004694391,
                0.0026246647,
                -0.025230337,
                -0.023410656,
                -0.00004882429,
                0.028727738,
                -0.007994983,
                -0.018713037,
                -0.0153834075,
                -0.008440225,
                -0.017861271,
                -0.008395055,
                -0.0050912364,
                0.023642955,
                0.020171363,
                -0.013195918,
                0.01289909,
                0.0053848377,
                -0.020752113,
                -0.0034683647,
                0.0022133007,
                0.0017535406,
                0.0011106833,
                -0.01702241,
                0.021358673,
                0.0048589366,
                -0.013331425,
                -0.022597605,
                -0.013473387,
                -0.00074085884,
                -0.008304717,
                0.006559242,
                0.013963797,
                0.038690817,
                -0.000021979753,
                0.057971697,
                -0.028108273,
                -0.017009506,
                -0.009388782,
                -0.0062946784,
                -0.004826673,
                -0.00009054045,
                -0.01876466
            ],   
    },
    {
    "@search.action": "upload",
    "HotelId": "2",
    "HotelName": "Twin Dome Motel",
    "Description": "The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Category": "Boutique",
    "Tags": [ "pool", "free wifi", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1979-02-18T00:00:00Z",
    "Rating": "3.60",
    "Address": {
        "StreetAddress": "140 University Town Center Dr",
        "City": "Sarasota",
        "StateProvince": "FL",
        "PostalCode": "34243",
        "Country": "USA"
        },
        "HotelNameVector": [
                -0.01341688,
                0.011854271,
                0.00070721505,
                0.00006256538,
                -0.027493827,
                0.033030313,
                0.00094884686,
                -0.013093581,
                0.012696194,
                -0.019532606,
                -0.0029585164,
                0.01906113,
                0.013443822,
                -0.0025493419,
                -0.025149915,
                0.00838555,
                0.009146648,
                -0.006331259,
                0.0058766203,
                -0.0124873975,
                0.0031875193,
                -0.0073483014,
                0.001624069,
                -0.016111033,
                -0.0070654154,
                -0.026672112,
                0.005523013,
                -0.005620676,
                -0.004586795,
                0.021795696,
                0.010965201,
                -0.035077866,
                -0.0010229361,
                -0.016178386,
                -0.0039166245,
                0.0063918773,
                0.00019006405,
                0.00163333,
                0.03564364,
                -0.011726299,
                0.012884785,
                -0.015073784,
                0.016555568,
                -0.018199,
                -0.026106339,
                0.005677927,
                0.018562712,
                0.020583326,
                -0.02453026,
                0.0073213596,
                -0.0036606798,
                0.01660945,
                -0.019155426,
                -0.022738649,
                -0.0013799113,
                -0.009779774,
                0.003977243,
                0.008991734,
                -0.007961221,
                -0.034673743,
                0.0135583235,
                -0.03726013,
                -0.03637106,
                0.009941423,
                -0.008068987,
                0.0041119503,
                -0.008082458,
                0.0098201865,
                0.008129606,
                -0.0142924795,
                0.036263295,
                0.019721197,
                -0.009544035,
                -0.04836004,
                0.0013984336,
                -0.015599144,
                -0.013120524,
                -0.0086010825,
                0.016097562,
                0.018279826,
                0.016501686,
                -0.021795696,
                -0.035455048,
                0.027224412,
                0.022523116,
                0.009267885,
                -0.012709665,
                0.024988266,
                -0.0090995,
                -0.022981122,
                0.013039699,
                0.037367895,
                0.010830494,
                0.029393205,
                -0.019290132,
                0.012143893,
                -0.005768854,
                0.01322829,
                -0.005421982,
                0.00546913,
                -0.0034788246,
                0.019640373,
                -0.008358609,
                -0.005674559,
                -0.021108685,
                -0.006597306,
                0.0020643945,
                -0.015275845,
                0.0019212676,
                -0.017242577,
                -0.008628024,
                0.011591592,
                0.018778244,
                -0.050784778,
                -0.011517502,
                0.0036842537,
                0.0018202369,
                0.0065737325,
                -0.008223901,
                -0.006610777,
                0.0061931834,
                0.021768752,
                0.024301257,
                -0.018670477,
                0.020179203,
                0.015073784,
                -0.026254518,
                -0.0012106849,
                0.038364735,
                0.014723544,
                0.01366609,
                -0.0024870397,
                0.00961139,
                -0.0031757324,
                -0.0038290645,
                -0.005489336,
                -0.00433085,
                -0.011167263,
                -0.033272784,
                0.002844015,
                0.029635679,
                0.03777202,
                -0.011591592,
                0.017983468,
                0.005021227,
                0.007826514,
                -0.0107429335,
                -0.008897439,
                0.01147709,
                -0.018939894,
                -0.003320543,
                -0.04782121,
                0.000102135746,
                0.029878153,
                0.016569039,
                0.037502605,
                -0.008755996,
                0.021485867,
                -0.02950097,
                0.009402593,
                -0.015666498,
                0.023897134,
                0.006843148,
                0.017229106,
                0.016717218,
                0.023546893,
                0.0070317383,
                0.027372591,
                -0.02365466,
                -0.020165732,
                0.011173998,
                -0.0036438415,
                -0.042729262,
                0.003147107,
                -0.009658537,
                0.024409022,
                -0.018912952,
                0.010271457,
                -0.027076233,
                -0.022159405,
                -0.030605573,
                0.007819778,
                0.015774263,
                0.035589755,
                -0.0003618163,
                -0.013147465,
                0.0041658334,
                -0.0015398766,
                0.010480254,
                -0.009005205,
                0.024045313,
                0.010608226,
                0.008015104,
                0.004388101,
                -0.66685665,
                0.003451883,
                -0.0072270646,
                -0.014871722,
                -0.009214002,
                0.020933567,
                0.03001286,
                0.024247374,
                -0.022711707,
                -0.018670477,
                -0.012622105,
                -0.005203082,
                0.019694256,
                0.009517094,
                -0.01923625,
                -0.020017555,
                -0.002702572,
                0.005499439,
                0.008419227,
                -0.0016038627,
                0.001437162,
                0.013672825,
                -0.011759976,
                0.0011643791,
                0.023169713,
                0.0056105726,
                0.015976325,
                0.009106236,
                -0.020610267,
                0.0076109814,
                -0.007826514,
                0.031359937,
                0.006166242,
                -0.026658641,
                0.042675376,
                0.00060913106,
                -0.022321055,
                0.019559547,
                -0.011813859,
                0.035050925,
                -0.0073348307,
                -0.013538117,
                0.025661804,
                0.018050823,
                0.0105543425,
                0.007799572,
                0.029635679,
                -0.023519952,
                0.009759568,
                -0.03173712,
                0.030767223,
                -0.005620676,
                -0.0013108737,
                0.014723544,
                0.002310236,
                0.0077928365,
                0.038553324,
                -0.0044049397,
                -0.0063582,
                0.006395245,
                -0.021027861,
                -0.0116454745,
                -0.02613328,
                -0.024099195,
                -0.009968365,
                -0.0033912647,
                -0.021378102,
                0.011497296,
                0.00031719438,
                0.002047556,
                -0.027480356,
                0.03585917,
                -0.0044891317,
                0.009759568,
                0.008156547,
                0.037529547,
                0.025055619,
                -0.018212471,
                0.000315721,
                -0.00507511,
                0.0048966226,
                -0.012662518,
                -0.015518319,
                -0.01675763,
                0.023573836,
                -0.0321143,
                -0.02454373,
                0.024018371,
                0.008304725,
                -0.009160119,
                0.013639147,
                0.011483826,
                0.009927953,
                -0.016542098,
                -0.003563017,
                0.01766017,
                -0.008553934,
                0.015733851,
                0.0381492,
                -0.020893153,
                -0.006610777,
                -0.013847944,
                0.017282989,
                0.004839372,
                0.014723544,
                -0.008776202,
                -0.0030376571,
                0.023856722,
                0.028908258,
                -0.00028351747,
                -0.023977958,
                -0.010318604,
                -0.022509646,
                -0.02380284,
                0.00971242,
                -0.037987553,
                0.017552404,
                0.008345137,
                0.013517911,
                0.0028608532,
                0.015302787,
                -0.016730689,
                -0.00037234032,
                0.00010771349,
                0.016811512,
                -0.002306868,
                0.013457293,
                -0.021849578,
                -0.004169201,
                -0.006220125,
                -0.019748138,
                0.022455763,
                0.018145118,
                -0.005748648,
                0.01015022,
                -0.002025666,
                0.009759568,
                -0.013598735,
                0.012999287,
                -0.011658945,
                -0.033245843,
                -0.016474742,
                -0.0046339426,
                0.00542535,
                0.0034165224,
                -0.03319196,
                -0.013039699,
                0.001023778,
                -0.022011228,
                -0.012164099,
                0.0031993063,
                -0.006816206,
                -0.016973162,
                0.00472487,
                -0.007981427,
                -0.014912135,
                0.016380448,
                -0.0022024699,
                -0.02365466,
                -0.024166549,
                0.017889174,
                -0.003670783,
                -0.028881317,
                -0.0020391368,
                -0.0064828047,
                0.0061595063,
                0.001426217,
                0.005351261,
                0.0020963876,
                -0.021943873,
                -0.01448107,
                -0.012339219,
                -0.0062908465,
                0.007752424,
                -0.00085286767,
                0.0029012656,
                0.0052637006,
                -0.02558098,
                0.004314012,
                0.00749648,
                -0.0021283806,
                0.02047556,
                -0.0144676,
                -0.0036741507,
                0.021364631,
                -0.008809879,
                0.038553324,
                0.014777428,
                -0.013214819,
                0.0075570983,
                -0.0027194102,
                0.011578121,
                -0.00595071,
                0.008331667,
                0.014858251,
                0.00829799,
                0.027749771,
                0.00631442,
                -0.00026962574,
                0.019896317,
                0.011160527,
                0.011598327,
                0.009894275,
                -0.0030006126,
                0.012292071,
                -0.03472763,
                0.0028558017,
                -0.041274417,
                0.030147567,
                -0.0015112512,
                -0.0058732526,
                -0.054125525,
                -0.020623738,
                -0.0018758038,
                0.0049201963,
                0.012810696,
                0.0014523166,
                0.025890807,
                0.0008128764,
                -0.005704868,
                -0.00048242172,
                0.013699766,
                0.012204511,
                0.012743342,
                -0.009375651,
                -0.007489744,
                0.0075773043,
                -0.0023270743,
                0.024907442,
                -0.012581693,
                -0.017794877,
                0.0019364222,
                -0.0029164203,
                0.014036535,
                0.01395571,
                0.006688234,
                0.003017451,
                -0.0016350138,
                0.0038425352,
                0.0010877642,
                0.026672112,
                0.011632004,
                0.020947037,
                -0.016569039,
                0.0023775897,
                0.0055701607,
                0.026052456,
                -0.0014396878,
                0.001558399,
                -0.010055925,
                -0.017740995,
                0.003734769,
                0.0066074096,
                0.016892336,
                0.008392286,
                0.0042197164,
                0.000031808893,
                0.006496276,
                0.013255231,
                0.02753424,
                0.011524238,
                -0.0093150325,
                -0.004607001,
                -0.009247679,
                0.008284519,
                -0.023573836,
                0.015221963,
                -0.0153431995,
                0.01297908,
                -0.008129606,
                -0.010608226,
                0.007725483,
                0.0167711,
                -0.015329729,
                -0.0024348404,
                0.0029955609,
                -0.0013024544,
                0.018926423,
                0.005139096,
                0.014723544,
                -0.00030961708,
                -0.017902644,
                0.026537403,
                -0.0107227275,
                -0.004276967,
                -0.01766017,
                -0.0033272784,
                0.004223084,
                0.0039705075,
                0.027493827,
                0.013268702,
                0.0003908626,
                0.0125278095,
                -0.010931524,
                -0.016151445,
                -0.016515156,
                -0.007988162,
                -0.034458213,
                0.002362435,
                0.01366609,
                0.021243393,
                0.011241352,
                -0.026335342,
                -0.021054802,
                0.0130329635,
                -0.01817206,
                -0.0015499797,
                -0.016232269,
                -0.023021534,
                -0.028558018,
                0.011813859,
                -0.0116050625,
                -0.006853251,
                0.0028911626,
                -0.02153975,
                -0.021378102,
                -0.017013574,
                -0.03903827,
                0.02594469,
                -0.0048090625,
                -0.0014438974,
                0.0064120833,
                -0.008863762,
                0.013093581,
                0.09375651,
                -0.014211656,
                0.022832943,
                0.008170018,
                -0.010695786,
                -0.00008340297,
                -0.024786204,
                -0.036398,
                -0.0013268702,
                -0.0018353915,
                -0.02260394,
                -0.003768446,
                0.00864823,
                0.0085808765,
                0.017040515,
                0.0010002041,
                -0.022482704,
                -0.0051458315,
                0.008163283,
                0.009220737,
                -0.029905094,
                0.009894275,
                -0.003919992,
                0.033649966,
                -0.0051559345,
                -0.018616594,
                0.019303603,
                0.033596084,
                -0.006294214,
                -0.019828964,
                0.0076715997,
                -0.017148282,
                0.0151546085,
                0.01957302,
                0.007132769,
                -0.012777019,
                0.0028507502,
                -0.015141138,
                0.019209309,
                -0.008136341,
                0.013901828,
                0.01394224,
                -0.0155317895,
                0.0041018473,
                0.020637209,
                -0.005203082,
                0.01535667,
                0.01518155,
                -0.0048292684,
                -0.01013675,
                0.032572307,
                0.0026318503,
                0.0026065926,
                -0.02645658,
                -0.014777428,
                0.0028709564,
                0.0043072766,
                -0.01850883,
                -0.009065824,
                0.00013133994,
                -0.022172876,
                -0.027143588,
                0.011429942,
                -0.0015171447,
                -0.015437495,
                -0.0508656,
                -0.00002024561,
                0.0080757225,
                -0.014508012,
                -0.031171346,
                -0.0016324881,
                -0.011638739,
                -0.02559445,
                -0.009631596,
                0.04760568,
                0.022738649,
                0.0073213596,
                -0.008984999,
                -0.0013235025,
                0.0131542,
                0.00023615932,
                -0.011443413,
                0.0028911626,
                -0.009079294,
                0.007025003,
                -0.0029248395,
                0.029958976,
                -0.013901828,
                -0.027803656,
                0.019802023,
                0.031279113,
                -0.01993673,
                0.018589653,
                -0.012837637,
                -0.0035697522,
                -0.0037886521,
                0.019344015,
                0.03351526,
                -0.024974795,
                0.004573324,
                0.033057254,
                -0.021081744,
                0.021122158,
                -0.0151546085,
                0.011726299,
                -0.009752832,
                -0.004280335,
                0.009362181,
                0.008419227,
                0.008506787,
                0.008264313,
                0.0074560675,
                -0.004169201,
                -0.009981835,
                -0.0012713033,
                0.01569344,
                -0.024233904,
                -0.010695786,
                -0.03707154,
                -0.012810696,
                0.006088785,
                -0.03884968,
                0.026685582,
                0.026712524,
                -0.0067118076,
                -0.023061946,
                0.030416984,
                -0.024786204,
                0.006041637,
                -0.0067926324,
                -0.010796817,
                0.002778345,
                0.013443822,
                -0.021243393,
                0.007839984,
                -0.02950097,
                -0.004913461,
                0.01093826,
                -0.004913461,
                -0.014548425,
                -0.01658251,
                0.0040816413,
                -0.01748505,
                0.0017562507,
                -0.0044924994,
                -0.009692214,
                -0.017727524,
                -0.013396674,
                0.0025072459,
                0.041840192,
                0.02294071,
                0.028800491,
                -0.00838555,
                -0.005903562,
                0.007159711,
                -0.050919484,
                -0.0335422,
                -0.008271049,
                0.002951781,
                0.0064289216,
                0.031171346,
                0.005213185,
                0.012453721,
                0.014023065,
                0.001110496,
                0.008324931,
                -0.013275437,
                0.025486683,
                -0.027264824,
                0.007900602,
                0.019155426,
                0.029393205,
                0.024799675,
                0.032733954,
                -0.005176141,
                0.011059497,
                0.020435147,
                -0.0014497909,
                -0.0023910603,
                -0.02066415,
                0.008641494,
                -0.02187652,
                -0.0006878509,
                -0.011988979,
                -0.019761609,
                0.0046777227,
                0.026402695,
                -0.0045261765,
                -0.0065198494,
                0.008082458,
                -0.0031824678,
                -0.013928769,
                0.02028697,
                0.0036000614,
                0.012420043,
                0.0098403925,
                0.004583427,
                0.0038492705,
                -0.019155426,
                0.018050823,
                -0.008271049,
                -0.012844373,
                0.016366977,
                0.005859782,
                0.009557506,
                0.034997042,
                -0.0052637006,
                -0.022334525,
                0.0042331875,
                -0.027803656,
                -0.021229923,
                -0.0153431995,
                -0.044588227,
                -0.04078947,
                0.031925708,
                0.0053209513,
                -0.001699,
                0.019128483,
                -0.017431168,
                -0.024974795,
                -0.0014472652,
                -0.025796512,
                0.03249148,
                0.011140321,
                0.022132464,
                0.027695889,
                -0.019344015,
                -0.011598327,
                0.02719747,
                -0.011887949,
                -0.0021536383,
                0.006853251,
                0.02030044,
                0.012709665,
                -0.023358304,
                0.0102175735,
                0.01818553,
                -0.002483672,
                -0.024422493,
                0.012931933,
                0.018293297,
                -0.0050448007,
                -0.009065824,
                0.00507511,
                0.020354323,
                0.0142924795,
                -0.007954486,
                -0.0049471376,
                -0.029204614,
                -0.009685479,
                -0.035966936,
                0.0073146243,
                -0.030416984,
                -0.002756455,
                -0.0014194817,
                -0.0047821207,
                0.021337688,
                -0.025688745,
                0.0026183797,
                0.007132769,
                0.0006785897,
                0.049707115,
                -0.020515973,
                0.0038829474,
                -0.010567814,
                -0.0020812328,
                -0.018293297,
                0.014508012,
                -0.013376468,
                0.019842435,
                -0.0016636392,
                -0.008668436,
                0.0026958366,
                -0.0062436988,
                0.028719667,
                -0.008641494,
                -0.0024954588,
                0.0059439745,
                -0.015639557,
                -0.0066579245,
                0.0065366877,
                0.0057250746,
                -0.000021429563,
                -0.021620575,
                -0.026335342,
                -0.031494644,
                -0.010116543,
                -0.0013293959,
                -0.00080656196,
                -0.042163488,
                -0.0070856214,
                -0.00007261583,
                -0.011874477,
                -0.0054388205,
                0.009072559,
                -0.00066680275,
                0.017956527,
                0.033111136,
                -0.018616594,
                0.032060415,
                0.0070923567,
                0.036613535,
                -0.026254518,
                -0.0063548326,
                0.041382186,
                0.0012788806,
                0.014427187,
                -0.008223901,
                -0.0028170734,
                -0.020771917,
                0.00542535,
                -0.0015920758,
                -0.016905807,
                0.0059271357,
                0.024732322,
                0.0047215023,
                0.007725483,
                -0.014252068,
                -0.030443925,
                0.023264008,
                -0.008614553,
                0.019168897,
                0.0015120931,
                0.017525462,
                -0.00058555725,
                0.02645658,
                -0.0070856214,
                0.009490153,
                -0.010756404,
                -0.010520666,
                -0.018252885,
                0.006991326,
                -0.004839372,
                0.0024988265,
                -0.004748444,
                -0.0158955,
                0.00962486,
                0.014521482,
                -0.01801041,
                0.0014034852,
                0.021216452,
                0.012554752,
                -0.03405409,
                0.02295418,
                -0.02155322,
                -0.040843353,
                0.0089782635,
                -0.0144406585,
                -0.024624554,
                -0.019599961,
                -0.010918054,
                0.016528627,
                -0.011086438,
                -0.031494644,
                0.012056333,
                0.008035311,
                -0.021216452,
                0.004997653,
                0.006132565,
                0.0014354782,
                0.03389244,
                0.012918462,
                -0.014198184,
                0.016340036,
                0.0017714053,
                0.016703747,
                -0.01341688,
                0.0032043578,
                0.018980306,
                -0.025702216,
                -0.007914074,
                -0.0032211964,
                -0.023573836,
                -0.006654557,
                0.008358609,
                0.0072944183,
                0.013740178,
                0.011699358,
                -0.0020172468,
                -0.010399429,
                -0.0020526075,
                -0.008668436,
                -0.016784571,
                0.006863354,
                -0.018778244,
                -0.0019364222,
                -0.009880804,
                -0.014925606,
                0.010958466,
                -0.016973162,
                -3.420311e-7,
                0.0121506285,
                0.0053310543,
                -0.0055297483,
                -0.028558018,
                -0.007206858,
                0.019640373,
                -0.03144076,
                0.0017916115,
                0.017740995,
                0.0023136036,
                -0.02647005,
                0.014723544,
                0.0017579346,
                -0.0057116034,
                0.012460456,
                -0.04765956,
                -0.03173712,
                0.006775794,
                -0.013996123,
                -0.013127259,
                -0.0003736032,
                0.023035005,
                -0.029231556,
                0.02012532,
                0.0073752427,
                0.0021687928,
                -0.010507195,
                -0.006449128,
                0.013059905,
                -0.011941832,
                0.011288499,
                -0.01923625,
                0.0030123994,
                0.0035798554,
                0.0027648741,
                0.008877233,
                -0.012622105,
                -0.002864221,
                0.0025644966,
                0.013834474,
                -0.0044352487,
                -0.020812329,
                0.005650985,
                -0.011032555,
                0.0064558634,
                0.2228603,
                0.020542914,
                -0.01906113,
                0.037529547,
                0.003802123,
                0.018225942,
                0.025823453,
                -0.00845964,
                -0.0018471784,
                0.024476377,
                -0.03284172,
                -0.011066232,
                0.016663333,
                0.0052199205,
                0.0153431995,
                -0.009591184,
                -0.045315646,
                -0.02912379,
                -0.026523933,
                -0.010190632,
                -0.003950301,
                0.0022614044,
                0.0053209513,
                -0.00090843457,
                0.028153894,
                0.007152975,
                -0.0066276155,
                -0.0026166958,
                0.014790898,
                0.041624658,
                -0.009389122,
                -0.003919992,
                0.009496888,
                -0.00050389074,
                -0.0025914381,
                0.010513931,
                0.014427187,
                0.0029804064,
                0.012292071,
                0.009065824,
                -0.019465253,
                -0.020893153,
                -0.017229106,
                0.011254823,
                -0.0016964743,
                0.008688643,
                -0.010406164,
                -0.0075840396,
                0.015814677,
                0.009402593,
                -0.02203817,
                -0.007893867,
                -0.016232269,
                0.011275029,
                0.0073819784,
                -0.017969998,
                0.021054802,
                -0.006870089,
                -0.017390756,
                0.009038882,
                -0.002766558,
                0.031629354,
                -0.03316502,
                0.007988162,
                0.0044251457,
                0.0004836846,
                0.014319422,
                0.016232269,
                0.011436678,
                -0.03197959,
                0.011167263,
                -0.020435147,
                -0.011288499,
                -0.00038370627,
                -0.010614961,
                -0.021687929,
                0.02083927,
                0.0042735995,
                0.0051357285,
                0.018252885,
                -0.021660987,
                -0.0058766203,
                0.0019330545,
                0.0061494033,
                0.031279113,
                -0.018993776,
                0.028261662,
                -0.0060483725,
                0.009591184,
                0.009584448,
                -0.018589653,
                -0.01200245,
                -0.01235269,
                -0.009954894,
                -0.001426217,
                0.0072944183,
                -0.00891091,
                0.02261741,
                0.0016594296,
                0.005708236,
                -0.028019188,
                0.00029993497,
                0.017727524,
                -0.001427059,
                -0.01729646,
                -0.007678335,
                0.023856722,
                0.019869376,
                0.024597613,
                -0.007536892,
                -0.0015743955,
                -0.017256048,
                0.0075840396,
                0.011261558,
                -0.021957343,
                0.014454129,
                0.01483131,
                -0.017835291,
                -0.0043982044,
                -0.0010633484,
                0.023277478,
                -0.022900298,
                0.012238189,
                -0.0061022555,
                0.012662518,
                -0.0045699566,
                -0.0098403925,
                -0.00025847027,
                0.03437739,
                -0.027413003,
                -0.0034182062,
                -0.0055836313,
                0.0413283,
                0.021391572,
                -0.008506787,
                0.0037280337,
                0.02524421,
                -0.009240944,
                -0.0063918773,
                0.012473927,
                0.009934688,
                -0.002700888,
                0.028531076,
                0.0021435353,
                -0.015612614,
                -0.016622921,
                0.014319422,
                0.012238189,
                -0.023088887,
                -0.0019633637,
                -0.0033929485,
                -0.019855905,
                0.016232269,
                0.0006339678,
                0.011153792,
                -0.036856007,
                -0.028692726,
                -0.017269518,
                -0.0052805394,
                0.016986633,
                -0.017794877,
                0.00012618316,
                0.010042454,
                -0.0069576493,
                -0.024597613,
                0.016030207,
                -0.17425779,
                0.029581796,
                -0.00573181,
                -0.005826105,
                -0.0036472091,
                0.0073752427,
                0.03882274,
                0.0025308197,
                -0.026820289,
                -0.010810288,
                -0.0067185434,
                -0.00047737017,
                -0.016111033,
                0.018199,
                0.026160222,
                -0.004115318,
                -0.004021023,
                -0.01782182,
                -0.009160119,
                -0.01191489,
                0.0035966937,
                -0.014669661,
                0.011544444,
                -0.027291765,
                -0.002700888,
                0.013901828,
                -0.0036775183,
                0.03319196,
                0.009476682,
                -0.004377998,
                -0.014454129,
                -0.009274621,
                0.008062252,
                0.026645169,
                0.014279009,
                -0.019303603,
                -0.011173998,
                0.009961629,
                -0.014844781,
                0.00084065984,
                -0.006405348,
                0.01032534,
                0.019909788,
                -0.029716503,
                -0.014050006,
                0.034835394,
                -0.022334525,
                0.0017949792,
                0.017027045,
                0.0064996434,
                0.022671295,
                -0.033057254,
                0.0028995818,
                0.0050380654,
                0.023775898,
                0.0047989595,
                -0.0011155476,
                0.014777428,
                0.014804369,
                0.012399837,
                -0.024341669,
                -0.025675274,
                0.013147465,
                -0.006371671,
                -0.013834474,
                -0.049814884,
                -0.009779774,
                -0.0061729774,
                -0.026079398,
                0.017444639,
                -0.010668844,
                -0.009375651,
                0.011328911,
                -0.023102358,
                0.012473927,
                0.008755996,
                -0.021809166,
                0.0040378612,
                0.017889174,
                -0.03615553,
                -0.032949485,
                0.051081132,
                -0.038230024,
                0.015801204,
                -0.014939076,
                -0.0010305134,
                -0.013201348,
                0.01077661,
                0.020637209,
                0.006897031,
                0.006220125,
                -0.0125278095,
                0.012386367,
                -0.015908971,
                0.018387591,
                0.024584142,
                0.0074291257,
                0.019761609,
                0.012258395,
                -0.01691928,
                -0.021660987,
                -0.006614145,
                -0.026025515,
                0.002702572,
                0.0065535265,
                0.00926115,
                0.01713481,
                0.026416166,
                0.002461782,
                0.0021586898,
                0.013181142,
                -0.008803144,
                0.00786019,
                0.0025038782,
                0.0010953414,
                -0.012985815,
                -0.0049841823,
                0.0014455813,
                0.01499296,
                0.006843148,
                0.037125424,
                -0.0057890606,
                -0.012406573,
                -0.0025914381,
                -0.010183897,
                -0.0020121953,
                -0.07182611,
                -0.013490969,
                0.014050006,
                0.015666498,
                -0.027089704,
                0.034808453,
                -0.010332075,
                0.031332996,
                -0.021634046,
                0.017404227,
                -0.001133228,
                -0.009874069,
                -0.009207266,
                -0.003061231,
                0.024799675,
                -0.006425554,
                0.003344117,
                -0.011315441,
                -0.02896214,
                0.010561079,
                0.011396266,
                0.0012814064,
                -0.0058429437,
                0.0032312993,
                -0.011072967,
                -0.024409022,
                -0.023870192,
                0.036236353,
                0.013133994,
                0.011968773,
                0.004886519,
                -0.0070923567,
                0.024301257,
                -0.016191857,
                -0.014359834,
                -0.007449332,
                -0.017202165,
                0.009214002,
                0.014763957,
                -0.005805899,
                -0.013275437,
                -0.003812226,
                -0.021634046,
                -0.026254518,
                -0.017565874,
                -0.009308297,
                -0.010278192,
                0.020973979,
                0.0030645987,
                -0.015585673,
                0.0119822435,
                0.008964793,
                -0.0052805394,
                -0.007260741,
                0.00918706,
                0.023102358,
                0.022725178,
                0.022563528,
                -0.019707726,
                0.009631596,
                0.0012510972,
                0.019977141,
                -0.0050245947,
                0.002561129,
                0.014642719,
                -0.0019296869,
                -0.014710073,
                0.0007918283,
                0.005553322,
                -0.010183897,
                -0.0044924994,
                -0.015141138,
                -0.004586795,
                -0.010648638,
                -0.013315849,
                0.001089448,
                -0.030228393,
                -0.025365448,
                0.010372488,
                -0.010399429,
                0.0047349734,
                -0.026052456,
                0.004596898,
                0.013484234,
                -0.0046272073,
                0.011894684,
                0.008991734,
                -0.014090418,
                0.0055263806,
                -0.027305236,
                0.0042735995,
                0.0199502,
                0.013490969,
                -0.011288499,
                -0.019909788,
                -0.0026806819,
                0.004324115,
                -0.0125076035,
                -0.004421778,
                0.0020172468,
                -0.033272784,
                -0.022118993,
                -0.09289438,
                -0.009665272,
                0.00969895,
                -0.007435861,
                0.011800389,
                -0.017444639,
                0.0024146342,
                -0.0055600577,
                0.0038593737,
                -0.00084992097,
                -0.011396266,
                0.0038795797,
                0.026820289,
                -0.005085213,
                -0.030390041,
                -0.028746609,
                0.024274316,
                0.010284928,
                0.009214002,
                0.022267172,
                0.017579345,
                0.008877233,
                0.011012349,
                -0.0011113379,
                0.00042538144,
                0.0038694767,
                0.0027278296,
                0.017687112,
                -0.004334218,
                0.033407494,
                0.00046432036,
                -0.028019188,
                0.006381774,
                0.034862336,
                -0.028692726,
                -0.020866212,
                -0.03971181,
                -0.019640373,
                -0.00104651,
                -0.005465762,
                -0.042594552,
                -0.030901931,
                -0.0019566284,
                -0.0046810904,
                -0.022981122,
                0.005502807,
                -0.009692214,
                0.009847128,
                0.011019085,
                -0.019532606,
                0.02100092,
                0.010662109,
                -0.01818553,
                -0.009557506,
                -0.008917646,
                0.0064861723,
                -0.004357792,
                -0.021634046,
                0.015262375,
                -0.012015921,
                0.0086010825,
                0.027453415,
                0.023169713,
                0.0049269316,
                0.03903827,
                -0.0026065926,
                -0.013618941,
                0.009133178,
                -0.014036535,
                -0.019909788,
                -0.017862232,
                0.009544035,
                0.02613328,
                0.01605715,
                -0.0144406585,
                -0.01209001,
                0.014198184,
                -0.024503319,
                -0.024274316,
                0.02912379,
                0.0349701,
                -0.013834474,
                -0.028665785,
                0.009779774,
                0.02241535,
                -0.0028877947,
                -0.01640739,
                0.02222676,
                0.018077765,
                0.023991428,
                -0.016569039,
                0.007099092,
                -0.0016392235,
                -0.0034047354,
                0.0031639454,
                0.014858251,
                -0.0028153895,
                0.01978855,
                0.04046617,
                0.022186346,
                -0.0006621722,
                -0.011753241,
                0.0011205991,
                -0.0005203082,
                -0.022334525,
                0.018859068,
                -0.018589653,
                -0.018050823,
                0.015612614,
                0.01713481,
                0.027399532,
                0.0035360754,
                -0.022725178,
                0.025836924,
                -0.002101439,
                -0.0041624657,
                0.014373304,
                -0.018023882,
                -0.015262375,
                0.0007960379,
                0.015908971,
                0.02331789,
                0.016717218,
                -0.016703747,
                0.002721094,
                0.013376468,
                -0.0064221863,
                -0.018657006,
                -0.018616594,
                0.003190887,
                0.007826514,
                -0.009564241,
                -0.0050549037,
                -0.010891112,
                -0.007523421,
                0.0057116034,
                -0.002123329,
                0.031548526,
                -0.015289316,
                0.056792736,
                0.017377285,
                -0.012568222,
                0.0190746,
                -0.012723136,
                -0.00032729746,
                0.00037065646,
                0.013006022,
                -0.02187652,
                -0.020758446,
                0.020044496,
                -0.0019684152,
                0.031144405,
                0.011328911,
                -0.022118993,
                -0.027210942,
                -0.009011941,
                0.002483672,
                0.012925197,
                -0.01297908,
                0.008479846,
                -0.041031945,
                0.008351874,
                -0.00033697957,
                -0.01394224,
                -0.00503133,
                -0.0027160426,
                0.00094042765,
                -0.007179917,
                -0.0021536383,
                0.0074762735,
                -0.008715584,
                -0.018683948,
                -0.018320238,
                -0.0067825294,
                -0.007045209,
                0.0009303246,
                0.013794062,
                0.017740995,
                -0.010352282,
                -0.018778244,
                0.03690989,
                -0.008230636,
                -0.006415451,
                0.047147673,
                -0.022401879,
                0.0056644557,
                -0.015302787,
                -0.015989795
            ],
            "DescriptionVector":[
                -0.018559298,
                0.023705104,
                0.011541688,
                -0.009802166,
                0.011078698,
                0.022752667,
                -0.017884655,
                -0.008373511,
                -0.003720458,
                0.0005737773,
                -0.0053905305,
                0.01567553,
                -0.009967521,
                -0.019908585,
                -0.0054004514,
                -0.019908585,
                0.016336944,
                -0.006683596,
                0.0024455811,
                -0.0054401364,
                -0.0064818645,
                0.01738198,
                -0.0007399578,
                -0.015516791,
                -0.0070639094,
                -0.00853225,
                0.010946414,
                -0.023017233,
                0.026059741,
                0.0109398,
                -0.009259807,
                0.0020437716,
                -0.035901595,
                -0.0072954046,
                -0.013274594,
                -0.021707632,
                -0.0037336864,
                0.0011831056,
                -0.017792057,
                0.008108945,
                0.020622913,
                -0.0038858117,
                0.0047886427,
                0.0065777698,
                -0.018876778,
                0.018757723,
                0.030821927,
                0.015477105,
                -0.033546958,
                0.0007469853,
                -0.0011922001,
                0.018718038,
                -0.0076327263,
                -0.027091548,
                -0.003459199,
                -0.048389103,
                -0.002174401,
                0.00966327,
                0.0044414,
                -0.0363249,
                0.010079961,
                -0.024221009,
                -0.024578173,
                0.0073350896,
                -0.0038791976,
                0.009762482,
                -0.018612212,
                0.0012153497,
                0.013036485,
                -0.0022785738,
                0.05423601,
                0.0071565076,
                0.0040908502,
                -0.0057543083,
                0.041404564,
                0.0030094371,
                0.01521254,
                0.01621789,
                0.014630495,
                0.0034029789,
                -0.0018949533,
                -0.02071551,
                -0.011938537,
                0.016747022,
                0.029234532,
                0.010179173,
                -0.0064289514,
                0.021694403,
                0.005152421,
                -0.011667356,
                0.012871131,
                0.02645659,
                -0.0039519533,
                -0.017249696,
                -0.035742853,
                0.0050465944,
                0.027091548,
                0.0036906945,
                0.0050697443,
                0.019935042,
                0.0038461268,
                0.014723092,
                -0.024776597,
                -0.0059725754,
                -0.047754146,
                -0.0069646975,
                -0.013691286,
                -0.026099427,
                0.008313984,
                0.029419728,
                -0.0011574759,
                0.030795472,
                0.017553948,
                -0.035901595,
                0.002017315,
                -0.011330035,
                0.019075202,
                -0.00083792984,
                0.016032694,
                -0.018109536,
                0.0018899926,
                0.011111768,
                0.003770064,
                -0.011812868,
                0.019511735,
                0.0006188362,
                -0.0027911703,
                -0.020146694,
                -0.029393272,
                0.0049639177,
                -0.022832038,
                -0.0067927297,
                0.0074276878,
                0.009451617,
                -0.014471755,
                -0.01404845,
                -0.014260102,
                0.020371575,
                -0.027382571,
                -0.012249402,
                0.0048680124,
                0.01489506,
                -0.016773479,
                -0.0013658215,
                -0.0060618664,
                0.025662893,
                -0.021390153,
                0.006812572,
                0.027065093,
                0.012434597,
                0.030239882,
                -0.030663189,
                0.014286559,
                0.005185492,
                0.000093734874,
                0.038679536,
                -0.014524668,
                0.016336944,
                -0.012871131,
                0.0012798376,
                -0.008671148,
                0.01660151,
                -0.0005183838,
                0.0064322585,
                0.02235582,
                0.007434302,
                0.024022585,
                0.008724061,
                0.0019346381,
                0.0045935255,
                -0.016019465,
                0.027514854,
                -0.014286559,
                0.0030739252,
                0.01637663,
                0.018942919,
                -0.008347054,
                0.016178206,
                -0.024935337,
                -0.012189874,
                -0.016310489,
                -0.0060254885,
                0.022395503,
                0.04203952,
                -0.008194929,
                0.0023149517,
                0.021787003,
                0.013625144,
                0.012983572,
                -0.0139161665,
                0.011237437,
                0.009511144,
                0.0075599705,
                -0.01129035,
                -0.6700925,
                -0.006455408,
                -0.03158917,
                -0.024181323,
                -0.0037303793,
                0.030636732,
                0.03135106,
                0.016667651,
                -0.019591104,
                -0.025689349,
                -0.021218186,
                0.012930659,
                0.00095822464,
                0.008519022,
                -0.026906352,
                -0.006187535,
                0.003049122,
                -0.005076358,
                -0.0053343102,
                -0.009921221,
                0.018784178,
                0.0014146008,
                -0.013228295,
                0.011263893,
                0.011766569,
                -0.0057443874,
                0.015331594,
                -0.004583604,
                -0.03375861,
                -0.0063098967,
                -0.00896217,
                0.03180082,
                0.008618235,
                0.0019197563,
                0.050928935,
                -0.0037535287,
                -0.021694403,
                0.053415857,
                -0.005076358,
                0.03561057,
                0.023652192,
                0.0010293267,
                0.018268276,
                -0.016575053,
                0.029340358,
                0.0038229772,
                0.029922403,
                -0.017792057,
                -0.015728444,
                -0.012110504,
                0.019882128,
                -0.0349227,
                -0.014670179,
                0.022289677,
                0.002266999,
                0.0038461268,
                0.034949157,
                -0.006273519,
                -0.014180732,
                -0.0035584115,
                0.007169736,
                -0.010490038,
                -0.017514262,
                -0.0437592,
                -0.0022504637,
                0.023572821,
                -0.015847497,
                0.033467587,
                -0.011475546,
                0.00026849305,
                -0.009769096,
                0.03428774,
                -0.022990776,
                -0.024723684,
                0.00023025501,
                0.0032425858,
                0.019339768,
                -0.004176834,
                -0.012864517,
                0.012427983,
                0.0075533567,
                -0.018532842,
                -0.011085311,
                0.015450649,
                0.02149598,
                -0.014802462,
                -0.03661592,
                0.020569999,
                0.041404564,
                0.007765009,
                0.011005942,
                -0.00049564766,
                0.010741375,
                -0.030663189,
                -0.02567612,
                0.0039254967,
                -0.018374102,
                0.0020917242,
                0.012117119,
                -0.04833619,
                -0.023863845,
                -0.010913343,
                0.0017477885,
                0.00020255827,
                -0.0025894388,
                0.015821042,
                0.008280912,
                0.025649665,
                0.021562122,
                -0.0065678484,
                0.008915871,
                0.012447826,
                -0.0020239293,
                -0.008896029,
                0.0062900544,
                -0.03034571,
                0.01535805,
                0.030239882,
                -0.0021479444,
                0.009451617,
                0.005929583,
                0.000019958707,
                0.019300083,
                -0.026628558,
                0.009140752,
                0.016866077,
                0.0065579275,
                -0.018585755,
                -0.011707041,
                -0.00011471412,
                0.018268276,
                0.0013501128,
                0.0349227,
                0.0020900706,
                0.039684884,
                0.019300083,
                0.027223831,
                -0.0136383725,
                0.009636813,
                -0.024181323,
                -0.028705401,
                -0.013519318,
                -0.001370782,
                0.010953029,
                -0.012613179,
                -0.02979012,
                -0.011217594,
                0.0033252628,
                -0.04677525,
                0.010278385,
                -0.036959857,
                -0.015450649,
                -0.040002365,
                0.01900906,
                0.0095772855,
                0.022316134,
                -0.0030854999,
                -0.025821632,
                0.000014248837,
                -0.015146398,
                0.03785938,
                0.0076922537,
                -0.018572526,
                0.020001182,
                0.011065469,
                -0.012719005,
                -0.0011955071,
                0.018942919,
                -0.001242633,
                -0.041404564,
                0.021747317,
                0.01792434,
                -0.0041669128,
                0.024101954,
                -0.009954292,
                -0.01769946,
                -0.0053905305,
                -0.023837388,
                0.0025249508,
                -0.0020189686,
                -0.008221385,
                0.0013418451,
                -0.020530313,
                -0.015014115,
                0.02304369,
                0.015926868,
                0.019458823,
                0.009901379,
                0.0015344822,
                -0.005608797,
                0.0049076974,
                0.0004032563,
                -0.022329362,
                -0.0001833359,
                -0.018427014,
                0.0062371413,
                0.012712392,
                -0.0065149353,
                -0.010185787,
                0.017038044,
                0.010655392,
                0.0037766783,
                0.026112655,
                0.0061280075,
                0.00841981,
                -0.026165567,
                -0.0031483343,
                -0.01050988,
                0.056140885,
                -0.001881725,
                0.014286559,
                -0.037145052,
                -0.043230068,
                -0.03240932,
                0.02404904,
                0.03312365,
                0.0069646975,
                0.01752749,
                -0.011475546,
                0.021469522,
                0.013135697,
                0.012361842,
                0.021615034,
                -0.012560266,
                -0.0417485,
                -0.021548893,
                0.009630199,
                0.024935337,
                0.002953217,
                -0.017633317,
                -0.009411932,
                0.014987659,
                -0.025795175,
                0.017580405,
                -0.004524077,
                0.037515447,
                0.00034662266,
                -0.02862603,
                0.0003096248,
                0.013691286,
                0.017024815,
                0.025821632,
                0.017831743,
                -0.019763073,
                -0.00019325712,
                -0.012745462,
                0.028308552,
                0.000055910215,
                0.0019181028,
                -0.014471755,
                -0.017937569,
                -0.0052648615,
                0.0005638561,
                0.0061147795,
                0.0037403004,
                0.002266999,
                0.009339176,
                -0.0047820285,
                0.005694781,
                0.025689349,
                -0.011508617,
                -0.0057939934,
                0.020212835,
                -0.009557444,
                0.006266905,
                -0.02297755,
                0.01660151,
                -0.031139407,
                0.026416905,
                0.026668243,
                0.003670852,
                -0.011879009,
                0.013043099,
                -0.003541876,
                -0.014167504,
                -0.015728444,
                -0.021667948,
                0.00349227,
                -0.020900706,
                0.013069555,
                -0.0027647137,
                -0.0014923171,
                0.02460463,
                0.014471755,
                0.015027343,
                -0.0048150993,
                -0.03949969,
                0.022474874,
                -0.01218326,
                0.038997013,
                -0.0044182506,
                -0.013955851,
                -0.017514262,
                -0.00006526303,
                -0.010642163,
                -0.005595569,
                0.016336944,
                -0.014021993,
                -0.007844379,
                -0.03240932,
                -0.007884064,
                -0.025623208,
                -0.013023256,
                -0.023797704,
                0.0017907805,
                0.016627967,
                -0.00880343,
                0.0020933778,
                -0.01082736,
                -0.017937569,
                0.0056683244,
                -0.005512892,
                0.03135106,
                -0.0018470007,
                -0.009511144,
                -0.007943591,
                -0.01419396,
                -0.0048911623,
                0.042886134,
                -0.011488775,
                0.00009316647,
                -0.03934095,
                -0.016270803,
                0.007857608,
                0.07720033,
                0.0069580832,
                0.0067133596,
                0.015040572,
                0.007844379,
                -0.01132342,
                -0.03219767,
                -0.011283736,
                -0.0025894388,
                0.0067034387,
                -0.0139426235,
                -0.025279272,
                0.02754131,
                0.016045922,
                0.0060420237,
                -0.0011831056,
                0.000118331234,
                -0.020424487,
                0.024340063,
                0.012527196,
                -0.004481085,
                0.0051921057,
                -0.009015083,
                0.013704514,
                0.0025166832,
                -0.015278681,
                0.016892534,
                -0.019710159,
                -0.0036873873,
                -0.007844379,
                0.019128114,
                -0.008512408,
                -0.0055492697,
                0.013334122,
                -0.023546366,
                0.032779716,
                0.003135106,
                -0.020186378,
                0.024088725,
                0.005929583,
                0.03680112,
                0.0045935255,
                -0.010655392,
                0.0073284754,
                -0.0028341622,
                -0.0009714529,
                0.008598392,
                0.008571935,
                -0.024776597,
                0.0010731454,
                0.033705696,
                -0.0073350896,
                -0.008710832,
                0.004001559,
                0.004533998,
                -0.0057179304,
                -0.0062206057,
                -0.0034658134,
                -0.026165567,
                -0.0051722634,
                -0.007765009,
                -0.048203908,
                0.0014633802,
                -0.010278385,
                -0.01101917,
                -0.028149812,
                -0.002716761,
                -0.0041437633,
                -0.013248137,
                -0.0014079866,
                -0.009001855,
                -0.0015964899,
                -0.03135106,
                -0.00074781204,
                0.03391735,
                0.00033194752,
                0.016098835,
                0.0012227906,
                0.0027481783,
                0.043626916,
                0.01605915,
                -0.012599952,
                -0.017712688,
                -0.017805286,
                0.00026022538,
                -0.00262747,
                -0.016627967,
                -0.006600919,
                -0.00566171,
                0.023718333,
                0.030742558,
                -0.025186675,
                0.013387035,
                0.00977571,
                -0.02374479,
                -0.015067028,
                -0.003892426,
                -0.0038858117,
                -0.001421215,
                -0.009689726,
                0.006551313,
                -0.0055360417,
                0.0006936587,
                -0.030001773,
                0.024564944,
                0.0051921057,
                -0.016045922,
                -0.0054467507,
                -0.027025407,
                0.023466995,
                0.014537896,
                -0.0051259645,
                -0.004967225,
                0.0009987362,
                -0.023691876,
                0.019233942,
                -0.0045935255,
                0.023837388,
                0.002999516,
                -0.010714919,
                -0.0038295914,
                -0.004705966,
                0.01210389,
                0.023374397,
                0.0136383725,
                -0.022593929,
                0.028705401,
                0.024776597,
                0.0029300675,
                -0.0056881667,
                0.00021847356,
                0.018956147,
                -0.011330035,
                -0.013208453,
                -0.021244641,
                -0.0075599705,
                -0.028996423,
                -0.016694108,
                -0.007487215,
                0.008591778,
                -0.016389858,
                0.01800371,
                -0.019882128,
                0.0032475465,
                -0.04143102,
                -0.020450944,
                0.003813056,
                -0.012599952,
                0.016151749,
                0.007619498,
                0.002106606,
                0.006772887,
                -0.013287823,
                0.012249402,
                -0.01660151,
                -0.02149598,
                -0.020887477,
                -0.04214535,
                0.005152421,
                0.032541607,
                0.0041834484,
                0.003027626,
                0.010529723,
                -0.005962654,
                -0.014656951,
                0.0095772855,
                -0.012163417,
                0.018969376,
                -0.017157098,
                0.040531497,
                0.03238287,
                0.022236764,
                0.019022288,
                -0.021072675,
                -0.027303202,
                0.03669529,
                -0.01676025,
                0.011587987,
                -0.028837683,
                -0.019921813,
                0.024525259,
                -0.0009144059,
                -0.0033500658,
                0.006918398,
                -0.023427311,
                -0.004709273,
                0.024088725,
                -0.013717742,
                -0.00038444734,
                -0.0058865915,
                0.0088828,
                -0.031086493,
                0.024234237,
                0.013301051,
                -0.0057410803,
                0.024088725,
                -0.010695077,
                -0.03283263,
                0.0038858117,
                0.016403086,
                0.010318071,
                -0.0058535207,
                0.0030127442,
                -0.024617856,
                0.024154866,
                0.033494044,
                0.014273331,
                -0.00093176804,
                0.0092994915,
                -0.013902938,
                -0.03484333,
                -0.02699895,
                -0.0010119646,
                -0.035240177,
                0.02770005,
                0.0043190382,
                -0.007222649,
                0.013109241,
                -0.027726507,
                -0.017990481,
                0.030292796,
                -0.012355228,
                0.012560266,
                0.002217393,
                -0.004474471,
                0.008915871,
                -0.009332562,
                -0.021548893,
                0.02559675,
                -0.012692549,
                -0.0061313147,
                0.01543742,
                -0.0062139914,
                -0.00093672867,
                -0.0139161665,
                0.007884064,
                0.022263221,
                -0.018823864,
                -0.018030167,
                0.032568064,
                0.016866077,
                0.0076724114,
                -0.020345118,
                0.0023265264,
                -0.009861694,
                0.0050697443,
                -0.0034757345,
                -0.0051193503,
                -0.015807813,
                -0.0024389669,
                -0.021681176,
                0.015238996,
                -0.026522731,
                0.019908585,
                0.010073347,
                -0.022699755,
                0.014987659,
                -0.01707773,
                0.0009061382,
                0.0017494421,
                0.0071565076,
                -0.003341798,
                -0.03508144,
                -0.0048051784,
                0.0038626622,
                -0.009266421,
                0.004798564,
                -0.0020569998,
                0.031642083,
                0.017646546,
                -0.018969376,
                -0.017831743,
                0.021707632,
                0.02591423,
                0.009445002,
                -0.0048250207,
                -0.005383916,
                0.005751001,
                -0.012791761,
                -0.012560266,
                -0.015940096,
                0.0090944525,
                -0.007936977,
                -0.024789825,
                0.009716183,
                -0.015543247,
                -0.010867044,
                0.02148275,
                0.0109398,
                -0.010258543,
                -0.0034823488,
                -0.0038626622,
                -0.008743903,
                0.013096012,
                0.012130347,
                0.009511144,
                0.00054194673,
                0.018083079,
                -0.021601805,
                -0.00063578493,
                0.0047886427,
                0.018784178,
                -0.017871426,
                -0.021694403,
                -0.0038395126,
                0.019181028,
                0.016733794,
                -0.0017395208,
                -0.037356704,
                -0.0047324225,
                0.0009879883,
                -0.017104186,
                0.0086645335,
                0.0046365173,
                0.0037766783,
                -0.0024935335,
                0.0036146317,
                -0.009101067,
                -0.017289381,
                0.02444589,
                0.017210012,
                0.002670462,
                0.014802462,
                -0.022157395,
                0.027911702,
                0.008995241,
                -0.020689053,
                -0.0042760465,
                -0.026390448,
                0.004669588,
                0.0028258946,
                0.01419396,
                -0.023255343,
                -0.00023645577,
                -0.022593929,
                -0.025570294,
                -0.009974134,
                0.011601215,
                -0.002553061,
                -0.027276745,
                -0.0030077836,
                0.02955201,
                0.0030408544,
                -0.0012484203,
                -0.014762778,
                -0.012719005,
                -0.00082842197,
                -0.007837765,
                -0.033176564,
                -0.027594224,
                -0.018347645,
                0.009365633,
                0.00035716398,
                -0.026972493,
                -0.011085311,
                0.010000591,
                -0.06069142,
                -0.008089103,
                0.0059031267,
                -0.0017758986,
                0.02661533,
                0.016971903,
                -0.0038395126,
                -0.0017180248,
                -0.016720565,
                0.0027465248,
                0.0014187347,
                0.001613852,
                0.0074607586,
                -0.034790415,
                -0.0016485762,
                -0.008194929,
                -0.041007716,
                -0.0198689,
                0.015953325,
                -0.004206598,
                0.013400263,
                0.027911702,
                -0.03002823,
                -0.0052582473,
                0.02676084,
                0.015238996,
                0.008009733,
                0.011237437,
                0.006333046,
                -0.002839123,
                -0.025014706,
                -0.0020189686,
                -0.001074799,
                -0.03415546,
                -0.0056187185,
                -0.0009904685,
                0.0065711555,
                0.013995537,
                0.0075996555,
                -0.007831151,
                0.010946414,
                -0.006812572,
                0.02513376,
                -0.02009378,
                -0.018109536,
                0.014630495,
                0.012097276,
                0.0061842278,
                -0.016813163,
                0.01870481,
                -0.029525556,
                -0.01256688,
                0.008631462,
                -0.0218796,
                0.008089103,
                -0.0117334975,
                0.03238287,
                -0.012084047,
                -0.01683962,
                0.012017906,
                -0.005836985,
                -0.011819482,
                -0.0011781451,
                0.03452585,
                -0.014273331,
                0.012388298,
                -0.024789825,
                0.0061643855,
                -0.0063859597,
                -0.0028573119,
                -0.00017248456,
                0.0014021993,
                -0.011363106,
                -0.002060307,
                -0.011515231,
                -0.0061643855,
                0.008128787,
                -0.0033318768,
                -0.0036576237,
                0.0037138439,
                0.19821277,
                0.015847497,
                -0.014260102,
                0.010007205,
                -0.008485951,
                0.026033284,
                0.03095421,
                -0.0020520394,
                -0.009451617,
                0.018281505,
                -0.022157395,
                0.014365928,
                -0.0025629823,
                0.0025464469,
                -0.0138897095,
                -0.019604333,
                -0.037092138,
                -0.03135106,
                -0.00397841,
                -0.0005646828,
                -0.0042760465,
                0.0017527491,
                -0.003717151,
                -0.003634474,
                0.02009378,
                -0.006154464,
                0.012507353,
                -0.007850993,
                0.041774955,
                0.038732447,
                0.0054798215,
                0.00519872,
                0.002263692,
                -0.016733794,
                0.0074078455,
                -0.0044314787,
                0.020107009,
                -0.013512704,
                0.007024225,
                0.005483128,
                0.007950205,
                0.001107043,
                -0.014074906,
                0.013678057,
                0.007745167,
                0.014008764,
                -0.028731856,
                0.009610357,
                0.016614739,
                0.01559616,
                -0.021112358,
                0.009074611,
                0.00013207625,
                0.030266339,
                0.014961202,
                -0.014101363,
                0.007831151,
                -0.0035517972,
                0.00725572,
                0.0012922391,
                -0.009888151,
                0.022990776,
                -0.02862603,
                -0.0049539967,
                0.0043322667,
                0.0039387248,
                -0.011118382,
                -0.005698088,
                0.011865781,
                0.011058855,
                0.0005870056,
                -0.013717742,
                -0.030583818,
                0.019419137,
                0.010073347,
                -0.02280558,
                0.05026752,
                -0.0048944694,
                0.03002823,
                0.0030854999,
                -0.0046431315,
                -0.0063760383,
                -0.017831743,
                0.0136119155,
                0.0131290825,
                -0.039155755,
                0.02055677,
                0.01015933,
                -0.0061478503,
                -0.013254751,
                -0.00023707586,
                0.0035220336,
                -0.0009912953,
                -0.0011450743,
                0.016879305,
                0.038811818,
                0.01723647,
                0.015318366,
                -0.012811604,
                0.015794585,
                0.0077319387,
                0.0032773102,
                0.020384803,
                -0.013307665,
                -0.02055677,
                0.003994945,
                -0.01199145,
                -0.0059395046,
                0.02677407,
                0.006991154,
                0.017646546,
                -0.0023314871,
                -0.015662301,
                -0.02530573,
                0.0037865995,
                -0.007308633,
                0.03902347,
                -0.021628262,
                0.011363106,
                -0.008505794,
                -0.0023232193,
                -0.017421665,
                0.0012409794,
                0.016971903,
                -0.0032591212,
                -0.013334122,
                -0.020702282,
                0.018718038,
                -0.0029796734,
                -0.03344113,
                -0.004530691,
                -0.007182964,
                0.005555884,
                0.012864517,
                -0.017884655,
                0.0028573119,
                -0.004977146,
                0.005827064,
                -0.008565322,
                -0.010979485,
                0.0022587313,
                -0.01800371,
                0.0020520394,
                -0.004689431,
                -0.0045604547,
                -0.011654128,
                -0.0029614845,
                -0.016178206,
                -0.0056319465,
                0.02119173,
                0.007791466,
                -0.013056328,
                -0.018135993,
                -0.026813755,
                0.020993304,
                -0.013466405,
                -0.017805286,
                -0.009822009,
                0.0017544026,
                0.00609163,
                -0.004530691,
                -0.013387035,
                0.025887774,
                0.0044678566,
                -0.0088828,
                -0.003988331,
                -0.1685814,
                0.0417485,
                0.013598687,
                -0.019061973,
                0.0083338255,
                -0.002167787,
                0.02280558,
                -0.003806442,
                -0.019921813,
                0.022712983,
                0.008042803,
                -0.015172855,
                -0.02031866,
                -0.009802166,
                0.0018040087,
                0.010728148,
                -0.0041867555,
                -0.0061511574,
                0.009742639,
                0.007341704,
                0.003942032,
                -0.00744753,
                0.0062139914,
                0.0031202242,
                -0.007341704,
                0.0012550345,
                -0.0038626622,
                -0.0051755705,
                -0.0018089694,
                -0.017183555,
                0.0018056623,
                0.00037080565,
                0.040452126,
                0.015887182,
                0.012924044,
                -0.00096814585,
                0.014762778,
                0.0018982603,
                -0.022990776,
                0.01955142,
                -0.0073152473,
                -0.029975317,
                0.015794585,
                -0.0032773102,
                -0.0048018713,
                0.036271986,
                0.017673003,
                -0.009431775,
                0.031086493,
                0.0023132982,
                0.0052813967,
                -0.017620089,
                0.012143575,
                0.009392089,
                0.013902938,
                0.021667948,
                -0.016522141,
                -0.0003118984,
                0.024088725,
                -0.0006494266,
                -0.0029267604,
                -0.003316995,
                -0.0022603848,
                -0.03002823,
                0.0065546203,
                -0.019829214,
                0.018890005,
                -0.0019379453,
                -0.01838733,
                0.016548596,
                -0.0033087274,
                -0.0038163632,
                -0.0044546286,
                -0.0051788776,
                0.026178796,
                0.010073347,
                0.0002109293,
                -0.0139426235,
                -0.017329067,
                -0.028969966,
                -0.023943214,
                0.033229478,
                -0.030504448,
                -0.008247842,
                -0.0058105285,
                -0.0055327346,
                -0.015556475,
                0.012408141,
                -0.0045174626,
                -0.0026605409,
                0.010165945,
                -0.009874922,
                -0.012639636,
                -0.017501034,
                0.0114557035,
                0.033732153,
                0.025980372,
                0.005602183,
                -0.014405614,
                -0.01047681,
                0.0024373133,
                0.008353668,
                -0.018175678,
                -0.016918989,
                0.021376925,
                0.037700642,
                0.0062172986,
                0.013473019,
                0.029446185,
                -0.009491302,
                0.014101363,
                -0.0061709997,
                0.0083867395,
                0.036748204,
                -0.016006237,
                0.012745462,
                0.022832038,
                -0.011660743,
                0.024895651,
                -0.013486247,
                0.024260694,
                0.011859166,
                -0.033494044,
                0.029446185,
                -0.00834044,
                -0.02366542,
                -0.09534955,
                -0.023731561,
                0.019075202,
                0.0068390286,
                -0.0045207697,
                0.029234532,
                -0.003298806,
                0.0010855469,
                0.0011822789,
                -0.00057791115,
                0.002053693,
                -0.02910225,
                -0.0020123543,
                -0.011587987,
                0.05323066,
                0.0015179468,
                -0.015953325,
                -0.007004382,
                -0.025279272,
                -0.0070109963,
                -0.021231413,
                -0.010410668,
                0.0057443874,
                -0.023321485,
                -0.010701691,
                0.0074938294,
                -0.01668088,
                0.011416019,
                0.020133466,
                0.009306106,
                0.0023232193,
                -0.006333046,
                0.013783883,
                -0.0025117225,
                0.020424487,
                -0.0039684884,
                0.025887774,
                0.013393649,
                0.010615706,
                -0.02049063,
                0.0013393649,
                0.003856048,
                -0.001885032,
                -0.0465636,
                0.007480601,
                -0.011872395,
                0.0075467424,
                0.0062536765,
                -0.01419396,
                -0.012090662,
                -0.030742558,
                0.02031866,
                0.0047158874,
                -0.02754131,
                0.0012128693,
                0.0026737691,
                -0.016244346,
                0.0067034387,
                -0.013691286,
                -0.017011588,
                -0.0042694323,
                0.0069316267,
                -0.010655392,
                -0.020358346,
                -0.00048862014,
                0.0059858034,
                -0.019842442,
                -0.0008300755,
                0.016561825,
                -0.022752667,
                0.00014737148,
                -0.002670462,
                0.0026324308,
                0.0069448547,
                -0.011151453,
                -0.0007837765,
                -0.019207485,
                0.0012368456,
                0.003981717,
                -0.033467587,
                -0.011905465,
                -0.01792434,
                0.0061114724,
                0.0051391926,
                0.012110504,
                0.037568357,
                0.00849918,
                -0.0058502136,
                -0.0007420247,
                -0.004884548,
                0.009352405,
                -0.009689726,
                0.0069316267,
                -0.03330885,
                -0.0077253245,
                0.015701987,
                -0.00506313,
                -0.015067028,
                -0.027091548,
                0.018612212,
                -0.037145052,
                -0.021694403,
                -0.063284166,
                -0.0050928937,
                0.011429247,
                0.01947205,
                -0.007044067,
                -0.006045331,
                0.008327212,
                0.01621789,
                0.0009689726,
                0.01806985,
                -0.036668833,
                -0.003988331,
                0.020940391,
                -0.015582932,
                -0.02310983,
                0.0037865995,
                0.013175382,
                0.026668243,
                -0.0020107008,
                0.0053706877,
                -0.0028440834,
                0.008631462,
                0.021601805,
                0.010919957,
                -0.026403677,
                0.003985024,
                -0.010318071,
                0.02645659,
                -0.013757427,
                0.009101067,
                0.017633317,
                -0.0151596265,
                -0.006005646,
                0.05354814,
                -0.010245315,
                -0.035954505,
                -0.01419396,
                0.01039744,
                -0.0019842442,
                0.0030921139,
                -0.041642673,
                -0.034711048,
                0.0026539266,
                -0.022541014,
                0.0047158874,
                0.007487215,
                0.012341999,
                0.0069117844,
                0.039631974,
                -0.004080929,
                -0.0017858199,
                0.024326835,
                -0.0027614066,
                -0.013757427,
                -0.0009937757,
                0.0021710938,
                0.004405022,
                0.004087543,
                0.01381034,
                -0.0012492471,
                0.036430724,
                0.0060651735,
                -0.0040908502,
                -0.008743903,
                0.0123684555,
                -0.035901595,
                -0.008552093,
                0.017990481,
                -0.00015780942,
                0.0054533645,
                -0.006223913,
                0.0100336615,
                0.019921813,
                0.011852552,
                0.007804694,
                -0.0013112548,
                -0.0028176268,
                -0.027805876,
                -0.01121098,
                0.03740962,
                0.011495388,
                -0.00039684886,
                -0.011607829,
                0.024326835,
                0.019286854,
                -0.009954292,
                -0.020596456,
                -0.006772887,
                0.0045207697,
                0.022607157,
                -0.010384211,
                0.018043395,
                -0.011707041,
                0.0129968,
                -0.0033996718,
                -0.0012451133,
                0.013195224,
                -0.002014008,
                0.0031615626,
                0.031007124,
                -0.017315838,
                0.020940391,
                -0.004530691,
                -0.025755491,
                -0.015582932,
                -0.0051292717,
                -0.0062139914,
                -0.019247169,
                0.017501034,
                0.020226063,
                0.013228295,
                0.021853143,
                0.02623171,
                0.008155244,
                0.007500443,
                -0.004934154,
                0.019895356,
                0.014604038,
                -0.014934746,
                0.01746135,
                -0.00016194327,
                0.013096012,
                0.007824536,
                -0.02553061,
                0.0033517194,
                0.001096295,
                -0.010430511,
                -0.0029052643,
                -0.008585163,
                0.01489506,
                -0.007030839,
                0.02002764,
                -0.010344527,
                -0.0009276342,
                -0.01637663,
                -0.0042429757,
                0.01722324,
                0.0020057403,
                0.009101067,
                0.067570135,
                0.026959265,
                -0.0021661334,
                -0.000322233,
                -0.012031134,
                0.028440835,
                0.015530018,
                -0.010867044,
                -0.026112655,
                -0.02119173,
                0.012758691,
                0.022236764,
                0.006190842,
                -0.0037270722,
                -0.021773774,
                -0.0150538,
                -0.030504448,
                -0.012871131,
                0.0032822706,
                0.0014195614,
                -0.0009449963,
                -0.018043395,
                -0.0019081816,
                -0.0022107789,
                -0.03685403,
                0.004216519,
                0.013466405,
                0.00065604073,
                -0.0050399806,
                -0.010212244,
                0.019207485,
                0.0041900626,
                -0.026218481,
                -0.019128114,
                -0.023956442,
                0.002124795,
                -0.004474471,
                -0.010992713,
                -0.0049473825,
                0.019220714,
                -0.013201838,
                0.047277927,
                -0.034102544,
                -0.021271098,
                0.0003902347,
                -0.0051755705,
                -0.0072292634,
                -0.014061678,
                -0.019683704
            ]
    },
    {
    "@search.action": "upload",
    "HotelId": "3",
    "HotelName": "Triple Landscape Hotel",
    "Description": "The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Category": "Resort and Spa",
    "Tags": [ "air conditioning", "bar", "continental breakfast" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "2015-09-20T00:00:00Z",
    "Rating": "4.80",
    "Address": {
        "StreetAddress": "3393 Peachtree Rd",
        "City": "Atlanta",
        "StateProvince": "GA",
        "PostalCode": "30326",
        "Country": "USA"
        },
        "HotelNameVector":[
                -0.007549237,
                0.004659034,
                0.015806006,
                -0.01224832,
                -0.013509863,
                0.03043724,
                -0.022307293,
                -0.0042885807,
                0.009064424,
                -0.0027316757,
                0.011460689,
                0.029315868,
                -0.009378141,
                -0.0025280933,
                0.002649909,
                0.008143297,
                0.0134631395,
                -0.011887879,
                0.009318068,
                0.0034175147,
                -0.017047524,
                0.007709433,
                -0.0030770984,
                -0.02968966,
                -0.008223395,
                0.013803556,
                0.0150851235,
                -0.006444552,
                -0.00821672,
                0.020304844,
                -0.010185796,
                -0.0073089427,
                -0.015872754,
                -0.016660385,
                0.0021292719,
                -0.00071671023,
                0.021920154,
                0.0042652185,
                0.005236407,
                0.009938827,
                0.011427315,
                -0.0042785686,
                -0.0057670563,
                0.0035643613,
                -0.021346118,
                0.019490514,
                0.025230872,
                0.006287693,
                -0.051102526,
                0.019303618,
                0.003868066,
                0.02365561,
                -0.018769631,
                -0.008984326,
                0.00013266232,
                -0.011547462,
                0.009478264,
                0.018329093,
                0.012768957,
                -0.027980903,
                0.0139637515,
                -0.039755307,
                -0.021346118,
                0.0020358243,
                -0.018168896,
                0.0055701486,
                0.0027450253,
                0.013302944,
                0.022721134,
                -0.012875754,
                0.035937306,
                0.016006252,
                0.0038013177,
                -0.013576612,
                -0.0015552362,
                -0.019463815,
                -0.013069324,
                -0.006768281,
                0.0035376619,
                0.016153097,
                0.020211397,
                -0.020571837,
                -0.008363566,
                -0.0010437771,
                0.02006455,
                0.008009801,
                -0.0045822733,
                0.017327867,
                0.0042418567,
                -0.018142197,
                -0.0015827699,
                0.002741688,
                -0.021920154,
                -0.0014459358,
                -0.032252796,
                -0.02409615,
                0.010746482,
                0.012528663,
                -0.006588061,
                -0.004355329,
                -0.009691859,
                0.009364791,
                -0.019316968,
                -0.007255544,
                -0.07021925,
                -0.025364367,
                0.0016845611,
                -0.0002240241,
                0.015191921,
                0.024563387,
                -0.029556163,
                0.013816905,
                0.0024880443,
                -0.04285243,
                -0.018996576,
                -0.026005153,
                0.016032951,
                0.007976426,
                -0.00071295566,
                -0.003524312,
                0.003981538,
                0.025818257,
                0.0067115454,
                -0.014924928,
                0.022240546,
                0.004795868,
                0.00011076052,
                -0.009771956,
                -0.022400742,
                0.012795656,
                0.003045393,
                0.015298719,
                0.0028601661,
                -0.011193696,
                -0.028061,
                0.015178571,
                -0.0014734695,
                -0.012328418,
                -0.03724557,
                -0.008210045,
                0.006207595,
                0.03126492,
                -0.017568162,
                -0.0042952555,
                0.025885005,
                -0.0036377844,
                -0.012348442,
                0.015191921,
                -0.023121623,
                -0.015098474,
                0.0145244375,
                -0.02766051,
                0.020117948,
                0.007055299,
                0.009244644,
                0.01850264,
                -0.018609436,
                0.017154323,
                -0.03807325,
                -0.027113175,
                -0.029182373,
                0.026766082,
                -0.0048959902,
                0.0063377544,
                0.033961553,
                0.024242995,
                0.031371716,
                0.025991801,
                -0.008864179,
                0.00026845347,
                -0.009985551,
                0.019316968,
                -0.0048158923,
                -0.01563246,
                -0.009351442,
                0.022414092,
                0.0005223057,
                -0.0012773963,
                -0.015285369,
                -0.012181572,
                -0.03380136,
                0.0006215939,
                0.019837605,
                0.013736808,
                -0.02401605,
                -0.010879979,
                0.0055868356,
                -0.0058605038,
                0.014924928,
                -0.020531788,
                0.03380136,
                0.018342443,
                0.00022715294,
                0.0064579016,
                -0.6634251,
                -0.0073156175,
                -0.014911578,
                -0.007822905,
                0.0142440945,
                0.01687398,
                0.0069618514,
                0.02329517,
                -0.017474715,
                -0.025978452,
                -0.004388703,
                0.0025848292,
                0.007916353,
                0.0068350295,
                0.00026136145,
                -0.0074424394,
                -0.014284144,
                -0.0095717115,
                0.012181572,
                -0.01164091,
                -0.0054700263,
                0.010386041,
                0.010032275,
                0.0011222064,
                0.0032105949,
                0.022574287,
                0.010920028,
                -0.002574817,
                -0.013209496,
                0.013596636,
                -0.007255544,
                0.012975876,
                -0.012735583,
                -0.038794134,
                0.04920687,
                -0.017074224,
                -0.008904228,
                0.030570738,
                0.011594186,
                0.033213973,
                -0.009731907,
                -0.010686409,
                0.0180354,
                0.011040174,
                0.010279244,
                0.009438215,
                0.04731122,
                -0.027980903,
                -0.018956527,
                -0.016166447,
                0.010906678,
                -0.0057703936,
                0.018676184,
                0.032813482,
                0.012275019,
                -0.008270118,
                0.033667862,
                -0.015485614,
                -0.0045221997,
                0.0054967254,
                -0.029796457,
                -0.0056802835,
                -0.008984326,
                -0.017207721,
                0.010920028,
                -0.0076827337,
                -0.011120273,
                0.025925053,
                0.0035510114,
                0.01082658,
                -0.01723442,
                0.026752733,
                -0.01023252,
                -0.019156773,
                -0.0020358243,
                0.02923577,
                0.009771956,
                -0.0049427142,
                -0.012254994,
                0.0067649437,
                0.015325418,
                -0.010793206,
                -0.008250094,
                -0.0019540575,
                0.020785432,
                -0.0056168726,
                -0.02365561,
                0.005727007,
                0.026645936,
                0.0060340497,
                0.005877191,
                0.022227196,
                -0.01643344,
                -0.0005410787,
                -0.0073556667,
                0.009698533,
                -0.013389716,
                0.023054875,
                0.015672509,
                -0.056122,
                -0.013843605,
                -0.011547462,
                -0.002529762,
                -0.0071220472,
                -0.0048325798,
                0.000473496,
                -0.018235646,
                -0.0022961427,
                0.044000503,
                -0.0031388404,
                -0.002122597,
                0.015031725,
                -0.013069324,
                -0.00823007,
                0.010512863,
                -0.037619364,
                0.006254319,
                0.009538338,
                0.011373917,
                -0.007182121,
                0.025898354,
                0.0014209052,
                0.03564361,
                -0.017821806,
                0.007215495,
                0.03534992,
                0.0001295335,
                -0.027446916,
                -0.0015235308,
                0.002065861,
                -0.019757507,
                0.027767308,
                0.04170436,
                -0.0140571995,
                0.011126948,
                0.01840919,
                0.031158123,
                -0.015191921,
                0.0030770984,
                -0.018676184,
                -0.028728483,
                -0.032306194,
                -0.012882429,
                0.012161546,
                -0.024176247,
                -0.023749057,
                -0.01687398,
                0.0049927756,
                -0.021479614,
                -0.009912128,
                -0.032519788,
                -0.010386041,
                -0.026952978,
                0.013296269,
                0.0022794558,
                -0.0030987915,
                0.008530437,
                -0.02726002,
                -0.025164122,
                -0.034789234,
                0.021840056,
                -0.011166996,
                -0.02169321,
                -0.0012465252,
                0.015045075,
                -0.013883654,
                0.007555912,
                0.0111603215,
                -0.0048092175,
                -0.027340118,
                -0.010873304,
                -0.031104725,
                -0.00081391254,
                0.011747707,
                -0.017020825,
                0.00078554446,
                0.011527438,
                -0.022814581,
                0.021039076,
                0.0070085754,
                -0.012321743,
                0.03126492,
                -0.009932152,
                0.006534662,
                0.019570611,
                0.016687084,
                0.022948079,
                0.02846149,
                0.0036945203,
                0.013336318,
                -0.00631773,
                0.021185922,
                -0.0007242194,
                0.008083223,
                -0.004595623,
                0.013336318,
                0.013603311,
                0.011360567,
                -0.0028301293,
                0.013569937,
                0.03572371,
                0.01605965,
                0.023909254,
                -0.011413965,
                0.01722107,
                -0.016339993,
                -0.0015001688,
                -0.0065446743,
                0.021706559,
                0.017167673,
                0.026338894,
                -0.028835282,
                -0.039274722,
                -0.0100256,
                0.0044654636,
                0.02123932,
                0.018676184,
                0.00842364,
                0.0059940005,
                0.0048392545,
                -0.007215495,
                0.001184783,
                -0.014350892,
                0.0007258882,
                -0.018569387,
                -0.008904228,
                -0.014898228,
                0.012388491,
                0.0017104261,
                -0.01926357,
                -0.019891005,
                0.014778081,
                -0.004235182,
                0.007068649,
                0.010339317,
                0.02771391,
                -0.013202821,
                -0.016259894,
                0.022227196,
                -0.019891005,
                0.027326768,
                0.025844956,
                0.02285463,
                -0.009097798,
                0.0055434494,
                0.008777406,
                0.034388743,
                -0.011494064,
                0.0137768565,
                -0.012341768,
                -0.022414092,
                0.018289044,
                -0.003868066,
                0.012722233,
                0.014204046,
                -0.0137768565,
                0.0037746185,
                0.012301718,
                0.015912803,
                0.017514763,
                -0.012254994,
                0.021786656,
                0.005436652,
                0.009331417,
                0.021199271,
                -0.019477164,
                -0.0003245638,
                0.004709095,
                0.0057870806,
                -0.00095867296,
                0.0015760951,
                0.015325418,
                0.016486838,
                -0.0061208224,
                -0.037966453,
                0.007742807,
                0.014404291,
                -0.022160448,
                -0.019357018,
                0.017448016,
                -0.0035109625,
                -0.006344429,
                0.033534363,
                0.004652359,
                -0.009645134,
                -0.032332893,
                -0.006901778,
                0.01563246,
                0.0014492732,
                0.03892763,
                0.0033691223,
                -0.010719783,
                0.019770857,
                -0.006277681,
                -0.018382492,
                -0.03540332,
                -0.0032139323,
                -0.040663086,
                -0.0015702546,
                -0.011293818,
                -0.0025180809,
                0.0018005364,
                -0.0045188623,
                -0.00841029,
                0.005446664,
                -0.020157997,
                -0.034415442,
                -0.0063110553,
                -0.012081449,
                -0.013803556,
                0.0070085754,
                -0.015832705,
                0.0055267625,
                -0.00077469787,
                -0.0066014105,
                -0.006831692,
                -0.010045624,
                0.006941827,
                0.028942078,
                -0.01597955,
                -0.026338894,
                -0.00742909,
                -0.023482064,
                0.008176671,
                0.10455459,
                0.019917704,
                0.012595411,
                0.015899453,
                0.0038313544,
                -0.019196821,
                -0.010519538,
                -0.041597564,
                -0.0016703771,
                0.0015694202,
                -0.015926152,
                -0.000987041,
                0.007842929,
                0.021813355,
                0.017821806,
                0.028701784,
                -0.020011151,
                -0.0046189846,
                0.0053865905,
                0.0006140847,
                0.016193146,
                -0.018462589,
                -0.0045455615,
                0.04058299,
                -0.018008702,
                -0.012128172,
                0.017434666,
                -0.00016478496,
                0.009785306,
                -0.0134965135,
                -0.0073222923,
                0.017741708,
                -0.005333192,
                0.012361792,
                -0.009858729,
                0.014137298,
                -0.0053699035,
                -0.0027950865,
                0.032626588,
                0.0015735921,
                0.021960203,
                0.013763507,
                -0.0010938384,
                -0.011333868,
                0.0020608548,
                0.0033541038,
                -0.021626461,
                0.017875204,
                -0.021853406,
                0.020972326,
                0.031798907,
                0.02846149,
                0.003988213,
                -0.011053525,
                -0.0026482402,
                -0.007502513,
                -0.005727007,
                -0.0068350295,
                0.014511088,
                -0.022187147,
                -0.015018376,
                -0.04942047,
                0.012728908,
                -0.017367916,
                0.0017221071,
                -0.017301168,
                -0.008256769,
                -0.001413396,
                -0.032359593,
                -0.0082033705,
                -0.022427442,
                -0.006588061,
                -0.027553713,
                -0.00983203,
                0.028701784,
                0.006254319,
                -0.0010496175,
                -0.0110068,
                0.0058338046,
                0.03358776,
                0.009531663,
                -0.0033557727,
                -0.013696759,
                -0.0068083303,
                0.000826845,
                -0.00088024366,
                0.023121623,
                -0.022894679,
                -0.0007575936,
                0.027900804,
                0.026205396,
                -0.023468714,
                -0.010399391,
                0.013236195,
                -0.013736808,
                0.00040361885,
                0.009812006,
                0.0023512102,
                0.0035443367,
                0.006644797,
                0.02854159,
                -0.019877654,
                0.0043686787,
                -0.021399517,
                0.019784207,
                0.0050461744,
                0.0012173228,
                -0.029102275,
                0.009651809,
                0.009892103,
                0.02130607,
                -0.007462464,
                -0.012048075,
                0.014938277,
                -0.019477164,
                0.016366692,
                -0.019023275,
                -0.017394617,
                -0.025364367,
                -0.019717459,
                -0.012875754,
                -0.024683535,
                0.0143375425,
                0.0059372643,
                -0.00215764,
                0.011053525,
                0.028274596,
                -0.000440539,
                0.003651134,
                0.002746694,
                -0.0031238222,
                0.016914029,
                0.007702758,
                0.012495289,
                -0.025804907,
                0.0013858624,
                -0.02127937,
                0.01922352,
                -0.0145244375,
                -0.015231971,
                -0.028701784,
                0.012008025,
                -0.008270118,
                0.007248869,
                -0.012241645,
                -0.013069324,
                -0.014697984,
                -0.010085674,
                0.003467576,
                0.04378691,
                -0.0034475515,
                0.0076894085,
                -0.028755182,
                0.02730007,
                -0.018302394,
                -0.044427693,
                -0.008470364,
                -0.020278145,
                -0.008750707,
                0.0042785686,
                0.009478264,
                -0.015579062,
                0.0064745885,
                0.016914029,
                -0.008590511,
                -0.0057203323,
                0.003325736,
                0.025711458,
                -0.019891005,
                0.025898354,
                0.029395968,
                0.030463941,
                0.0069618514,
                0.0037913055,
                -0.013883654,
                0.017915253,
                -0.009137847,
                -0.008009801,
                -0.016967427,
                -0.019543912,
                0.006451227,
                -0.0040249247,
                -0.009004351,
                -0.009551687,
                0.0015794325,
                -0.00440539,
                0.018329093,
                0.0046323347,
                -0.008029825,
                -0.00070961827,
                -0.0016853955,
                -0.022801232,
                0.0043219547,
                -0.012435215,
                0.000831434,
                0.015966201,
                0.0007233851,
                -0.017354567,
                -0.026218746,
                -0.0037612687,
                0.0076960833,
                -0.01065971,
                0.018929828,
                -0.02041164,
                0.010259219,
                0.025457816,
                0.010212495,
                0.004051624,
                -0.0126554845,
                -0.029369269,
                0.007736132,
                -0.010265894,
                0.003981538,
                -0.02891538,
                0.027820706,
                -0.01770166,
                -0.018088799,
                0.028327994,
                -0.016299944,
                -0.001733788,
                0.014017151,
                0.015018376,
                0.060714286,
                0.0051930207,
                0.004388703,
                0.009171221,
                -0.00007274526,
                -0.005453339,
                0.017995352,
                -0.013282919,
                -0.02681948,
                0.011307168,
                0.032973677,
                -0.004468801,
                -0.010793206,
                0.0013508195,
                0.029929955,
                0.004308605,
                -0.021359468,
                0.01485818,
                0.016006252,
                0.007742807,
                -0.016833931,
                -0.013523213,
                0.0009761944,
                0.03043724,
                -0.0123751415,
                -0.0050094626,
                -0.024590086,
                0.00802315,
                -0.028141098,
                0.0113472175,
                -0.019944403,
                0.010292593,
                0.0054800385,
                0.0072822436,
                0.026392292,
                -0.016513538,
                0.008597186,
                -0.0004939377,
                -0.009658484,
                0.04530877,
                -0.0026599213,
                0.00036398703,
                -0.0070419493,
                0.013383041,
                -0.010272569,
                -0.0016687084,
                0.0023211734,
                0.022200497,
                -0.026151998,
                -0.0023044862,
                0.0071086977,
                0.0148047805,
                0.028034301,
                -0.009364791,
                0.008977652,
                -0.0026482402,
                -0.003175552,
                -0.00661476,
                0.004989438,
                -0.012788981,
                -0.0115341125,
                -0.011026825,
                -0.0061842334,
                -0.013883654,
                0.009124498,
                0.0042652185,
                -0.020732034,
                -0.038580537,
                0.010793206,
                0.0062209447,
                0.0010554581,
                0.022253895,
                -0.005890541,
                -0.008944277,
                -0.012368467,
                0.025097374,
                -0.033534363,
                0.00045263715,
                0.028354693,
                0.024937179,
                -0.026245445,
                0.012942502,
                0.0034642387,
                0.0062810183,
                0.014604536,
                -0.008356892,
                -0.0026015164,
                -0.013269569,
                -0.014577837,
                -0.010559587,
                -0.0028801907,
                -0.00015602424,
                0.0018205609,
                -0.029983353,
                0.004171771,
                -0.011507413,
                -0.024416542,
                0.016299944,
                -0.004742469,
                0.008730683,
                0.01563246,
                0.004795868,
                0.013910353,
                -0.000772612,
                0.0032689997,
                -0.0046957457,
                -0.0072355196,
                0.01282903,
                -0.0014951627,
                0.037539262,
                -0.008884204,
                -0.015752608,
                -0.011774407,
                -0.008550461,
                -0.015418866,
                0.017541463,
                -0.01759486,
                -0.008290144,
                0.048699588,
                0.03009015,
                0.024136199,
                0.0036611462,
                -0.0035576862,
                -0.030463941,
                0.01223497,
                -0.010092349,
                -0.03174551,
                -0.02768721,
                -0.019090025,
                0.016486838,
                -0.020011151,
                -0.013650035,
                0.0115341125,
                -0.00089859945,
                -0.015071774,
                -0.009892103,
                0.007529212,
                0.0022844619,
                0.011801106,
                0.005500063,
                0.008603861,
                0.0200512,
                0.0013207827,
                0.014097248,
                0.00005850909,
                -0.0010187465,
                0.000001528563,
                -0.035910606,
                -0.005646909,
                -0.004622322,
                -0.028408092,
                -0.01444434,
                0.020651935,
                0.022187147,
                -0.0074023902,
                0.01764826,
                -0.028247897,
                -0.027500315,
                0.017728359,
                -0.010579611,
                -0.024149548,
                0.019770857,
                -0.0053365296,
                -0.030170247,
                0.009237969,
                0.0029903254,
                -0.0018539351,
                -0.016914029,
                0.017034175,
                0.016713783,
                -0.009351442,
                0.017314518,
                -0.005139622,
                -0.020772083,
                -0.006668159,
                -0.033534363,
                0.024723584,
                0.014631235,
                -0.005042837,
                0.016366692,
                0.016767181,
                -0.014831481,
                0.010666384,
                -0.013369692,
                -0.019383717,
                -0.016540237,
                0.021933503,
                -0.018329093,
                0.008777406,
                -0.015472264,
                0.00016322054,
                -0.010339317,
                0.013536562,
                -0.014738033,
                0.0002713737,
                0.0046657086,
                0.027233321,
                0.020958977,
                -0.008457014,
                0.01061966,
                -0.036044102,
                -0.007529212,
                -0.019290268,
                -0.00092696754,
                0.017781757,
                -0.03660479,
                -0.00582713,
                0.009037725,
                -0.0035142999,
                -0.019543912,
                0.018622786,
                0.007949727,
                -0.008103248,
                0.007916353,
                0.217119,
                -0.0062209447,
                -0.01762156,
                0.021519663,
                0.019343669,
                0.042078152,
                0.033854757,
                0.0023361917,
                -0.0057870806,
                0.013736808,
                -0.022147099,
                0.0085771615,
                0.011587511,
                -0.00016770519,
                -0.016647035,
                -0.021359468,
                -0.038553838,
                -0.025591312,
                -0.0054066153,
                -0.008784081,
                -0.0098520545,
                -0.007909678,
                -0.0008468695,
                -0.030383842,
                0.03006345,
                -0.01525867,
                -0.006975201,
                -0.004742469,
                0.012141522,
                0.039114524,
                0.007742807,
                -0.0028434792,
                -0.014137298,
                0.0139637515,
                0.012134847,
                0.011620886,
                0.010032275,
                0.0044487766,
                -0.0024863756,
                0.023041526,
                -0.02968966,
                0.009371466,
                0.02446994,
                -0.0140571995,
                0.006144184,
                -0.0007434096,
                -0.011140297,
                0.0035643613,
                0.022974778,
                0.02364226,
                -0.019423766,
                0.011620886,
                0.01264881,
                0.026418991,
                0.000053450814,
                0.004548899,
                0.015552362,
                -0.003828017,
                -0.009758607,
                0.0013900341,
                0.0012548687,
                0.024536688,
                -0.0059706387,
                -0.014137298,
                0.012915803,
                -0.010639685,
                0.004505513,
                0.014924928,
                0.021786656,
                -0.037859656,
                -0.01061966,
                -0.02971636,
                -0.003674496,
                0.0070419493,
                -0.01926357,
                -0.026993027,
                0.051155925,
                -0.00551675,
                0.042318445,
                0.015458914,
                -0.018609436,
                -0.022587636,
                -0.00670487,
                0.0065546865,
                0.008957626,
                -0.039248023,
                0.016593637,
                0.017287819,
                -0.017367916,
                0.013416415,
                -0.0069284774,
                -0.012168222,
                -0.008183346,
                -0.001187286,
                0.018262345,
                0.031772207,
                0.007502513,
                0.0016269907,
                -0.019570611,
                -0.011901229,
                0.0024246334,
                0.0025264246,
                0.0054700263,
                -0.0055134124,
                -0.054466642,
                -0.00530983,
                0.0074157403,
                -0.012421865,
                0.009765281,
                -0.0031321656,
                0.004458789,
                -0.016780531,
                0.014017151,
                -0.008830805,
                -0.009518312,
                0.007969751,
                0.01969076,
                -0.034121748,
                0.010539562,
                -0.027059775,
                0.0090443995,
                -0.008970977,
                0.0029703008,
                -0.002112585,
                0.014511088,
                -0.016032951,
                0.0050895605,
                -0.023829157,
                0.017181022,
                -0.01840919,
                0.012261669,
                0.0006395325,
                0.034735832,
                0.0026582524,
                -0.013309618,
                0.019196821,
                0.020171348,
                -0.008590511,
                -0.0077161076,
                0.00471577,
                0.0052530942,
                -0.007756157,
                -0.010112373,
                -0.027927503,
                0.0010629672,
                -0.01485818,
                -0.0012882429,
                -0.0021159223,
                -0.011387266,
                -0.014270795,
                -0.0023428665,
                -0.017060874,
                -0.034308646,
                -0.0066915206,
                0.03532322,
                -0.019410416,
                -0.043920405,
                -0.019837605,
                0.014003801,
                0.02766051,
                -0.0015068437,
                -0.0137768565,
                0.026592538,
                -0.014724683,
                -0.0180354,
                0.009237969,
                -0.17194371,
                0.008904228,
                0.010859954,
                -0.003691183,
                0.00802315,
                0.015966201,
                0.026365593,
                -0.0026532463,
                -0.023722358,
                -0.002643234,
                0.0057136575,
                0.010966752,
                -0.046029653,
                -0.010713108,
                -0.004235182,
                0.023175022,
                0.0018539351,
                0.0023328543,
                0.0012990895,
                -0.0020691985,
                0.02888868,
                -0.01807545,
                0.0049994504,
                -0.013696759,
                0.010372692,
                0.0034809257,
                -0.009758607,
                0.0028518226,
                -0.003674496,
                0.01680723,
                -0.016500188,
                0.019130073,
                0.016606987,
                0.008643909,
                0.01525867,
                0.022160448,
                0.009117823,
                -0.0030770984,
                -0.011527438,
                -0.0063043805,
                0.009051074,
                0.00085938483,
                0.01571256,
                0.0059839883,
                -0.021012377,
                0.028381392,
                -0.012909128,
                0.009898778,
                0.025471166,
                -0.0046189846,
                -0.011280469,
                -0.020625236,
                0.012475264,
                -0.00021255173,
                0.023041526,
                0.02245414,
                -0.001985763,
                0.019023275,
                -0.007782856,
                -0.005890541,
                -0.045789357,
                -0.013529887,
                -0.0024696884,
                -0.018102149,
                -0.010753157,
                -0.021172572,
                -0.005466689,
                0.011327192,
                0.004118372,
                0.028114399,
                -0.015018376,
                -0.025457816,
                -0.0059038904,
                -0.007475814,
                0.020611886,
                0.0026348906,
                0.0016161441,
                -0.0022877993,
                0.024509989,
                -0.017354567,
                -0.017354567,
                0.036364492,
                -0.049233574,
                0.0061141476,
                0.008457014,
                -0.00090861175,
                -0.02042499,
                -0.0020625235,
                -0.012301718,
                -0.0032122636,
                0.02329517,
                -0.03129162,
                -0.0020475052,
                -0.009972202,
                0.019023275,
                0.0032890243,
                0.007836254,
                0.017341217,
                -0.0077962056,
                -0.0077962056,
                -0.005810443,
                0.012315068,
                -0.02491048,
                0.026632586,
                0.012034725,
                0.015899453,
                0.012782306,
                0.025791558,
                0.015752608,
                -0.019904355,
                0.004125047,
                0.010058974,
                0.019063326,
                0.025057325,
                -0.021519663,
                -0.016593637,
                -0.0034942755,
                -0.019009925,
                0.02771391,
                0.017888553,
                0.020531788,
                0.0010112373,
                -0.007849604,
                0.009264669,
                -0.0037712809,
                -0.010532888,
                -0.051930204,
                0.0013750157,
                0.011747707,
                0.006090786,
                -0.024056101,
                0.020331543,
                -0.0033724597,
                0.007742807,
                0.011847829,
                0.015592411,
                -0.033828057,
                -0.0019206834,
                -0.00023716518,
                -0.019583961,
                0.040716484,
                -0.018168896,
                0.00042176605,
                -0.0028167798,
                0.0061408468,
                0.016032951,
                -0.006414515,
                -0.02125267,
                0.023228422,
                -0.027313419,
                -0.0007834586,
                -0.00025009768,
                -0.026312195,
                0.0028167798,
                0.017901903,
                0.015739258,
                0.00600735,
                0.0038480416,
                0.008537112,
                -0.007055299,
                0.010833255,
                0.0065213125,
                -0.0060473992,
                -0.0055935103,
                0.013990452,
                -0.015405516,
                -0.0036611462,
                -0.008009801,
                0.0007050293,
                -0.05975311,
                -0.01926357,
                0.0011856173,
                -0.015432215,
                0.009785306,
                -0.00061074726,
                -0.008303493,
                -0.007382366,
                -0.0060407245,
                -0.016246544,
                0.0019106711,
                0.010773181,
                0.011974651,
                -0.008683958,
                0.02573816,
                -0.004074986,
                0.0084837135,
                -0.018102149,
                0.0048626163,
                -0.016540237,
                0.013636685,
                0.025364367,
                -0.0037913055,
                -0.009124498,
                -0.007869629,
                0.009992226,
                -0.004462126,
                -0.016914029,
                0.0074824886,
                -0.013569937,
                0.023442015,
                -0.015458914,
                -0.011567486,
                -0.049073376,
                -0.032653287,
                0.0143375425,
                -0.018168896,
                0.0052430816,
                -0.025310969,
                0.01684728,
                -0.010733132,
                0.0142440945,
                0.025484515,
                -0.002249419,
                -0.0055234246,
                0.011794431,
                -0.026726034,
                0.006714883,
                -0.0068483795,
                0.0014701321,
                -0.028594987,
                -0.00885083,
                -0.029502764,
                0.002895209,
                -0.0031204848,
                -0.0142440945,
                0.013670059,
                -0.033881456,
                -0.014257445,
                -0.09910793,
                0.013363017,
                0.010399391,
                0.004578936,
                0.013156097,
                -0.0077227824,
                0.028167797,
                -0.01850264,
                0.0067315698,
                -0.002483038,
                -0.0068116677,
                0.0044320896,
                0.033667862,
                0.0005907228,
                0.00041404826,
                -0.01768831,
                0.015792657,
                0.01766161,
                0.024229646,
                0.0143375425,
                -0.0128423795,
                -0.002501394,
                0.009304718,
                0.0067282324,
                -0.0065546865,
                -0.0074424394,
                -0.007789531,
                0.022320643,
                -0.0046490217,
                0.013636685,
                0.010065649,
                -0.008196696,
                0.00070544647,
                0.051556416,
                -0.03358776,
                -0.036364492,
                -0.019824255,
                0.009878754,
                -0.007295593,
                0.0055234246,
                -0.031612013,
                -0.027366817,
                0.028274596,
                -0.0040950105,
                -0.018782983,
                0.018929828,
                -0.001662868,
                0.0121882465,
                0.013763507,
                -0.009805331,
                0.014617885,
                0.01003895,
                -0.002117591,
                0.0021376153,
                -0.0005389928,
                -0.003938152,
                0.01340974,
                -0.0066648214,
                -0.002614866,
                -0.017835155,
                0.009117823,
                0.020651935,
                0.025698109,
                0.008290144,
                0.025551263,
                -0.008443665,
                -0.0075892857,
                -0.0010621329,
                -0.004979426,
                -0.0070819985,
                -0.019917704,
                0.013856955,
                0.017514763,
                0.031051327,
                -0.008697309,
                -0.01445769,
                0.0011972983,
                -0.007582611,
                -0.025911704,
                0.023575513,
                0.012715558,
                0.009771956,
                -0.032519788,
                -0.002781737,
                0.00941819,
                -0.017314518,
                -0.025310969,
                -0.0058171176,
                -0.010739807,
                0.025898354,
                -0.0064412146,
                0.018916478,
                0.00842364,
                -0.004245194,
                -0.0125620365,
                0.009745257,
                0.0043519917,
                -0.012949177,
                0.0119079035,
                0.02366896,
                -0.011220396,
                0.016486838,
                -0.0060140253,
                -0.016326644,
                0.0024913817,
                -0.001373347,
                -0.017941954,
                -0.030784333,
                0.015298719,
                0.012775632,
                0.018983226,
                0.022414092,
                0.01561911,
                0.0078295795,
                -0.011146972,
                -0.012141522,
                0.01642009,
                -0.010946727,
                -0.01521862,
                0.008970977,
                0.019570611,
                0.026352243,
                0.009705208,
                -0.011040174,
                0.017861854,
                0.030143548,
                0.008991001,
                0.010192471,
                -0.014377591,
                0.0060740984,
                -0.013376366,
                -0.0067782938,
                -0.02003785,
                -0.0097919805,
                -0.020011151,
                0.011393941,
                -0.027446916,
                0.014484389,
                -0.0180621,
                0.05852494,
                0.01883638,
                -0.0070753237,
                0.023014827,
                -0.008650584,
                0.038660638,
                0.0040649734,
                -0.00075217034,
                -0.018382492,
                -0.029983353,
                -0.0020224745,
                0.0028851968,
                -0.01763491,
                -0.005339867,
                -0.023855856,
                -0.040689785,
                -0.0139637515,
                -0.00571032,
                -0.0038747408,
                -0.000063723804,
                0.02763381,
                -0.010572936,
                0.0071353973,
                0.021519663,
                -0.017047524,
                -0.0076159853,
                0.00013433103,
                0.0068350295,
                -0.007809555,
                -0.014310843,
                0.007989775,
                0.0034408767,
                -0.03540332,
                -0.027313419,
                0.00088274677,
                -0.022267245,
                -0.0013841937,
                -0.008143297,
                -0.0019824256,
                0.0063377544,
                -0.017968653,
                0.031879004,
                -0.0120414,
                -0.031078026,
                0.023842506,
                -0.021025727,
                0.007916353,
                0.014724683,
                -0.029502764
            ],
            "DescriptionVector":[
                -0.013993491,
                0.0017941656,
                0.000053725013,
                -0.0031968465,
                -0.00025238263,
                0.005813963,
                0.00025883794,
                0.0015517783,
                0.0019624205,
                -0.0023005966,
                0.009009143,
                0.0009412289,
                -0.017911669,
                0.015446149,
                -0.016578956,
                -0.019124439,
                0.031425383,
                -0.011574617,
                -0.0024255386,
                -0.018724624,
                -0.019111112,
                0.00018335223,
                0.0059272433,
                0.002788703,
                -0.018458081,
                0.02097691,
                0.0025688051,
                -0.012061057,
                0.018964512,
                0.0036982798,
                0.0001381649,
                -0.00054141483,
                -0.018831242,
                -0.0008795909,
                0.0036116536,
                -0.0117678605,
                0.004524562,
                -0.007942973,
                0.011148148,
                0.0020523788,
                0.026134511,
                -0.002308926,
                0.0043379823,
                0.0049377033,
                -0.02209639,
                0.009515574,
                -0.00081920234,
                0.0055107703,
                -0.015112971,
                -0.0071833255,
                -0.0032801412,
                0.0201373,
                -0.010188594,
                -0.029053153,
                0.0004447931,
                0.0031901828,
                -0.009335658,
                0.0038082288,
                0.011867814,
                -0.022642802,
                0.0025005036,
                0.007856346,
                -0.024895089,
                -0.0096155275,
                -0.011334728,
                0.0064170156,
                -0.043366496,
                0.023522394,
                0.025614753,
                0.021829847,
                0.022389587,
                0.006253758,
                -0.002187316,
                -0.010641717,
                0.046618316,
                -0.0041214162,
                -0.02321587,
                0.010741671,
                0.019084457,
                0.009049124,
                -0.003143538,
                -0.041100882,
                -0.007689757,
                0.019284364,
                0.017098714,
                0.003988145,
                -0.021216799,
                0.021190144,
                -0.030012708,
                -0.0049210447,
                0.006347048,
                0.02609453,
                0.005437471,
                -0.0057973037,
                -0.031798545,
                0.004577871,
                -0.0070567178,
                0.022416241,
                -0.00057098444,
                -0.00060805056,
                -0.0025754687,
                0.020856965,
                -0.0016508988,
                -0.01441996,
                -0.025548117,
                -0.014793119,
                -0.0141134355,
                -0.015712691,
                0.01641903,
                0.0006888463,
                -0.0063803657,
                0.012973966,
                0.010635054,
                -0.027960328,
                -0.0333978,
                0.010548427,
                0.003924841,
                -0.018498063,
                -0.008742601,
                -0.033291183,
                0.015046335,
                0.008482722,
                0.010508446,
                -0.010002014,
                0.006287076,
                0.016019216,
                -0.015646057,
                -0.01860468,
                0.0132271815,
                -0.0044479314,
                0.016245777,
                0.0018291492,
                0.027373934,
                0.007703084,
                -0.01701875,
                -0.0032218348,
                -0.03446397,
                0.018911205,
                -0.0016983767,
                -0.000017635028,
                0.00988207,
                0.011974431,
                -0.015606075,
                0.002387223,
                -0.012307609,
                0.010208585,
                0.008709283,
                0.028360143,
                0.020510461,
                0.0025738028,
                0.023935534,
                -0.02628111,
                0.014966372,
                -0.010715016,
                0.02400217,
                0.019017821,
                -0.0060405238,
                0.018591354,
                -0.032065086,
                -0.013107237,
                -0.021709902,
                0.017378584,
                -0.0071699987,
                0.0008920851,
                0.006790175,
                0.025987914,
                0.019124439,
                0.021803193,
                -0.004238029,
                -0.0016733884,
                0.006766853,
                0.026760887,
                -0.029452968,
                0.011481327,
                0.0037382613,
                0.014673175,
                -0.013926856,
                -0.0045812028,
                -0.011741206,
                0.0013318806,
                -0.010568418,
                -0.0004422943,
                0.015286224,
                0.039501626,
                0.00012723249,
                -0.0019474276,
                0.026521,
                0.053335194,
                0.009555556,
                -0.016658919,
                0.010928251,
                0.02032388,
                0.0062604216,
                -0.008522702,
                -0.6746729,
                0.00086209906,
                -0.029612893,
                0.0032801412,
                0.008462731,
                0.020017356,
                0.030492485,
                0.0118211685,
                0.019097785,
                -0.022936,
                -0.001339377,
                -0.0015817643,
                0.0061238185,
                -0.012247637,
                -0.0057406635,
                -0.027187355,
                0.004997676,
                -0.018058268,
                -0.022149699,
                0.017205331,
                0.0074365414,
                0.011108167,
                -0.0004098094,
                -0.0018191539,
                0.04643174,
                0.011534635,
                0.0034284054,
                -0.0024388656,
                -0.032518208,
                -0.015272897,
                -0.016152488,
                0.019577561,
                0.0154328225,
                0.0016508988,
                0.04227367,
                -0.0023639007,
                -0.01985743,
                0.026547652,
                0.002478847,
                0.03241159,
                0.005517434,
                -0.009442275,
                0.018204866,
                0.004834418,
                0.025601426,
                0.0030252596,
                0.009342321,
                0.0024072137,
                -0.007849682,
                -0.009715482,
                -0.0084160855,
                -0.014473269,
                -0.0067435303,
                0.005973888,
                0.028226871,
                -0.0061704633,
                0.02172323,
                -0.02250953,
                -0.012607469,
                0.0039181774,
                0.017138695,
                0.011074849,
                -0.0068967924,
                -0.033051293,
                -0.0001923064,
                0.008622657,
                -0.019404309,
                0.041740585,
                0.033424456,
                0.002727065,
                -0.012374245,
                0.031452037,
                -0.016285758,
                0.009442275,
                -0.010788316,
                0.027560515,
                0.010861615,
                -0.008262823,
                0.010388502,
                0.00948892,
                -0.011874477,
                -0.006373702,
                -0.012540834,
                0.032491554,
                0.02688083,
                0.00073674065,
                -0.027347282,
                -0.014526577,
                0.011547962,
                -0.021936463,
                0.021043546,
                0.0025188285,
                -0.0130539285,
                0.00042251183,
                0.0023622348,
                0.030732373,
                -0.010435146,
                0.025268247,
                0.0150863165,
                -0.051496048,
                -0.022389587,
                0.0066802264,
                0.004301333,
                -0.0075564855,
                0.0040547806,
                0.020497134,
                -0.020803658,
                0.020163955,
                0.016485665,
                0.008942507,
                0.019510925,
                -0.002788703,
                -0.019804122,
                0.0048244228,
                0.012267628,
                -0.033077948,
                -0.009422285,
                0.0079563,
                0.00058264565,
                -0.0237223,
                0.0025488145,
                -0.00071216875,
                0.016805517,
                -0.0220031,
                -0.019684179,
                0.022469549,
                -0.0009478925,
                -0.02385557,
                -0.025214938,
                0.028866574,
                0.015152953,
                0.01358035,
                0.02497505,
                -0.015526112,
                0.023269176,
                0.0112481015,
                0.004977685,
                -0.018111575,
                0.012933984,
                -0.01981745,
                -0.012207655,
                -0.012121029,
                0.0022123044,
                -0.0037082753,
                -0.013780258,
                -0.025748024,
                0.020896947,
                -0.0152595695,
                -0.03265148,
                -0.009515574,
                -0.003128545,
                -0.0040747714,
                -0.031798545,
                0.020070665,
                0.013207191,
                0.013107237,
                -0.0018241516,
                -0.011468,
                -0.004004804,
                -0.020203937,
                0.00014857673,
                -0.006100496,
                -0.020870293,
                0.008209515,
                -0.014393305,
                -0.01739191,
                0.002650434,
                -0.0073565785,
                -0.0075165043,
                -0.018178212,
                0.04088765,
                -0.0006109658,
                0.016538974,
                0.024708508,
                0.007962963,
                0.0053041996,
                0.0027737098,
                0.017565163,
                0.0034317372,
                -0.0074565317,
                -0.012021076,
                -0.0048610726,
                -0.015686037,
                -0.022642802,
                0.024801798,
                -0.008709283,
                -0.0009029134,
                0.017245313,
                -0.01991074,
                0.0096488455,
                -0.010435146,
                -0.01083496,
                -0.011874477,
                -0.010561754,
                -0.012734077,
                -0.004021463,
                0.0061438093,
                -0.007989617,
                -0.013387107,
                0.0073365876,
                0.03102557,
                0.012274291,
                0.01697877,
                -0.0016034209,
                0.024028825,
                -0.018244848,
                0.013147218,
                -0.020630404,
                0.037156053,
                0.0076297848,
                0.009142415,
                -0.021869829,
                -0.028226871,
                0.002958624,
                0.019284364,
                0.005160933,
                -0.009515574,
                -0.0038682008,
                -0.007623121,
                0.014753139,
                0.028173562,
                0.002833682,
                0.0017391911,
                -0.0002746639,
                -0.020763677,
                0.003804897,
                0.006287076,
                0.0015134627,
                -0.010381838,
                -0.0036449714,
                -0.010028669,
                0.021749884,
                -0.02162994,
                0.0149130635,
                0.00427801,
                -0.0075431583,
                -0.013686967,
                -0.025774678,
                0.010335193,
                -0.011074849,
                0.01688548,
                0.0166989,
                0.021616613,
                0.0055107703,
                0.012161011,
                -0.013273826,
                0.011134821,
                0.0074631954,
                -0.035316907,
                -0.018391445,
                0.0018241516,
                0.024228731,
                -0.009375639,
                -0.0097887805,
                0.0026237797,
                0.017871687,
                0.006870138,
                -0.0024855107,
                0.0034067489,
                0.021856502,
                -0.021963118,
                0.011841159,
                0.020657059,
                0.0042313654,
                0.019750813,
                -0.021896483,
                0.012707423,
                -0.017751744,
                -0.0049810167,
                0.008469394,
                0.008342787,
                -0.016005889,
                0.01907113,
                0.0008379436,
                -0.0037949015,
                -0.002387223,
                -0.008915853,
                0.004188052,
                -0.040061366,
                0.029826127,
                -0.012574152,
                -0.011448009,
                0.023962189,
                0.022456221,
                0.00209236,
                -0.0010586743,
                -0.018551372,
                0.01697877,
                -0.018578026,
                0.023549046,
                0.007823029,
                0.010321866,
                -0.0055107703,
                -0.004591198,
                0.013980164,
                -0.0081895245,
                0.008709283,
                -0.02097691,
                0.0051809237,
                -0.0068201614,
                0.0100486595,
                -0.02581466,
                0.009682164,
                -0.025374865,
                0.032944676,
                -0.0027903689,
                -0.018204866,
                -0.008309469,
                -0.0006409519,
                -0.0039681545,
                0.016618937,
                0.008282814,
                0.008396095,
                -0.021456687,
                0.016059197,
                0.007030064,
                -0.01753851,
                0.0023555711,
                0.024175422,
                -0.0146865025,
                -0.029452968,
                -0.01869797,
                -0.008476058,
                -0.013167209,
                0.113973655,
                0.017964978,
                0.01130141,
                -0.00019449288,
                0.011847823,
                0.0009795444,
                -0.025214938,
                -0.018964512,
                -0.0063803657,
                0.0033367814,
                0.010255231,
                0.015472803,
                0.025428174,
                0.018524718,
                -0.005857276,
                -0.0019007826,
                -0.016538974,
                -0.011141485,
                0.019710833,
                0.009568883,
                -0.00541748,
                0.011514644,
                -0.0034150784,
                0.014300016,
                0.006133814,
                -0.0050643114,
                0.034117464,
                0.009315668,
                -0.018631335,
                -0.010768325,
                0.009495583,
                -0.0048110955,
                -0.0028886565,
                0.004917713,
                -0.02497505,
                0.0148597555,
                0.0030219278,
                -0.031292114,
                0.01609918,
                -0.0060805054,
                0.026947467,
                -0.0052642184,
                -0.0069567645,
                0.005810631,
                -0.0037882382,
                0.016472338,
                -0.019231055,
                0.029506277,
                -0.030252596,
                -0.0064803194,
                0.021936463,
                0.021230126,
                -0.012647451,
                -0.0054907794,
                0.007863009,
                0.0046311794,
                0.013180536,
                -0.018044941,
                -0.0041080895,
                -0.0014609871,
                -0.01888455,
                -0.021696575,
                -0.013073919,
                -0.012307609,
                -0.008869208,
                -0.013873547,
                -0.02097691,
                -0.016858825,
                -0.009055788,
                0.0030419186,
                0.018991167,
                -0.006636913,
                -0.027107392,
                -0.0059872153,
                0.043872926,
                0.012334263,
                0.0005759821,
                0.0027470556,
                0.0062770806,
                0.009782117,
                0.0074498686,
                0.011541299,
                -0.007783047,
                -0.013273826,
                -0.020563768,
                0.013220518,
                0.0017275298,
                -0.02688083,
                -0.005730668,
                0.0022989307,
                0.0015142957,
                -0.0028936542,
                -0.0027853712,
                0.006856811,
                -0.015992561,
                -0.031878505,
                0.003961491,
                -0.0081895245,
                -0.0004714474,
                -0.014833101,
                -0.00215733,
                0.004761119,
                -0.02828018,
                -0.0072366344,
                0.026307764,
                -0.0069234464,
                -0.017605145,
                -0.03027925,
                -0.0039181774,
                0.011081513,
                0.019284364,
                0.008142879,
                -0.009448939,
                -0.0042546876,
                -0.027000776,
                0.03121215,
                -0.007996281,
                0.01688548,
                -0.00237223,
                0.009009143,
                -0.0052475594,
                -0.028679995,
                0.01823152,
                0.025987914,
                0.003236828,
                -0.024921741,
                0.017431892,
                -0.00062595884,
                -0.0013660314,
                -0.008182861,
                -0.0054874476,
                -0.014779792,
                -0.0060638464,
                0.005054316,
                -0.014473269,
                -0.018044941,
                -0.042966682,
                -0.010281884,
                -0.025268247,
                -0.0128806755,
                -0.011474663,
                -0.0063670385,
                0.011274756,
                -0.00853603,
                -0.02456191,
                -0.027960328,
                0.004884395,
                -0.004174725,
                -0.0014260034,
                0.023735628,
                0.025294902,
                0.00076797616,
                -0.02139005,
                0.0036616302,
                0.0044712536,
                -0.016299086,
                -0.014966372,
                -0.004467922,
                -0.013793585,
                0.027960328,
                0.041687276,
                -0.0005289207,
                0.027853712,
                0.0023339144,
                -0.021016892,
                0.0030435845,
                -0.01090826,
                -0.0043946225,
                -0.026894158,
                0.028973192,
                0.03209174,
                0.00855602,
                0.012587478,
                -0.012101038,
                -0.020430498,
                -0.0024738493,
                -0.008895863,
                -0.008209515,
                -0.031505346,
                -0.016205795,
                0.016872153,
                -0.02865334,
                0.013700294,
                -0.0014001821,
                0.009049124,
                0.0026637611,
                0.039928097,
                0.0119077945,
                -0.0018574694,
                0.016525647,
                -0.0006097164,
                -0.018404773,
                0.019604215,
                0.0050709746,
                -0.003266814,
                -0.002418875,
                -0.019311018,
                -0.014180072,
                -0.023082597,
                0.019297691,
                0.008902526,
                -0.0013227181,
                -0.004917713,
                -0.013806912,
                -0.013873547,
                0.02116349,
                -0.0062437626,
                -0.010701689,
                0.008582675,
                -0.026014566,
                -0.049870137,
                -0.024508601,
                -0.0023322485,
                -0.0076497756,
                0.017138695,
                0.008682628,
                -0.023015961,
                0.011967767,
                -0.011101503,
                -0.023135906,
                0.013660314,
                -0.011974431,
                0.023535721,
                -0.0037515885,
                0.01623245,
                0.03027925,
                0.0064136838,
                -0.014020146,
                0.034730513,
                -0.0011111499,
                -0.017871687,
                -0.0069567645,
                -0.0089891525,
                0.03227832,
                -0.028493414,
                -0.0021656593,
                0.02878661,
                -0.004301333,
                -0.018524718,
                0.03784906,
                -0.0074765226,
                0.0096488455,
                -0.004341314,
                0.009022471,
                0.012094375,
                0.02753386,
                -0.012187664,
                -0.008702619,
                -0.019924067,
                -0.009222377,
                -0.033291183,
                0.0103951655,
                -0.028360143,
                0.0030585774,
                -0.0095622195,
                -0.0014093445,
                0.012154347,
                -0.039421666,
                -0.028973192,
                0.019977376,
                -0.016125834,
                0.023868898,
                -0.044592593,
                0.008862545,
                0.0025971255,
                0.015152953,
                -0.006373702,
                0.010521773,
                -0.0012869014,
                0.0150330085,
                -0.012947312,
                -0.024441965,
                0.027613824,
                -0.0027970325,
                0.00017793807,
                -0.011308074,
                -0.009888734,
                -0.004238029,
                -0.0030402527,
                -0.012794049,
                -0.008895863,
                0.009328994,
                -0.0073565785,
                0.011601271,
                0.01083496,
                -0.03545018,
                -0.01100155,
                0.0033834265,
                -0.008742601,
                -0.013167209,
                -0.0050243298,
                -0.024242058,
                -0.0067535257,
                -0.00227894,
                0.025388192,
                0.016752208,
                -0.0083294595,
                0.016618937,
                -0.013660314,
                0.009375639,
                -0.018178212,
                0.020337207,
                -0.01460654,
                0.005820626,
                -0.01034852,
                0.004717806,
                0.027427243,
                0.0014668178,
                -0.029053153,
                -0.003155199,
                0.01673888,
                0.0005834786,
                0.0021006896,
                0.011514644,
                -0.0037615837,
                -0.008422749,
                0.011141485,
                0.007483186,
                -0.031452037,
                0.009888734,
                -0.013527042,
                -0.023282504,
                0.0071566715,
                -0.015779328,
                -0.004854409,
                0.06210445,
                -0.0149130635,
                -0.01376693,
                -0.018031614,
                -0.0032135055,
                -0.01641903,
                0.030759027,
                -0.00039252578,
                0.008802572,
                -0.014579886,
                -0.0044312724,
                -0.016019216,
                0.02125678,
                -0.0026870836,
                -0.009482256,
                0.013593677,
                0.016205795,
                0.0069700917,
                0.020630404,
                -0.0063070664,
                -0.013613668,
                -0.0069367737,
                -0.0059805517,
                -0.04720471,
                -0.026134511,
                -0.034170773,
                0.013080583,
                0.01707206,
                -0.016312413,
                -0.025827987,
                0.01879126,
                -0.018724624,
                -0.025148304,
                -0.015579421,
                -0.00034692197,
                0.023748955,
                -0.002032388,
                -0.018124903,
                0.002308926,
                -0.011721215,
                0.01423338,
                0.008002944,
                0.006290408,
                0.026640942,
                -0.025548117,
                -0.0063670385,
                -0.006563614,
                -0.032491554,
                -0.0063836975,
                0.0132804895,
                -0.0055640787,
                0.03302464,
                -0.014246707,
                -0.008855881,
                -0.003881528,
                0.0016692237,
                0.0118211685,
                0.02144336,
                0.032678135,
                0.008176197,
                0.0042546876,
                -0.013340462,
                0.007729738,
                -0.006310398,
                -0.028520068,
                0.0033700992,
                -0.0024855107,
                -0.016898807,
                0.028733302,
                -0.0019440958,
                -0.0058406168,
                0.020150628,
                0.002218968,
                0.01414009,
                -0.0047811097,
                -0.012260964,
                0.007110026,
                0.0154861305,
                -0.0070966994,
                -0.0074365414,
                0.0048077637,
                -0.045712072,
                -0.0054108165,
                -0.025974587,
                -0.02600124,
                -0.0031402062,
                -0.029612893,
                0.011128157,
                -0.0023472416,
                0.0066169226,
                -0.0010703355,
                -0.00021198473,
                -0.00012587896,
                0.011008213,
                0.03326453,
                -0.02441531,
                0.014819774,
                -0.029719511,
                0.02609453,
                -0.012360917,
                -0.007329924,
                0.006510305,
                -0.015899273,
                -0.033184566,
                0.02456191,
                -0.013900202,
                -0.0192044,
                0.016565628,
                -0.009435612,
                0.002328917,
                0.025468154,
                0.22730762,
                0.0036616302,
                -0.025854642,
                0.033744305,
                0.009155742,
                0.010921587,
                0.013267162,
                0.019244382,
                -0.0042746784,
                0.010801643,
                0.00006830157,
                0.023402449,
                -0.0015925926,
                -0.000839193,
                0.0039714864,
                -0.03427739,
                -0.05325523,
                -0.034304045,
                0.003881528,
                -0.0033800947,
                0.013780258,
                -0.030199287,
                0.0025321557,
                -0.008662637,
                0.021936463,
                -0.0050676432,
                -0.03065241,
                -0.005177592,
                0.03995475,
                0.040621106,
                0.006606927,
                0.014086781,
                0.0036749574,
                -0.015792655,
                0.014539904,
                0.004624516,
                -0.0022073067,
                -0.009508911,
                0.006683558,
                -0.0023805595,
                -0.0061704633,
                -0.010468464,
                0.0014618201,
                -0.015232915,
                0.016658919,
                -0.0061371457,
                -0.016912134,
                0.019417636,
                0.025574772,
                0.021283435,
                -0.024508601,
                -0.012001085,
                -0.0058439486,
                0.019177748,
                0.014126763,
                -0.0068967924,
                0.0011044863,
                -0.00015690619,
                -0.02377561,
                0.001602588,
                -0.025174957,
                0.016165815,
                -0.035503488,
                0.013620332,
                -0.007030064,
                0.007916318,
                -0.022389587,
                0.013660314,
                0.012560825,
                0.0060471874,
                0.012794049,
                -0.053468466,
                -0.013180536,
                0.015766,
                -0.010568418,
                -0.012940648,
                0.020163955,
                -0.011121494,
                0.0002334331,
                0.020177282,
                -0.010175267,
                0.030812336,
                -0.022056408,
                0.014993027,
                0.0073099337,
                -0.023562374,
                0.013886875,
                0.009982024,
                -0.00086459785,
                -0.033077948,
                -0.0080162715,
                -0.016112506,
                -0.02125678,
                0.011268092,
                0.0034100807,
                0.0013893538,
                -0.0043879594,
                0.007922982,
                0.0039415,
                0.009755462,
                -0.006870138,
                0.018178212,
                -0.007703084,
                0.006626918,
                -0.030119324,
                -0.00029319696,
                -0.013200527,
                -0.006890129,
                0.026547652,
                0.012101038,
                -0.008356114,
                0.002233961,
                0.0033617697,
                -0.0078430185,
                -0.0028586704,
                0.0053275223,
                0.018631335,
                -0.010608399,
                0.012694096,
                -0.025428174,
                0.029452968,
                -0.0005530761,
                0.0037949015,
                0.014206726,
                -0.01785836,
                -0.005254223,
                -0.011547962,
                0.014459942,
                0.0020057338,
                -0.04094096,
                -0.00034275724,
                -0.006823493,
                0.0073699057,
                -0.007483186,
                -0.0104484735,
                -0.013673641,
                0.0096488455,
                -0.0038915235,
                -0.04413947,
                -0.00002336153,
                -0.0005497443,
                -0.0103618475,
                0.00030610763,
                -0.0033034636,
                0.021230126,
                -0.048963893,
                0.0062837442,
                -0.005760654,
                -0.0058406168,
                0.0010570084,
                -0.002572137,
                -0.008262823,
                -0.03083899,
                -0.0048610726,
                0.010041996,
                -0.015566094,
                -0.0324649,
                -0.013487061,
                0.004701147,
                -0.02753386,
                -0.0333978,
                -0.013913529,
                0.017138695,
                -0.027000776,
                -0.003405083,
                0.018444754,
                -0.17133364,
                0.03971486,
                0.00007720367,
                -0.028066946,
                0.036543004,
                -0.008549357,
                0.030119324,
                0.005407485,
                -0.0063403845,
                0.019284364,
                0.0005664032,
                -0.011761197,
                -0.03768914,
                -0.007189989,
                0.010561754,
                0.0024388656,
                -0.012987293,
                -0.00030464996,
                -0.004761119,
                0.0096755,
                0.017991632,
                0.008542693,
                -0.006390361,
                -0.0050876336,
                0.0140468,
                0.008942507,
                -0.000018754297,
                0.023362467,
                -0.012760731,
                -0.017485201,
                -0.0018891214,
                0.011501317,
                0.025761351,
                0.005517434,
                0.0107949795,
                -0.0006996746,
                -0.0039115143,
                -0.016379049,
                -0.010261894,
                -0.0013743608,
                -0.0023738958,
                0.024508601,
                0.013846893,
                -0.00089874864,
                -0.009342321,
                0.0132605,
                0.0065436233,
                0.021829847,
                0.014486596,
                -0.007989617,
                0.020057337,
                -0.008222843,
                -0.0023322485,
                0.013233845,
                0.004081435,
                0.010288548,
                0.0038715326,
                0.006730203,
                -0.0012177669,
                -0.0004943534,
                -0.0010903262,
                -0.02041717,
                0.0009462266,
                -0.013180536,
                0.0065402915,
                -0.023748955,
                -0.007050054,
                -0.018484736,
                -0.0052075777,
                0.0051842555,
                0.003711607,
                -0.013913529,
                -0.012240973,
                -0.009622191,
                -0.008382767,
                0.0029236402,
                0.009442275,
                -0.026121184,
                0.006073842,
                -0.030892298,
                -0.017005425,
                0.014033473,
                -0.015019681,
                -0.010568418,
                -0.0053808307,
                -0.012247637,
                -0.00063678715,
                0.022242988,
                0.025094995,
                -0.0071966527,
                0.00900248,
                -0.003066907,
                -0.0063270573,
                -0.023882225,
                0.00834945,
                0.030599102,
                0.022149699,
                0.026827522,
                -0.0037815745,
                -0.01242089,
                0.001602588,
                0.011707888,
                -0.026534325,
                -0.006513637,
                0.024508601,
                0.027453898,
                -0.0019457617,
                0.03446397,
                0.039688207,
                0.0007188323,
                -0.0077963737,
                0.02111018,
                0.024042152,
                0.019870758,
                -0.0020973578,
                0.03414412,
                0.010708353,
                -0.02153665,
                0.0201373,
                0.024148768,
                0.04856408,
                -0.009968697,
                -0.008062917,
                0.01828483,
                -0.021043546,
                0.00607051,
                -0.09334325,
                -0.015192933,
                0.012580816,
                -0.0040147994,
                0.0018074927,
                0.018164884,
                0.012354254,
                -0.013926856,
                -0.032571517,
                0.0075231674,
                0.00042501066,
                -0.0318252,
                -0.023335813,
                0.00023947195,
                0.025881296,
                0.01118813,
                -0.0043346505,
                -0.02088362,
                -0.009982024,
                -0.009189059,
                -0.0049643577,
                -0.013633659,
                0.0038248878,
                -0.007363242,
                -0.004561212,
                0.02377561,
                -0.021430032,
                0.021190144,
                0.009902061,
                0.01309391,
                0.0069700917,
                -0.0057439953,
                0.024175422,
                -0.0039181774,
                0.018498063,
                0.0016808849,
                0.010761661,
                -0.0031785218,
                0.018618008,
                -0.004804432,
                0.0103618475,
                0.008935844,
                0.0010378506,
                -0.035316907,
                0.0037315977,
                0.0015601077,
                -0.008269487,
                0.007443205,
                0.016205795,
                -0.00071591703,
                -0.02377561,
                0.005870603,
                -0.016552301,
                -0.015206261,
                0.00035067025,
                -0.012594142,
                -0.008469394,
                -0.0013635325,
                -0.011294747,
                -0.0008704285,
                0.011221448,
                0.008456067,
                -0.006950101,
                -0.0028270185,
                0.018324811,
                0.0020090656,
                -0.0104484735,
                -0.017085386,
                0.014926391,
                -0.017192004,
                -0.0014684837,
                0.015126298,
                0.009935379,
                0.00018991168,
                -0.006830157,
                0.0032284984,
                -0.02390888,
                -0.0026337751,
                -0.0099620335,
                -0.01632574,
                0.0007692256,
                -0.0333978,
                0.017791726,
                -0.009249032,
                0.03033256,
                0.017831706,
                -0.007689757,
                0.008882536,
                0.010901596,
                -0.006903456,
                0.004018131,
                -0.0060271965,
                -0.0004893557,
                -0.021136835,
                -0.0063637067,
                0.025801333,
                0.0060905004,
                -0.013207191,
                -0.0068368204,
                -0.024588564,
                -0.025774678,
                -0.000439379,
                -0.067222066,
                0.012214319,
                -0.02125678,
                0.015566094,
                -0.01595258,
                -0.02828018,
                0.019550906,
                0.022389587,
                -0.003128545,
                0.04059445,
                -0.021123508,
                -0.0041713933,
                0.021736557,
                -0.0005176759,
                -0.037609175,
                -0.014779792,
                0.025801333,
                -0.0032568185,
                0.029559584,
                0.019377654,
                -0.011294747,
                0.01813823,
                0.025481481,
                -0.00062346004,
                -0.002327251,
                0.009902061,
                -0.011354719,
                0.026267782,
                -0.003311793,
                0.0018891214,
                0.025108323,
                -0.014539904,
                0.008076244,
                0.041234154,
                -0.019217728,
                -0.028626686,
                -0.008002944,
                0.0046478384,
                0.020150628,
                0.016125834,
                -0.012307609,
                -0.038035642,
                0.005097629,
                -0.009902061,
                -0.0098754065,
                0.010215249,
                -0.004018131,
                0.0034117466,
                0.026734233,
                -0.001802495,
                -0.000050393228,
                0.021363396,
                0.00049393694,
                -0.02334914,
                0.017978305,
                -0.0061238185,
                -0.0040447856,
                -0.012107702,
                0.011641252,
                -0.012660778,
                0.028440105,
                -0.023269176,
                -0.010028669,
                -0.011494654,
                -0.00065969315,
                -0.008462731,
                0.004634511,
                -0.0029836122,
                0.0016850496,
                -0.009029134,
                -0.03033256,
                0.000103337356,
                0.016205795,
                0.026427709,
                0.014966372,
                -0.027187355,
                0.0002665427,
                -0.011208121,
                -0.030172633,
                0.026560979,
                0.001079498,
                0.02153665,
                -0.02990609,
                0.044086162,
                0.02209639,
                0.011947776,
                -0.011887805,
                -0.009222377,
                0.011274756,
                0.013527042,
                0.006823493,
                0.018964512,
                0.00026216972,
                -0.0038848598,
                0.002665427,
                -0.0035216953,
                0.022896018,
                -0.031532,
                0.010635054,
                0.020363862,
                0.0018807919,
                0.017791726,
                -0.006410352,
                -0.018324811,
                -0.0055207657,
                -0.008615993,
                -0.018324811,
                -0.013080583,
                0.010328529,
                -0.024401983,
                0.03720936,
                0.0077763833,
                0.033850923,
                0.019297691,
                -0.015872618,
                -0.018764606,
                0.009195723,
                0.0073699057,
                -0.017605145,
                0.018804587,
                0.0017525182,
                0.004621184,
                0.019897413,
                -0.0033017977,
                0.0030152642,
                0.022163026,
                0.006093832,
                -0.016112506,
                -0.008009608,
                -0.003541686,
                0.009595537,
                0.0058239577,
                -0.013900202,
                -0.002728731,
                0.00836944,
                -0.0038315512,
                -0.0051642647,
                -0.004574539,
                -0.023509067,
                0.06301069,
                0.011408027,
                -0.009682164,
                0.0047811097,
                -0.0032235007,
                0.02758717,
                -0.009662173,
                -0.016645592,
                -0.029932745,
                -0.012680769,
                0.019884085,
                -0.003664962,
                -0.0091624055,
                -0.0036749574,
                -0.02418875,
                -0.00061554706,
                -0.0065036416,
                -0.009315668,
                -0.015739346,
                -0.0080162715,
                0.009642182,
                0.0042147064,
                0.0092157135,
                0.0096488455,
                -0.015139625,
                0.001787502,
                0.0020390516,
                0.018018287,
                0.014899737,
                -0.020643732,
                0.023975516,
                0.014953045,
                -0.038995195,
                -0.020390516,
                -0.011068186,
                -0.024348676,
                -0.00089291803,
                -0.004114753,
                0.008895863,
                0.036383078,
                -0.019217728,
                0.03768914,
                -0.017711762,
                -0.020670386,
                -0.000011908523,
                -0.009575547,
                -0.0077963737,
                0.008176197,
                0.0035283589
            ]
    },
    {
    "@search.action": "upload",
    "HotelId": "4",
    "HotelName": "Sublime Cliff Hotel",
    "Description": "Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.",
    "Description_fr": "Le sublime Cliff Hotel est situé au coeur du centre historique de sublime dans un quartier extrêmement animé et vivant, à courte distance de marche des sites et monuments de la ville et est entouré par l'extraordinaire beauté des églises, des bâtiments, des commerces et Monuments. Sublime Cliff fait partie d'un Palace 1800 restauré avec amour.",
    "Category": "Boutique",
    "Tags": [ "concierge", "view", "24-hour front desk service" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "1960-02-06T00:00:00Z",
    "Rating": "4.60",
    "Address": {
        "StreetAddress": "7400 San Pedro Ave",
        "City": "San Antonio",
        "StateProvince": "TX",
        "PostalCode": "78216",
        "Country": "USA"
        },
        "HotelNameVector":[
                0.0019320576,
                0.003545151,
                0.0074358466,
                -0.007987388,
                -0.0003997432,
                0.040162876,
                -0.028892817,
                -0.0077149402,
                0.020546593,
                0.002689597,
                0.004492075,
                0.013057586,
                0.0073361704,
                -0.0078079714,
                -0.012140565,
                -0.017689208,
                0.022314185,
                -0.027204966,
                0.0038408574,
                -0.006565341,
                -0.028839655,
                0.018951774,
                0.028653594,
                -0.004575139,
                -0.0076484894,
                -0.0013182181,
                0.013243648,
                -0.006419149,
                -0.007562103,
                0.016639285,
                0.014712212,
                -0.023045145,
                -0.030142091,
                -0.01375532,
                -0.022327475,
                -0.016838638,
                -0.018154364,
                -0.013296809,
                -0.0019569767,
                0.0049671987,
                -0.0010690276,
                -0.024533642,
                -0.007887713,
                -0.018207524,
                0.0017891884,
                0.020466853,
                0.018300556,
                0.008731638,
                -0.037797228,
                0.02353688,
                0.009675239,
                0.0031431238,
                -0.024985507,
                -0.013715449,
                0.0056649325,
                0.005053585,
                -0.008857894,
                0.015895035,
                0.014805242,
                -0.0033956368,
                0.0037976643,
                -0.03091292,
                -0.031231886,
                0.0036381823,
                -0.0105523905,
                0.010678647,
                -0.013502806,
                0.005189809,
                -0.0005058569,
                0.0023058434,
                0.038674377,
                0.024028616,
                -0.0054888376,
                -0.01950996,
                0.013675579,
                -0.02099846,
                -0.015802005,
                0.002345714,
                0.018048042,
                0.0023855844,
                0.010346392,
                -0.020772526,
                -0.008107,
                -0.005694835,
                0.02430771,
                0.012034243,
                -0.001382177,
                -0.009190149,
                -0.014499568,
                -0.020546593,
                0.014659051,
                0.027430898,
                0.0050801653,
                0.011223543,
                -0.031045822,
                0.003169704,
                0.00975498,
                0.007535523,
                0.010412843,
                0.0012733638,
                -0.0007592006,
                0.023085015,
                -0.018420167,
                -0.009156923,
                -0.043485414,
                -0.005827737,
                -0.006621824,
                0.000546558,
                0.0008160991,
                0.015217237,
                -0.019124545,
                0.005153261,
                0.012054179,
                -0.030115511,
                0.003415572,
                -0.008718347,
                0.006166636,
                -0.002462003,
                -0.011808311,
                -0.027178384,
                0.00030255888,
                0.009010731,
                -0.0055054505,
                -0.016772186,
                0.014007833,
                0.026474006,
                -0.015655814,
                -0.027590381,
                0.02021434,
                0.0014677325,
                0.0067912736,
                0.0044887527,
                0.0062496997,
                -0.009661949,
                -0.012798428,
                0.0033856693,
                0.0028706754,
                0.00022053365,
                -0.019842215,
                0.0033574277,
                0.014180604,
                0.028254889,
                -0.019390348,
                -0.001903816,
                0.010678647,
                0.014911564,
                -0.01340313,
                -0.0020716044,
                0.012486109,
                -0.011356445,
                0.0044123344,
                -0.04223614,
                0.01261901,
                0.013190487,
                -0.005970606,
                0.028786495,
                0.0016313676,
                0.027989086,
                -0.036042925,
                -0.0028158536,
                0.00934963,
                0.042874068,
                0.028148567,
                0.0006487261,
                0.026434137,
                0.013595837,
                0.030593958,
                0.014180604,
                -0.028600432,
                0.0154298805,
                0.018779002,
                0.012373143,
                -0.004495398,
                -0.0051698736,
                0.007621909,
                0.016134258,
                0.010273296,
                0.02998261,
                -0.014140734,
                -0.014964725,
                -0.05276195,
                0.00887783,
                0.018287266,
                0.028839655,
                -0.00894428,
                -0.002468648,
                0.005000424,
                -0.0020134598,
                -0.0050336495,
                -0.0014079268,
                0.015589362,
                0.01865939,
                -0.01059226,
                -0.0015059417,
                -0.6668473,
                -0.01383506,
                -0.031497687,
                -0.015682394,
                0.03734536,
                0.028281469,
                0.024786156,
                0.030966083,
                -0.0037976643,
                -0.02277934,
                0.0059739286,
                0.015948197,
                0.027111934,
                0.006219797,
                -0.00079034944,
                -0.0099011725,
                0.005485515,
                -0.0070105614,
                -0.017410114,
                -0.009808141,
                0.0029520777,
                0.0131506175,
                -0.01624058,
                0.019111255,
                0.04276775,
                -0.012479464,
                0.0011504298,
                -0.0068444344,
                -0.040189456,
                -0.007150108,
                0.003704633,
                0.02425455,
                -0.004046855,
                -0.0014810227,
                0.054542832,
                -0.025889238,
                -0.011542507,
                0.030620538,
                0.012233595,
                0.040162876,
                0.01299778,
                -0.010745098,
                0.0064822775,
                -0.0010706888,
                0.0047944267,
                -0.0047445884,
                0.022420507,
                -0.02028079,
                -0.006638437,
                -0.0035285384,
                0.035139192,
                -0.0259424,
                0.000057158086,
                0.036920074,
                0.006266312,
                -0.014884983,
                0.028627014,
                -0.01056568,
                0.0006416657,
                0.0075022974,
                -0.0026248074,
                -0.009416081,
                -0.015655814,
                -0.032428,
                -0.003365734,
                0.0037677614,
                -0.0056616096,
                -0.00039392876,
                0.0064590196,
                0.030274993,
                0.007641844,
                0.013888221,
                -0.012751912,
                -0.014978015,
                -0.00690424,
                0.025463954,
                0.024826026,
                -0.030407894,
                -0.0002051669,
                0.004727976,
                0.010060654,
                -0.0013057586,
                -0.014791952,
                -0.018220814,
                0.01695825,
                -0.013994543,
                -0.029078878,
                -0.011270059,
                0.02276605,
                -0.0024769544,
                -0.007356106,
                0.03901992,
                0.012539269,
                -0.0036880204,
                -0.018061332,
                0.019975116,
                -0.009396146,
                0.008691767,
                0.037132718,
                -0.06953414,
                -0.0032527675,
                -0.010073944,
                -0.00046764765,
                0.0137951905,
                0.0099011725,
                -0.024095068,
                0.004605042,
                -0.018074622,
                0.028175147,
                -0.0067613707,
                0.0086053815,
                -0.0032893156,
                -0.024812736,
                -0.005422387,
                0.0014386602,
                -0.028573852,
                -0.009821431,
                0.008266482,
                0.011276704,
                -0.021450324,
                0.0108779995,
                -0.00689095,
                0.014074284,
                -0.0065022125,
                0.014472988,
                0.0016047873,
                0.023151465,
                -0.024600094,
                0.013622418,
                -0.008598736,
                -0.004917361,
                0.026992323,
                0.04470811,
                -0.008372803,
                0.0038973405,
                0.0076019736,
                0.014871693,
                -0.018566359,
                -0.0013431372,
                -0.014326797,
                -0.04959889,
                0.0043591736,
                -0.0040634675,
                0.0019054773,
                -0.0055552884,
                -0.044415727,
                -0.017556306,
                -0.01865939,
                -0.019377058,
                -0.019762473,
                -0.015921617,
                -0.000106165564,
                -0.013456291,
                0.027111934,
                -0.00014806073,
                -0.011482702,
                -0.0029271585,
                -0.031471107,
                -0.00847248,
                -0.01701141,
                0.02194206,
                -0.0042229495,
                -0.007522233,
                0.015615943,
                -0.008851249,
                0.0050103916,
                0.0016089404,
                0.036388468,
                -0.013675579,
                -0.03466075,
                0.025277892,
                0.0039704363,
                0.01137638,
                0.021397164,
                -0.01620071,
                0.0055120955,
                -0.023975456,
                -0.002045024,
                -0.014007833,
                0.009356275,
                -0.0042229495,
                0.0008223289,
                -0.01546975,
                0.0058011566,
                0.02841437,
                0.020639624,
                0.04119951,
                0.014047703,
                0.008618671,
                0.005036972,
                0.01784869,
                0.020838978,
                -0.007150108,
                0.020652914,
                -0.0034355074,
                0.013336679,
                0.032720383,
                -0.012034243,
                -0.009529048,
                0.043272775,
                -0.0008821346,
                0.031178724,
                0.01172857,
                -0.00093695655,
                0.03750484,
                -0.018566359,
                0.00936292,
                -0.011987728,
                0.026606908,
                0.023802683,
                0.019815635,
                -0.026128462,
                -0.02677968,
                -0.021583226,
                0.021304132,
                0.009934397,
                -0.007847842,
                0.016865218,
                0.005226357,
                -0.015177367,
                -0.0077681006,
                0.010871354,
                0.009828076,
                -0.014725502,
                -0.020865558,
                0.018499909,
                -0.011821601,
                0.0118548265,
                0.0013622418,
                -0.005711448,
                -0.0054954826,
                -0.0054689026,
                0.0026264687,
                -0.009854657,
                -0.0002838696,
                -0.0042960453,
                -0.008784798,
                -0.041598212,
                0.01950996,
                0.0001323825,
                0.0061267656,
                0.006183249,
                0.013044296,
                -0.018313846,
                0.008804734,
                0.016599415,
                0.03747826,
                0.0018058011,
                0.007947518,
                -0.005977251,
                -0.016333612,
                0.02021434,
                -0.011535862,
                0.035431575,
                0.0015524572,
                0.00466817,
                -0.0007666763,
                0.030673698,
                -0.000036080717,
                0.008359513,
                -0.009628723,
                0.0022128122,
                0.005797834,
                0.00406679,
                0.014884983,
                -0.0111969635,
                0.001682867,
                -0.015961487,
                0.016466513,
                -0.001596481,
                0.0049971016,
                -0.0027660152,
                0.006635114,
                0.013569257,
                0.012918039,
                -0.01706457,
                0.0023855844,
                0.024600094,
                0.00549216,
                0.009136988,
                -0.025782919,
                -0.008744928,
                0.029105458,
                0.0028025634,
                -0.014100864,
                -0.00068444345,
                -0.013330034,
                0.015296979,
                -0.01706457,
                0.030354735,
                0.015775425,
                0.0112036085,
                0.004914038,
                -0.017184181,
                -0.03091292,
                -0.031391367,
                -0.00020745116,
                -0.0359366,
                -0.0032826704,
                -0.011456122,
                0.016134258,
                -0.0069507556,
                -0.009196794,
                0.004644912,
                0.017463276,
                -0.01221366,
                -0.008479125,
                -0.011881406,
                -0.006591921,
                -0.0018722519,
                0.0052994527,
                -0.01137638,
                -0.004797749,
                -0.009230019,
                0.005392484,
                -0.009442661,
                0.0055552884,
                -0.020426981,
                0.022061672,
                -0.016041229,
                -0.01457931,
                -0.019018225,
                -0.022486957,
                -0.008366158,
                0.10350379,
                0.013256938,
                0.011562442,
                0.018446747,
                0.020932008,
                -0.007376041,
                -0.050103918,
                -0.01695825,
                -0.00847248,
                -0.012778493,
                -0.007289655,
                -0.00264142,
                0.008592091,
                0.0056416746,
                0.018832162,
                0.00025126716,
                -0.017131021,
                -0.013874931,
                0.02018776,
                -0.0070238514,
                -0.013715449,
                0.014433118,
                0.005289485,
                0.030540796,
                -0.0025085185,
                0.008140225,
                0.022274315,
                -0.003980404,
                0.017210763,
                -0.015310269,
                0.018911904,
                -0.0019802344,
                -0.0061633135,
                -0.0042594974,
                0.015974777,
                -0.0000052206724,
                -0.013609128,
                -0.021596516,
                0.02426784,
                -0.016320322,
                0.026580328,
                0.0017925109,
                0.004545236,
                -0.017330375,
                -0.007475717,
                0.008293062,
                0.0024835994,
                0.039365467,
                -0.033624116,
                0.00012552975,
                0.03808961,
                0.016812058,
                -0.0047911042,
                -0.008146871,
                -0.02029408,
                0.021623097,
                0.011482702,
                -0.014419828,
                0.016639285,
                -0.017529726,
                -0.014499568,
                -0.03808961,
                -0.00609354,
                -0.007960808,
                -0.017263923,
                -0.020559883,
                0.01259243,
                0.01383506,
                -0.005017037,
                -0.007974098,
                0.028015666,
                -0.0020599754,
                -0.03673401,
                -0.0029753353,
                0.024068486,
                -0.0028341275,
                -0.010838129,
                -0.00889112,
                -0.0064125042,
                0.007057077,
                -0.00971511,
                -0.011343155,
                -0.00030588143,
                -0.014154024,
                -0.0021015073,
                -0.020108018,
                0.03253432,
                -0.029291522,
                0.009588853,
                0.040189456,
                0.026819551,
                0.010871354,
                0.0021480226,
                0.003624892,
                0.0052529373,
                0.0016704075,
                0.014858403,
                0.018180944,
                -0.00813358,
                0.0018656068,
                0.044043604,
                -0.038381994,
                -0.00020194819,
                -0.014340087,
                0.024573512,
                -0.010286586,
                -0.012439594,
                0.0017376889,
                -0.01013375,
                0.009688529,
                0.020772526,
                -0.015974777,
                -0.009210084,
                0.0007475717,
                -0.013901511,
                0.03412914,
                -0.0066616945,
                0.0033026058,
                -0.018779002,
                -0.0037146008,
                0.020373821,
                -0.030381314,
                0.03516577,
                0.035272095,
                -0.0012567511,
                -0.015031176,
                0.019682733,
                -0.024520352,
                0.002707871,
                0.00426282,
                -0.003850825,
                -0.015071046,
                -0.005834382,
                -0.017410114,
                -0.015031176,
                -0.021330712,
                -0.011163738,
                0.008598736,
                -0.016360192,
                -0.005718093,
                -0.012805073,
                0.000502119,
                -0.01624058,
                0.03173691,
                -0.036122665,
                -0.018127784,
                -0.0016205694,
                -0.008419319,
                -0.00019260355,
                0.02923836,
                -0.003930566,
                0.023377398,
                -0.032986186,
                0.014114154,
                -0.0020134598,
                -0.027377738,
                -0.019895375,
                -0.0018705906,
                0.00004106453,
                0.008771508,
                -0.0031281724,
                0.010645421,
                0.021237683,
                0.019935247,
                -0.010260006,
                -0.023018563,
                0.02355017,
                0.0074159116,
                -0.022473667,
                0.023802683,
                0.023935584,
                0.0016936653,
                0.024068486,
                -0.023430558,
                -0.012260176,
                0.022274315,
                -0.0014876677,
                0.003987049,
                -0.01941693,
                -0.017582886,
                0.0077149402,
                -0.004631622,
                0.0018340426,
                0.008851249,
                -0.005957316,
                -0.03245458,
                0.03886044,
                -0.007216559,
                -0.009495822,
                -0.006000509,
                0.019988406,
                -0.02348372,
                0.00812029,
                -0.012472819,
                0.016652575,
                -0.00031917158,
                0.015124206,
                0.0033142345,
                0.003299283,
                0.0055918363,
                0.023403978,
                0.001947009,
                0.0019868794,
                -0.022340765,
                0.0024752931,
                0.023563461,
                0.009721755,
                0.0038275672,
                0.00042611588,
                -0.010632131,
                0.0005170704,
                -0.019018225,
                0.006764693,
                -0.01694496,
                0.0020001696,
                0.015961487,
                -0.008897765,
                0.03591002,
                -0.013555967,
                0.0003096193,
                0.0062530222,
                -0.024081778,
                0.015296979,
                -0.005429032,
                0.013077521,
                0.020134598,
                -0.018087912,
                -0.015562782,
                0.030540796,
                -0.020161178,
                -0.00668163,
                -0.0064357617,
                0.020121308,
                0.0059971865,
                -0.012007663,
                -0.005811124,
                0.024321,
                0.007077012,
                -0.007681715,
                0.02105162,
                0.006113475,
                0.022367345,
                -0.02992945,
                0.0018473328,
                0.0057214154,
                0.022500247,
                0.012366498,
                -0.0064889225,
                -0.010745098,
                0.0038774053,
                -0.026168333,
                0.0018107848,
                -0.010725162,
                0.009522403,
                0.015177367,
                0.01539001,
                0.015310269,
                -0.011097287,
                -0.019230867,
                0.01295791,
                -0.0032627352,
                0.041066606,
                -0.025543695,
                -0.0026115172,
                0.015164077,
                0.009794851,
                0.008259837,
                -0.0065819537,
                -0.0007542168,
                0.023124885,
                -0.036149245,
                -0.030567376,
                0.0028324663,
                0.009010731,
                0.017237343,
                -0.02430771,
                0.008538931,
                -0.0170247,
                -0.01137638,
                -0.016320322,
                0.007376041,
                0.0125060445,
                -0.010386263,
                0.0009178519,
                -0.0008160991,
                -0.024905767,
                0.0015017885,
                0.0021380552,
                -0.019297319,
                -0.014154024,
                -0.015855165,
                -0.015576072,
                0.0023756167,
                0.019071385,
                0.007449137,
                0.0029703516,
                0.0014428134,
                0.05425045,
                -0.04061474,
                0.009980913,
                0.021742707,
                0.036840335,
                -0.0059672836,
                0.007794681,
                0.0017493179,
                -0.0058808974,
                0.007960808,
                -0.015230528,
                -0.020493433,
                -0.029371262,
                -0.004146531,
                -0.002229425,
                0.009974268,
                -0.007070367,
                0.01052581,
                -0.004963876,
                -0.010891289,
                -0.01703799,
                -0.036468208,
                -0.009030666,
                -0.012771848,
                0.016891798,
                0.011356445,
                0.0030417861,
                0.03577712,
                0.034315202,
                -0.018114494,
                -0.0018406877,
                -0.021676257,
                -0.00024254549,
                0.009821431,
                0.014406538,
                -0.011070707,
                -0.030859761,
                -0.0134762265,
                -0.018818872,
                -0.017449984,
                0.013482871,
                -0.03487339,
                -0.007681715,
                0.011442831,
                0.05042288,
                -0.0058742524,
                -0.008638606,
                -0.014379958,
                -0.01795501,
                -0.004887458,
                -0.008100355,
                -0.026168333,
                -0.019988406,
                -0.011356445,
                0.007083657,
                -0.010990965,
                -0.0017709143,
                0.00073220494,
                -0.011701989,
                -0.02519815,
                0.01463247,
                -0.018300556,
                -0.010379618,
                0.034501266,
                0.01013375,
                -0.0068975952,
                0.016453223,
                0.015908327,
                0.021716127,
                0.007469072,
                -0.0013597499,
                0.02114465,
                -0.016346902,
                -0.003980404,
                -0.016293742,
                -0.028759915,
                0.002648065,
                0.014818532,
                0.011117222,
                -0.0040501775,
                0.011502637,
                -0.02421468,
                -0.0050336495,
                0.039232563,
                -0.021623097,
                0.011090642,
                0.02677968,
                -0.012193725,
                -0.030248413,
                -0.0041099833,
                0.01374203,
                0.00005809255,
                -0.025344342,
                0.012094049,
                0.0089177,
                -0.015775425,
                -0.00014951434,
                -0.009176858,
                -0.018925194,
                -0.0073228804,
                -0.015642524,
                0.016107678,
                0.0062065064,
                -0.018420167,
                0.024879187,
                0.0005075181,
                -0.022194574,
                0.0031148822,
                0.011256768,
                -0.045638423,
                -0.018965064,
                -0.0051864865,
                -0.022579988,
                -0.01133651,
                -0.019137835,
                0.011708634,
                -0.02105162,
                0.008299707,
                -0.010452714,
                -0.010020783,
                0.0049738437,
                0.0113498,
                0.015310269,
                -0.015270398,
                0.012034243,
                -0.014154024,
                0.0012010986,
                -0.011210253,
                -0.003379024,
                0.011648828,
                -0.034501266,
                0.0129712,
                0.007475717,
                -0.0043159807,
                -0.025889238,
                0.016214,
                0.009622078,
                -0.00066616945,
                0.020108018,
                0.21721444,
                0.0049738437,
                -0.01953654,
                0.037797228,
                0.021237683,
                -0.0016164161,
                0.013715449,
                0.015961487,
                -0.024839316,
                0.009123698,
                -0.0029088845,
                0.0043824315,
                0.006871015,
                0.0041232733,
                -0.01539001,
                -0.032428,
                -0.039418627,
                -0.01536343,
                -0.02672652,
                -0.024826026,
                0.012439594,
                -0.004894103,
                -0.012931329,
                -0.004036887,
                0.015655814,
                -0.01620071,
                -0.019350478,
                -0.012964555,
                0.029105458,
                0.048695162,
                0.00014691861,
                0.0018523165,
                0.016918378,
                -0.013237003,
                0.015124206,
                0.0022958757,
                0.014247056,
                0.00089127163,
                0.016453223,
                0.019642862,
                -0.03814277,
                -0.003704633,
                -0.010100525,
                -0.0028889494,
                0.00507352,
                -0.0137819,
                -0.015270398,
                0.013037651,
                0.014831823,
                0.01298449,
                -0.011555797,
                -0.022048382,
                0.00448543,
                0.014379958,
                0.005369226,
                -0.021795869,
                0.00813358,
                -0.016918378,
                -0.019735893,
                0.026394265,
                -0.009887882,
                0.020054856,
                -0.012977845,
                0.03088634,
                -0.01618742,
                0.0029504164,
                -0.0040335646,
                0.009615433,
                0.02761696,
                -0.017317083,
                0.016506383,
                -0.0071633984,
                -0.017489856,
                -0.0018639455,
                -0.014034413,
                -0.02921178,
                0.026859421,
                -0.0016745606,
                0.025596855,
                0.0194568,
                -0.025862658,
                -0.0005627554,
                0.000064426145,
                -0.020387111,
                0.0029902868,
                -0.029610485,
                0.031524267,
                0.016479803,
                -0.0134762265,
                -0.007967453,
                -0.0053360006,
                -0.010652066,
                -0.01099761,
                0.016679155,
                0.014818532,
                0.022128122,
                0.0018224138,
                0.015788715,
                -0.014645761,
                0.021423744,
                0.00894428,
                -0.011495992,
                0.007136818,
                0.013635708,
                -0.033836756,
                -0.001532522,
                -0.0033092508,
                0.000104244726,
                -0.004428947,
                -0.0010424472,
                0.0016081098,
                -0.00549216,
                0.000027203305,
                0.006023767,
                -0.012207015,
                -0.010977675,
                0.011436186,
                -0.030540796,
                0.025676597,
                -0.03003577,
                0.014924854,
                -0.013901511,
                0.0005046109,
                0.01059226,
                0.015802005,
                -0.018021462,
                0.0025018733,
                -0.0012650575,
                0.0073228804,
                -0.036069505,
                0.010432779,
                0.010306522,
                0.022952113,
                0.006768016,
                -0.0030932857,
                0.02442732,
                0.034208883,
                -0.02514499,
                0.004492075,
                -0.011907986,
                -0.00055984815,
                0.00046349448,
                -0.0027111934,
                0.00064000447,
                0.010532455,
                -0.009057247,
                0.011117222,
                -0.008246547,
                0.000202675,
                -0.014791952,
                -0.018619519,
                -0.0118481815,
                -0.015788715,
                -0.029557325,
                0.03726562,
                -0.024055196,
                -0.035378415,
                -0.028095406,
                -0.007449137,
                0.0030750118,
                -0.006970691,
                -0.0021197812,
                0.036016345,
                -0.015071046,
                -0.0179683,
                -0.009269889,
                -0.17149627,
                0.014552729,
                0.0015657474,
                -0.032162197,
                0.01009388,
                0.014911564,
                0.029876288,
                -0.0032461225,
                -0.020147888,
                0.016798766,
                -0.0108779995,
                -0.009748335,
                -0.037584584,
                -0.0053160656,
                0.018194234,
                -0.002438745,
                0.013137327,
                -0.017410114,
                -0.0013813464,
                -0.0021579904,
                0.017994883,
                -0.024068486,
                0.009695174,
                -0.0032942994,
                0.019762473,
                -0.012698752,
                -0.014340087,
                0.0113498,
                0.010691937,
                -0.0002616501,
                -0.00073220494,
                0.013090812,
                0.017569596,
                -0.009608788,
                0.003538506,
                0.023683071,
                -0.0007334509,
                -0.015775425,
                0.0009818109,
                0.023191337,
                0.0014594261,
                0.016865218,
                0.0212111,
                0.002530115,
                -0.016758896,
                0.019549832,
                0.008379448,
                -0.0105523905,
                0.03412914,
                0.002807547,
                -0.0022277636,
                -0.03489997,
                0.014459698,
                -0.002898917,
                0.015270398,
                0.017476566,
                0.0013946366,
                -0.005110068,
                -0.003156414,
                -0.0077548106,
                -0.034394942,
                -0.0051565836,
                0.00017817126,
                -0.009562273,
                0.008239902,
                -0.024812736,
                0.003917276,
                -0.014233765,
                -0.00029092998,
                0.017170891,
                0.006153346,
                -0.011595668,
                -0.009841367,
                -0.032029293,
                0.010672001,
                0.020559883,
                -0.005289485,
                0.002033395,
                -0.0031098984,
                -0.014818532,
                -0.02751064,
                0.038009867,
                -0.028361209,
                0.0014536117,
                0.020453563,
                -0.000007456249,
                -0.0032212036,
                -0.0018207524,
                -0.008020614,
                -0.005289485,
                0.022261024,
                -0.01787527,
                0.0061998614,
                -0.026354395,
                0.035404995,
                0.012100694,
                0.017144311,
                0.01865939,
                -0.014539439,
                -0.01172857,
                0.0025400827,
                -0.00068942725,
                -0.006558696,
                0.006073605,
                0.027032193,
                0.020041566,
                -0.007469072,
                0.027723283,
                0.00852564,
                0.009123698,
                -0.016373482,
                0.012293401,
                0.016546253,
                0.0024852606,
                -0.02426784,
                0.0040402096,
                -0.0003912292,
                -0.04271459,
                0.013177197,
                -0.0035783767,
                0.016413352,
                -0.011130512,
                -0.023350818,
                -0.0035717315,
                -0.00897086,
                -0.022938823,
                -0.058742523,
                0.010213491,
                0.012705397,
                0.008226612,
                -0.0034421524,
                0.016054519,
                -0.013090812,
                0.013031006,
                0.007854487,
                0.015031176,
                0.020161178,
                -0.0076019736,
                0.0039438563,
                0.0004074266,
                0.012785138,
                0.019190997,
                -0.024002036,
                -0.02194206,
                -0.01869926,
                -0.0009377872,
                -0.009655304,
                -0.009675239,
                0.007050432,
                -0.0051499386,
                -0.00021949536,
                -0.0011653813,
                -0.023071725,
                0.025357632,
                0.026992323,
                0.019097965,
                0.027803022,
                -0.00669492,
                0.012552559,
                0.0022643118,
                -0.004428947,
                -0.0012118968,
                -0.0015549492,
                0.00889112,
                0.010990965,
                -0.034155723,
                0.0018074623,
                -0.009017376,
                0.002985303,
                -0.049944434,
                -0.021689547,
                0.009356275,
                -0.0055619334,
                0.0030351412,
                0.007734875,
                -0.0038906955,
                0.00809371,
                -0.012964555,
                -0.008020614,
                -0.005651642,
                0.022978693,
                0.01536343,
                0.017556306,
                0.014765372,
                -0.023164755,
                0.01790185,
                -0.01772908,
                0.01947009,
                0.0037478262,
                0.0024354225,
                0.036787175,
                0.0028108696,
                -0.0072763646,
                -0.005183164,
                0.022593278,
                0.010851419,
                0.0024121648,
                0.025410794,
                -0.029610485,
                0.03237484,
                -0.016293742,
                0.014831823,
                -0.034288622,
                -0.019230867,
                -0.0061566685,
                -0.020121308,
                -0.008040549,
                -0.008425964,
                -0.0041498537,
                -0.00730959,
                -0.0018822195,
                0.035006292,
                -0.0027012257,
                -0.017157601,
                -0.00063419,
                -0.017609468,
                0.020852268,
                0.018180944,
                -0.0011138818,
                -0.018539779,
                0.009535693,
                -0.016466513,
                -0.010612195,
                -0.001329847,
                -0.022619858,
                0.009489177,
                -0.041598212,
                -0.03155085,
                -0.087023996,
                0.025570275,
                0.0064856,
                -0.004797749,
                0.014286926,
                -0.020613045,
                0.0028208373,
                -0.001118035,
                -0.01778224,
                0.003784374,
                -0.020493433,
                0.0074956524,
                0.003144785,
                -0.0029703516,
                -0.009263244,
                -0.0047246534,
                0.0009593837,
                -0.004126596,
                0.0005224696,
                0.0059805736,
                -0.01775566,
                -0.008645251,
                -0.009389501,
                0.015057756,
                -0.018606229,
                -0.00045103492,
                -0.02438745,
                0.0210782,
                0.00018865803,
                0.011077352,
                0.013675579,
                -0.019828925,
                -0.013256938,
                0.050927907,
                -0.03179007,
                -0.0041099833,
                -0.0015291994,
                0.00056026346,
                0.012552559,
                0.006449052,
                -0.002113136,
                -0.01703799,
                0.013482871,
                0.004718008,
                -0.026886001,
                0.0069507556,
                -0.008040549,
                0.012878168,
                0.022221154,
                -0.02275276,
                0.0125060445,
                0.02190219,
                0.0039338884,
                -0.019908665,
                -0.012333272,
                -0.017091151,
                0.014060994,
                0.0043591736,
                0.009183503,
                -0.011289994,
                0.009243309,
                0.01694496,
                0.00610683,
                0.0023307626,
                0.039152823,
                0.0064656646,
                -0.0059440257,
                0.0052064215,
                0.009828076,
                0.0043757865,
                -0.020347241,
                0.012153855,
                0.017556306,
                0.011343155,
                0.007050432,
                -0.008764863,
                -0.0012974523,
                -0.028095406,
                -0.027962504,
                0.019071385,
                0.010067299,
                0.014273636,
                -0.039365467,
                -0.005183164,
                0.020506723,
                0.005249615,
                -0.010113815,
                -0.002290892,
                -0.005907478,
                0.02428113,
                0.005249615,
                -0.0010457698,
                -0.010260006,
                -0.0034720553,
                0.013874931,
                0.012306692,
                0.010725162,
                -0.020440271,
                -0.005083488,
                0.04212982,
                -0.02355017,
                -0.0009676901,
                -0.007914293,
                -0.013575902,
                -0.011323219,
                0.0065753087,
                -0.017596178,
                -0.045186557,
                -0.0025284537,
                -0.0049539087,
                0.020108018,
                0.010745098,
                0.021609806,
                0.02672652,
                -0.013429711,
                -0.013648998,
                -0.015549492,
                -0.0018024785,
                -0.015708974,
                0.0010166975,
                0.004521978,
                0.015855165,
                0.03240142,
                -0.017981593,
                0.010333102,
                0.008824669,
                -0.0050768424,
                0.011708634,
                -0.0056184167,
                -0.009183503,
                -0.009136988,
                -0.016679155,
                -0.011941212,
                -0.010844774,
                -0.0154298805,
                0.011974437,
                -0.004000339,
                -0.0066982424,
                -0.02518486,
                0.05116713,
                0.022167994,
                -0.0144464085,
                0.0008613687,
                -0.018885322,
                0.042900648,
                0.0056250617,
                -0.00005027939,
                -0.020493433,
                -0.023962166,
                0.0071833334,
                -0.015150786,
                -0.0039505013,
                -0.0026331136,
                -0.02596898,
                -0.020453563,
                -0.0068510794,
                0.006379279,
                -0.01872584,
                -0.023217916,
                0.005718093,
                0.008897765,
                0.004718008,
                0.007861132,
                -0.004618332,
                -0.010014138,
                -0.004076758,
                0.017237343,
                -0.016812058,
                -0.022247734,
                0.00933634,
                0.0078079714,
                -0.0390465,
                -0.020426981,
                -0.0020367177,
                -0.007083657,
                -0.017144311,
                0.003192962,
                -0.011157093,
                -0.0020500077,
                -0.00524297,
                0.018380297,
                -0.039445207,
                -0.019177707,
                0.009243309,
                -0.014725502,
                -0.01775566,
                0.003850825,
                -0.023576751
            ],
            "DescriptionVector":[
                0.0026614275,
                0.009162238,
                0.002262785,
                -0.0138675235,
                0.0026107803,
                0.024964156,
                -0.029329617,
                0.0023591782,
                0.023630992,
                0.0013748257,
                -0.006077661,
                0.01120773,
                -0.0021369841,
                -0.0060547884,
                -0.008404164,
                -0.018180702,
                0.026950832,
                -0.029956987,
                -0.0010709427,
                -0.0138675235,
                -0.027787328,
                0.008168899,
                0.018311406,
                -0.0076983706,
                -0.008273461,
                0.0007637921,
                0.017135084,
                -0.004796778,
                0.005234631,
                0.0007592992,
                0.0018281997,
                -0.017344207,
                -0.03058436,
                -0.006404417,
                -0.023382658,
                -0.007502317,
                -0.0169521,
                -0.011841636,
                -0.010233997,
                0.005522176,
                0.002159857,
                -0.032178927,
                -0.007534993,
                -0.0063946145,
                0.00045459924,
                0.02241546,
                0.015030775,
                -0.0023559106,
                -0.032048225,
                0.013945945,
                -0.0032250814,
                0.010854834,
                -0.016795257,
                -0.011965804,
                0.008574077,
                -0.0027594543,
                -0.0014393601,
                0.00011650893,
                0.00019584937,
                -0.019017197,
                0.012246814,
                -0.04247828,
                -0.017670963,
                -0.0031123506,
                -0.0040190984,
                0.0014311911,
                -0.012384051,
                0.0017269053,
                0.0014777539,
                -0.0019915777,
                0.03602158,
                0.020847032,
                0.0059110154,
                -0.01640315,
                0.009090351,
                -0.020049747,
                -0.008704779,
                0.014011296,
                0.013919804,
                0.012475543,
                0.009155703,
                -0.032675598,
                -0.0013217279,
                0.002063464,
                0.019265532,
                0.007423896,
                -0.000058356574,
                -0.0044471486,
                -0.019487726,
                -0.01581499,
                0.010090224,
                0.024794243,
                0.0019164238,
                0.013540767,
                -0.036910355,
                0.0028525796,
                0.011612907,
                0.002679399,
                0.017082803,
                0.0012114478,
                0.010443121,
                0.025905214,
                -0.021618174,
                -0.0045909216,
                -0.046843737,
                -0.014494895,
                -0.011808961,
                -0.020951593,
                0.010338559,
                0.022219406,
                -0.012795764,
                0.010456191,
                0.010116365,
                -0.011678259,
                0.0016762582,
                0.0047837077,
                0.021722738,
                -0.0015463727,
                0.004587654,
                -0.041380376,
                -0.0071951663,
                0.010874439,
                -0.0022578838,
                -0.007861748,
                0.01697824,
                0.017592542,
                -0.011906988,
                -0.03879247,
                0.0051692794,
                -0.006031915,
                -0.01286765,
                0.0136453295,
                -0.014298841,
                -0.008240785,
                -0.0045125,
                -0.013932874,
                0.008273461,
                -0.0041890116,
                -0.009756933,
                0.0015316686,
                0.016703766,
                0.013501557,
                -0.019330883,
                -0.0024359657,
                0.022140985,
                0.020977734,
                -0.009488993,
                0.0012122646,
                0.009136097,
                -0.0038165099,
                0.018115351,
                -0.032649457,
                0.0045451755,
                0.014625598,
                0.0011518148,
                0.035420347,
                0.0038981987,
                0.020520275,
                -0.014625598,
                -0.007227842,
                0.0018641428,
                0.027813468,
                0.02411459,
                0.00086426956,
                0.022689935,
                0.014952353,
                0.030479798,
                0.0047248914,
                -0.03458385,
                0.009796144,
                0.029852426,
                0.01582806,
                -0.013410065,
                -0.005522176,
                0.0042249546,
                0.01968378,
                0.01061957,
                0.023147393,
                -0.023082042,
                -0.019905973,
                -0.052725345,
                0.0012629118,
                0.0077963974,
                0.037041057,
                -0.021422122,
                0.005587527,
                0.007325869,
                0.004090985,
                -0.011142379,
                -0.007757187,
                -0.0048555937,
                0.0149000725,
                -0.01371068,
                -0.00614628,
                -0.660413,
                -0.0035616402,
                -0.028101014,
                -0.021487473,
                0.03340753,
                0.015501304,
                0.03123787,
                0.03288472,
                -0.0076787653,
                -0.035315786,
                0.006002507,
                0.010090224,
                0.021578964,
                0.006113604,
                -0.0077441167,
                -0.008162364,
                0.011776285,
                -0.009423642,
                -0.017710174,
                -0.013207477,
                0.0112011945,
                -0.005809721,
                -0.024741963,
                0.030375235,
                0.039733525,
                0.0017922565,
                -0.0012735314,
                -0.011377643,
                -0.05635887,
                -0.0067115678,
                0.005420882,
                0.013266292,
                0.0019948452,
                0.0065743304,
                0.05578378,
                -0.020637907,
                -0.01744877,
                0.032231208,
                0.009247194,
                0.045144606,
                0.026009776,
                -0.010854834,
                0.007894425,
                -0.007861748,
                0.013514627,
                -0.012501684,
                0.033773497,
                -0.01969685,
                -0.015030775,
                -0.00020095492,
                0.03889703,
                -0.043105647,
                -0.0026450895,
                0.048856553,
                -0.003591048,
                -0.002775792,
                0.036962636,
                -0.024768103,
                -0.015004634,
                -0.0014050507,
                0.002034056,
                -0.009488993,
                -0.014102788,
                -0.04386372,
                -0.0026892016,
                0.027761187,
                0.0002397572,
                0.008057802,
                -0.0039145364,
                0.025591528,
                -0.0022513487,
                0.02028501,
                -0.018455178,
                -0.024245294,
                -0.0026009777,
                0.01747491,
                0.028336277,
                -0.015148407,
                -0.0047379616,
                0.013553838,
                0.015527444,
                0.0012718976,
                -0.022703005,
                -0.028231716,
                0.004868664,
                -0.023134323,
                -0.02194493,
                -0.002583006,
                0.025500037,
                0.0059110154,
                -0.00027774257,
                0.041145112,
                0.01586727,
                -0.039550543,
                -0.022885988,
                0.02019352,
                -0.006538387,
                0.018141491,
                0.034270167,
                -0.05204569,
                -0.003747891,
                -0.010697991,
                -0.015357531,
                0.02021966,
                0.016311659,
                -0.023408799,
                -0.021382911,
                -0.008750525,
                0.03275402,
                -0.012684667,
                -0.0009181843,
                -0.0058816075,
                -0.024715822,
                -0.0025911748,
                -0.007397755,
                -0.029303476,
                -0.00074418675,
                0.0040387036,
                0.013207477,
                -0.005760708,
                0.006169153,
                -0.0057705105,
                0.0037609614,
                -0.011985409,
                0.01205076,
                0.0094301775,
                0.028310137,
                -0.019749131,
                -0.0007286658,
                0.0015921185,
                0.0064174873,
                0.028728385,
                0.044647936,
                0.0027921298,
                0.005420882,
                0.021474402,
                0.025931355,
                -0.027029254,
                0.0011902086,
                -0.015501304,
                -0.052228674,
                0.0032806299,
                0.0020503937,
                -0.0015112464,
                0.0017056662,
                -0.047654092,
                -0.011188124,
                -0.016795257,
                -0.034191746,
                -0.030505938,
                -0.027970312,
                -0.005927353,
                -0.02025887,
                0.036492106,
                0.006446895,
                0.0060678585,
                -0.004394868,
                -0.0317084,
                0.0019670709,
                -0.01578885,
                0.036884215,
                -0.0017759188,
                -0.012645456,
                0.01696517,
                -0.004326249,
                0.010678385,
                0.004319714,
                0.035106663,
                -0.0014271067,
                -0.046896018,
                0.017867018,
                0.025578458,
                0.0127565535,
                0.033564374,
                -0.010894044,
                -0.0018134956,
                -0.020703258,
                -0.004587654,
                -0.01859895,
                0.009155703,
                -0.00010997381,
                -0.0029440713,
                -0.016285518,
                -0.0039733527,
                0.015239899,
                0.011599837,
                0.033564374,
                0.0076853004,
                -0.0044536837,
                0.021199927,
                0.034792975,
                0.019879833,
                -0.019958256,
                0.013919804,
                -0.00765916,
                0.021160716,
                0.014704019,
                -0.012436332,
                -0.016220167,
                0.024467487,
                -0.0016460333,
                0.0404916,
                0.018402897,
                0.0055777244,
                0.026950832,
                -0.029643303,
                0.0075153876,
                -0.01857281,
                0.04203389,
                0.0045680483,
                0.01578885,
                -0.024441347,
                -0.021801159,
                -0.03387806,
                0.010351629,
                0.028728385,
                0.007489247,
                0.016076393,
                0.0077375816,
                -0.012455937,
                0.016141744,
                0.013540767,
                0.01746184,
                -0.012187998,
                -0.034087185,
                0.002101041,
                -0.010273208,
                0.005143139,
                -0.017723244,
                -0.0043752627,
                0.003943945,
                -0.013155196,
                0.0024016565,
                -0.007816003,
                -0.011325362,
                0.006744243,
                -0.0069337618,
                -0.05468588,
                0.010345094,
                0.010423516,
                0.0051464066,
                0.016076393,
                0.0083192075,
                -0.023827046,
                0.0010856467,
                0.021696597,
                0.027081534,
                0.0017448768,
                0.021461332,
                -0.01288072,
                -0.028623823,
                0.014494895,
                0.0017203703,
                0.028571542,
                -0.0052150255,
                0.018455178,
                0.0037838342,
                0.021565894,
                0.0127565535,
                0.0049307477,
                -0.0023689808,
                -0.004277236,
                0.01861202,
                0.0003406431,
                0.014063577,
                -0.012005014,
                0.0013642062,
                -0.020128168,
                0.023291165,
                -0.0041301954,
                0.025042579,
                -0.0018886495,
                -0.0035224294,
                0.0029898172,
                0.008678638,
                -0.023199674,
                -0.0004423459,
                0.024911875,
                0.00032267152,
                0.009926846,
                -0.02078168,
                0.00088550866,
                0.020481065,
                0.009083816,
                -0.002337939,
                0.00090593094,
                -0.034662273,
                0.022885988,
                -0.008842017,
                0.046791457,
                0.010992071,
                0.010449656,
                -0.0131225195,
                -0.018232983,
                -0.024702752,
                -0.020912383,
                0.009619696,
                -0.021121506,
                0.01744877,
                -0.022193266,
                -0.00314666,
                0.0019262264,
                -0.028048733,
                -0.0015978367,
                0.014481825,
                -0.002797031,
                -0.0015765976,
                -0.0020356898,
                0.0005893861,
                -0.003324742,
                -0.004254363,
                -0.027159957,
                0.018781934,
                -0.019017197,
                0.010364699,
                -0.016638415,
                -0.008201575,
                0.00055180915,
                0.020585626,
                -0.017631752,
                -0.0069141565,
                -0.026179688,
                -0.017631752,
                -0.011887383,
                0.10863983,
                0.003957015,
                0.0039341417,
                0.015161477,
                0.015971832,
                -0.0074500362,
                -0.041772485,
                -0.009822285,
                -0.008247321,
                0.0028983254,
                -0.012697737,
                -0.02021966,
                0.016886748,
                0.02021966,
                0.020585626,
                -0.0011191393,
                -0.021134576,
                -0.01750105,
                0.017644823,
                0.008417234,
                -0.008528331,
                0.021500543,
                0.0034472756,
                0.021422122,
                -0.009377897,
                0.012887255,
                0.02570916,
                -0.018559739,
                0.005705159,
                -0.014638668,
                0.018755794,
                0.0036400615,
                -0.012501684,
                0.002197434,
                0.0000949124,
                0.004934015,
                -0.0112011945,
                -0.021030014,
                0.013972086,
                -0.014115858,
                0.026166618,
                -0.0044961623,
                0.0009688315,
                -0.016194027,
                -0.00066454,
                -0.0066592866,
                0.007613414,
                0.03759001,
                -0.041145112,
                -0.0035616402,
                0.03453157,
                0.010743736,
                -0.006273715,
                -0.021801159,
                -0.021016944,
                0.01259971,
                0.0066854274,
                -0.0086851735,
                -0.004963423,
                -0.006946832,
                -0.0069272267,
                -0.0426874,
                -0.005456825,
                -0.011521416,
                -0.02186651,
                -0.021173786,
                0.0009892537,
                0.0069272267,
                -0.018912636,
                0.0069206916,
                0.02461126,
                0.0037969043,
                -0.041641783,
                -0.0018216645,
                0.03662281,
                -0.010560753,
                -0.012861115,
                0.0027676232,
                0.0005056549,
                0.019801412,
                0.0000645343,
                -0.012109577,
                -0.0049372828,
                -0.0068030595,
                -0.009763468,
                -0.010247068,
                0.017906228,
                -0.033564374,
                0.004028901,
                0.033799637,
                0.021618174,
                0.0061168717,
                0.0021059422,
                0.005270574,
                0.0014998099,
                0.011188124,
                0.014913143,
                0.005685554,
                -0.013135591,
                0.0067115678,
                0.028649963,
                -0.033668935,
                -0.008279996,
                -0.0064436276,
                0.030375235,
                -0.021513613,
                -0.010325489,
                0.0059306207,
                -0.008940044,
                0.01588034,
                0.0096523715,
                -0.010822158,
                0.008482585,
                0.015475163,
                -0.015971832,
                0.028153295,
                0.00057427364,
                0.029852426,
                0.00018053268,
                -0.007613414,
                0.011671724,
                -0.025643809,
                0.036518246,
                0.035106663,
                0.0065416545,
                -0.016037183,
                0.010854834,
                -0.0025944423,
                -0.0022905595,
                -0.004711821,
                0.004711821,
                -0.0017465107,
                -0.011344967,
                -0.02844084,
                -0.007639555,
                -0.01867737,
                0.002932635,
                -0.0008164815,
                -0.032544896,
                0.011586767,
                -0.013436206,
                -0.0037805666,
                -0.020690188,
                0.015658146,
                -0.03555105,
                -0.023108182,
                0.0018314673,
                -0.013501557,
                0.0034374727,
                0.028545402,
                -0.013220547,
                0.018115351,
                -0.037119478,
                0.009364827,
                -0.0007552148,
                -0.01336432,
                -0.024349855,
                -0.011135844,
                0.006322728,
                0.005705159,
                -0.0021582232,
                0.015435952,
                0.019357024,
                0.017344207,
                -0.008103549,
                -0.009671977,
                0.017853945,
                0.008940044,
                -0.03445315,
                0.023879327,
                0.018873425,
                0.016756047,
                0.014246561,
                -0.033721216,
                -0.006731173,
                0.028101014,
                -0.0042935736,
                0.007182096,
                -0.006051521,
                -0.022755286,
                0.015579725,
                0.012906861,
                -0.017605612,
                0.005940424,
                -0.012658526,
                -0.03178682,
                0.029774005,
                0.0024179942,
                -0.00096066255,
                -0.004757567,
                0.020886242,
                -0.013083309,
                0.017344207,
                0.002277489,
                0.01205076,
                -0.011286152,
                0.010397375,
                -0.02190572,
                0.01582806,
                0.0086851735,
                0.024219153,
                -0.004146533,
                0.006355404,
                -0.030427516,
                0.008142759,
                0.012103042,
                0.004185744,
                -0.00037168493,
                -0.009671977,
                0.0012106309,
                0.012279489,
                -0.0046399347,
                0.004594189,
                -0.02901593,
                -0.0014973592,
                0.016769117,
                0.003754426,
                0.032414194,
                -0.02127835,
                -0.021735808,
                0.024741963,
                -0.022088705,
                0.00042805032,
                -0.015435952,
                0.019056408,
                0.0065187817,
                -0.017383417,
                -0.016847538,
                0.013057169,
                -0.012044225,
                -0.0018004255,
                0.0021043085,
                0.0023412067,
                0.0038785934,
                -0.011593302,
                -0.0034995566,
                0.023853187,
                0.017161224,
                -0.01016211,
                0.019592289,
                0.0054502897,
                0.0092079835,
                -0.023134323,
                0.010475797,
                0.013854453,
                0.016233237,
                0.0164685,
                0.003236518,
                -0.008691709,
                0.011508346,
                -0.03332911,
                0.008214645,
                -0.022010282,
                0.016559992,
                0.016285518,
                0.014821651,
                0.023225814,
                -0.013338178,
                -0.027944172,
                0.008097013,
                -0.0014360925,
                0.01640315,
                -0.03615228,
                -0.016925959,
                0.021134576,
                0.0022317432,
                0.019840622,
                -0.009155703,
                0.011998479,
                0.018128421,
                -0.03176068,
                -0.023892397,
                0.007306264,
                0.009613161,
                0.015252969,
                -0.016756047,
                0.000015240103,
                -0.013285898,
                -0.0034113324,
                -0.009737328,
                0.006953367,
                0.019553078,
                -0.011639047,
                -0.016886748,
                0.01578885,
                -0.023644062,
                0.0016517515,
                0.004816383,
                -0.012286024,
                0.005309785,
                -0.019513866,
                0.001580682,
                0.0028819877,
                0.015958762,
                0.01582806,
                0.011652118,
                -0.0026875678,
                0.04064844,
                -0.03717176,
                0.01696517,
                0.014821651,
                0.036805794,
                -0.0045451755,
                -0.005643076,
                -0.00045786682,
                -0.0061789555,
                0.012933002,
                -0.009031535,
                -0.026650216,
                -0.0202458,
                -0.00873092,
                -0.0046693427,
                0.020533346,
                -0.007267053,
                0.0033492488,
                -0.012168393,
                0.00050279574,
                -0.012913396,
                -0.028571542,
                -0.0045288377,
                -0.016886748,
                0.00642729,
                0.026532585,
                -0.008136224,
                0.045641273,
                0.023265025,
                -0.017540261,
                0.013893664,
                -0.020075887,
                -0.003161364,
                -0.007293193,
                0.008338813,
                -0.010233997,
                -0.021435192,
                -0.010985536,
                -0.026048986,
                -0.017409557,
                0.0030224929,
                -0.024441347,
                -0.010933255,
                0.0025813722,
                0.05364026,
                -0.0021614907,
                -0.003184237,
                0.002212138,
                -0.012096507,
                -0.014246561,
                -0.014651738,
                -0.025421616,
                -0.026637146,
                -0.011482205,
                0.025957495,
                -0.013063704,
                -0.009351756,
                -0.0012187997,
                -0.0037348208,
                -0.039106157,
                0.0069076214,
                -0.0028476783,
                0.0005844848,
                0.015252969,
                0.018821144,
                -0.015645076,
                0.0109790005,
                -0.0012261518,
                0.029068211,
                0.009410572,
                -0.00010711469,
                0.027107676,
                -0.02296441,
                0.010423516,
                -0.014011296,
                -0.01913483,
                -0.005946959,
                0.024964156,
                0.015096126,
                -0.015449023,
                0.026872411,
                -0.03165612,
                0.0028264392,
                0.02791803,
                -0.012201068,
                0.018402897,
                0.016285518,
                -0.008168899,
                -0.033172265,
                -0.014324982,
                0.0010733934,
                -0.0027643556,
                -0.028179435,
                0.004950353,
                0.007005648,
                -0.006531852,
                0.00029203814,
                -0.0031973072,
                -0.019905973,
                0.005669216,
                -0.0097242575,
                0.01867737,
                0.0032871652,
                -0.030218393,
                0.02519942,
                0.005352263,
                -0.016220167,
                -0.0035649077,
                0.008070872,
                -0.032361913,
                -0.009743863,
                -0.005260771,
                -0.031133309,
                -0.011090098,
                -0.031054888,
                0.013985156,
                -0.0241538,
                0.0071428856,
                -0.0035453024,
                -0.003823045,
                -0.015043845,
                0.0051986873,
                0.040099494,
                -0.017906228,
                0.017840875,
                -0.01010983,
                -0.0018200307,
                -0.019108688,
                -0.0047150888,
                0.007155956,
                -0.016599204,
                0.02299055,
                0.0031809695,
                -0.0050353096,
                -0.029983127,
                -0.0016672723,
                0.0134231355,
                -0.017135084,
                0.03055822,
                0.18486546,
                0.008436839,
                -0.0155666545,
                0.017108943,
                0.020115098,
                0.004809848,
                -0.0001253926,
                0.020023607,
                -0.027029254,
                0.0037217506,
                -0.010299348,
                -0.0030241266,
                0.010665315,
                0.011135844,
                -0.009894171,
                -0.049353223,
                -0.040674582,
                -0.015710426,
                -0.013998226,
                -0.01120773,
                -0.0012808834,
                0.000927987,
                -0.024977226,
                -0.0038165099,
                -0.0032087436,
                -0.0202458,
                -0.016742976,
                -0.01801079,
                0.032231208,
                0.055417813,
                0.0043621925,
                -0.001306207,
                0.022258617,
                -0.019513866,
                0.007293193,
                -0.0014377263,
                0.007508852,
                0.0011795891,
                0.0119461985,
                0.0146778785,
                -0.02355257,
                -0.0023101647,
                -0.004881734,
                0.00036739625,
                0.010377769,
                -0.00004316753,
                -0.013985156,
                0.021134576,
                0.016194027,
                0.013919804,
                -0.014965423,
                -0.029225055,
                0.00066208927,
                0.0169521,
                0.009783074,
                -0.014272701,
                0.0073389392,
                -0.0032691935,
                -0.012135717,
                0.041301955,
                -0.003702145,
                0.019043338,
                -0.021069225,
                0.032152787,
                -0.01637701,
                0.010436586,
                -0.0031417587,
                0.00048809173,
                0.03494982,
                -0.015083056,
                0.029381897,
                -0.013959015,
                -0.015370601,
                -0.012606245,
                0.005166012,
                -0.01289379,
                0.035760175,
                -0.002553598,
                0.024663541,
                0.008881227,
                -0.020023607,
                -0.013593049,
                -0.0052019553,
                -0.0218273,
                0.020010537,
                -0.009966057,
                0.034322448,
                0.025303982,
                -0.0115018105,
                0.00036208646,
                -0.0069010863,
                0.0022366447,
                -0.005692089,
                0.016559992,
                0.015030775,
                0.01336432,
                0.0051169987,
                0.014246561,
                -0.028702244,
                0.02569609,
                0.028231716,
                -0.01176975,
                0.012710808,
                -0.005267306,
                -0.01961843,
                0.006724638,
                0.009756933,
                0.0028934241,
                -0.00685534,
                0.002694103,
                0.012926467,
                -0.017723244,
                0.002664695,
                0.000029408038,
                -0.005025507,
                -0.00053996424,
                0.025382403,
                -0.034113325,
                0.012233743,
                -0.030427516,
                0.0041988143,
                -0.004208617,
                -0.0054862327,
                0.022624584,
                0.007757187,
                -0.012567035,
                -0.011031282,
                0.0007523556,
                -0.0007613414,
                -0.026558725,
                0.011227335,
                -0.00025037676,
                0.007384685,
                0.011665189,
                -0.0032005748,
                0.025042579,
                0.023108182,
                -0.032440335,
                -0.00082791795,
                -0.00047175394,
                -0.0027986648,
                0.013553838,
                -0.014168139,
                -0.0009492261,
                0.0064763036,
                0.008528331,
                0.001395248,
                -0.0033051367,
                -0.0049111424,
                -0.013919804,
                -0.0064501627,
                0.00017185321,
                -0.009390967,
                -0.034217887,
                0.034871396,
                -0.020716328,
                -0.026493374,
                -0.020036677,
                -0.017383417,
                -0.0074565713,
                -0.00097536657,
                0.0031940397,
                0.024846524,
                -0.011822031,
                -0.007312799,
                -0.0056136674,
                -0.16531238,
                0.024572048,
                0.0077963974,
                -0.021003874,
                0.010188251,
                0.014743229,
                0.026663287,
                -0.006959902,
                -0.029225055,
                0.0131225195,
                0.0007670596,
                -0.0024474023,
                -0.03393034,
                -0.013115984,
                0.006626611,
                -0.0038328476,
                0.016024113,
                -0.00986803,
                0.014024367,
                -0.00507452,
                0.020585626,
                -0.027159957,
                0.020938523,
                -0.00083281926,
                0.023225814,
                -0.025513107,
                -0.017357277,
                0.000030863124,
                0.02676785,
                0.0037315532,
                -0.009384432,
                0.0062279687,
                0.01748798,
                -0.013298968,
                0.007554598,
                0.015488233,
                0.004015831,
                -0.018455178,
                -0.008815876,
                0.024402136,
                0.00008373122,
                -0.0011591668,
                0.012913396,
                0.0018951846,
                -0.013037563,
                0.030924186,
                0.0117958905,
                -0.017958509,
                0.035158943,
                0.008312672,
                0.006152815,
                -0.027238378,
                0.020820891,
                -0.008188505,
                0.012952607,
                0.033642795,
                0.0008254673,
                -0.000708652,
                -0.0033949947,
                -0.0071298154,
                -0.027029254,
                0.00023730652,
                -0.0024751765,
                -0.01744877,
                0.021618174,
                -0.030479798,
                0.0134231355,
                -0.013998226,
                -0.011978874,
                0.0074500362,
                0.008940044,
                -0.0035420347,
                -0.019200182,
                -0.019304743,
                0.005192152,
                0.026689429,
                0.005512373,
                0.0036760047,
                0.019213252,
                -0.020415714,
                -0.018951846,
                0.030139972,
                -0.02356564,
                -0.004215152,
                -0.0005224011,
                0.0029260998,
                -0.009083816,
                0.0012890523,
                -0.012959142,
                0.0026565262,
                0.009423642,
                -0.02674171,
                0.010495402,
                -0.013174801,
                0.030715061,
                0.0070383237,
                0.019004127,
                0.007920565,
                -0.021539753,
                -0.020964663,
                0.0115018105,
                -0.0055548516,
                -0.0077049057,
                0.00982882,
                0.037093338,
                0.034740694,
                -0.0071298154,
                0.021082295,
                0.013089844,
                0.024820384,
                -0.012272954,
                -0.0097242575,
                0.007665695,
                0.020167379,
                -0.023330377,
                0.000092053284,
                0.016167885,
                -0.037955973,
                0.011429924,
                -0.0044112056,
                0.015200688,
                -0.021748878,
                -0.034740694,
                0.017853945,
                -0.011161984,
                -0.020912383,
                -0.08412006,
                0.013749891,
                0.016886748,
                0.0035616402,
                -0.004339319,
                0.015226828,
                -0.027656626,
                -0.0055842595,
                0.019827552,
                0.013828313,
                0.0061332094,
                -0.0076983706,
                0.005388206,
                0.0015741469,
                0.020023607,
                0.016729906,
                -0.019513866,
                -0.009547809,
                -0.03123787,
                -0.006188758,
                -0.010724131,
                -0.0067050327,
                0.00018053268,
                -0.016141744,
                0.0044340785,
                -0.0039472124,
                -0.009783074,
                0.015579725,
                0.025029508,
                0.028545402,
                0.019866763,
                -0.010083689,
                0.016298588,
                0.0017530458,
                0.0218273,
                0.0033378124,
                0.009521669,
                0.0020062816,
                -0.0011109703,
                -0.024938015,
                -0.010469262,
                -0.000601231,
                0.008574077,
                -0.048882693,
                0.0007066098,
                -0.009390967,
                -0.0039210715,
                -0.012488613,
                -0.0045092325,
                -0.017592542,
                -0.0028819877,
                -0.0016272448,
                -0.0045092325,
                -0.012109577,
                0.020637907,
                0.01641622,
                0.004117125,
                0.0048948047,
                -0.022506952,
                0.014887002,
                -0.028754525,
                0.01644236,
                -0.0016599203,
                -0.0053065172,
                0.034819115,
                0.004721624,
                -0.005643076,
                0.011096633,
                0.009737328,
                -0.004796778,
                -0.0109071145,
                0.016102534,
                -0.022101775,
                0.028728385,
                -0.014651738,
                0.011338432,
                -0.013579979,
                -0.018520528,
                -0.005211758,
                -0.015017704,
                -0.005012437,
                0.000523218,
                -0.012318701,
                -0.010253603,
                -0.011599837,
                0.022598444,
                0.0066364137,
                -0.022402389,
                0.0070121833,
                -0.022023352,
                0.03664895,
                0.024441347,
                0.001209814,
                -0.019749131,
                0.011050887,
                -0.007280123,
                -0.0038655233,
                -0.008345348,
                -0.028676104,
                0.004470022,
                -0.044621795,
                -0.034270167,
                -0.07146807,
                0.015187617,
                0.0059927045,
                -0.005989437,
                0.009900706,
                -0.022533093,
                -0.0059044803,
                0.00764609,
                -0.016311659,
                0.007103675,
                -0.023160463,
                0.007907495,
                0.017252715,
                -0.013383925,
                -0.018873425,
                0.0034603458,
                0.004104055,
                0.010364699,
                -0.0011452797,
                -0.008031662,
                -0.018154562,
                -0.008900832,
                -0.0043687276,
                0.022559233,
                -0.031133309,
                0.008992325,
                -0.015906481,
                0.026898552,
                0.0041301954,
                0.0049928315,
                0.008887762,
                -0.024428276,
                -0.010711061,
                0.05364026,
                -0.024728892,
                -0.012926467,
                -0.013776032,
                0.021592034,
                0.012181463,
                0.023944678,
                -0.005401276,
                -0.022885988,
                0.014991564,
                -0.008325743,
                -0.008051267,
                0.014507965,
                0.0022105041,
                0.010037944,
                0.02679399,
                -0.013762962,
                0.02407538,
                0.02023273,
                0.001164885,
                -0.027996453,
                -0.0155666545,
                -0.006662554,
                0.008547937,
                0.019487726,
                -0.003659667,
                -0.0080251265,
                0.028493121,
                0.019043338,
                0.0022611513,
                0.005260771,
                0.047235843,
                -0.017396487,
                -0.001075844,
                0.0015553584,
                0.006241039,
                0.0052738413,
                -0.023905467,
                0.006456698,
                0.016782187,
                0.006391347,
                -0.0016321461,
                -0.0006812862,
                -0.014377262,
                -0.023931608,
                -0.016638415,
                0.011822031,
                0.0003896565,
                0.0008618189,
                -0.028623823,
                -0.001803693,
                0.017971579,
                -0.00068251154,
                -0.005662681,
                0.011083563,
                -0.0169521,
                0.011299222,
                0.0018576077,
                -0.011109703,
                -0.0034995566,
                0.0032986016,
                0.00478044,
                -0.0088354815,
                0.0025944423,
                -0.022193266,
                -0.00981575,
                0.037433166,
                -0.03160384,
                0.014390333,
                -0.007946705,
                -0.022833707,
                -0.006152815,
                0.0053195874,
                -0.014573316,
                -0.041223533,
                -0.015056916,
                -0.006136477,
                0.015383671,
                0.017553331,
                0.022245547,
                0.01862509,
                -0.011652118,
                -0.023199674,
                -0.021226069,
                0.024689682,
                -0.00929294,
                0.022716075,
                0.004290306,
                0.016808327,
                0.020023607,
                -0.009266799,
                0.012750018,
                0.012782694,
                0.0028313405,
                0.00063186436,
                -0.016259378,
                0.0047673695,
                0.0019981128,
                -0.017357277,
                -0.01645543,
                -0.003154829,
                -0.018546669,
                0.013985156,
                0.0069141565,
                -0.0075742034,
                -0.025486967,
                0.03434859,
                0.027003113,
                -0.00628025,
                0.007829073,
                -0.013945945,
                0.04553671,
                0.0031875046,
                0.012606245,
                -0.015723497,
                -0.011116238,
                0.010743736,
                -0.0016452164,
                0.0040125633,
                -0.009404037,
                -0.033747356,
                0.0036792723,
                -0.020637907,
                0.0069076214,
                -0.011534486,
                -0.019278603,
                0.01010983,
                -0.0050941254,
                0.009273334,
                0.0037250181,
                -0.011920058,
                0.0049830284,
                0.018232983,
                0.0071428856,
                -0.029277336,
                -0.024977226,
                0.018285265,
                0.02242853,
                -0.04653005,
                -0.011841636,
                -0.00930601,
                0.0017465107,
                -0.0015864002,
                0.021382911,
                -0.002069999,
                0.008449909,
                -0.004920945,
                0.036361404,
                -0.034766834,
                -0.019540008,
                0.0008879594,
                -0.010436586,
                -0.008456444,
                -0.0008177068,
                -0.03445315
            ]
    }
]

Formulate the request. This upload_documents request targets the docs collection of the hotels-quickstart index and pushes the documents provided in the previous step into the search index.

In [57]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

Upload of new document succeeded: True


In [58]:
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='HotelName,Description', 
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])

Total Documents Matching Query: 4
1.0
Triple Landscape Hotel
1.0
Twin Dome Motel
1.0
Sublime Cliff Hotel
1.0
Secret Point Motel


In [59]:
results =  search_client.search(query_type='simple',
    search_text="hotel" , order_by='Rating desc',
    select='HotelId,HotelName,Rating',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: {} - {} rating".format(result["HotelId"], result["HotelName"], result["Rating"]))

Total Documents Matching Query: 4
1.0144587
3: Triple Landscape Hotel - 4.80 rating
0.9284746
4: Sublime Cliff Hotel - 4.60 rating
0.51237863
1: Secret Point Motel - 3.60 rating
0.509938
2: Twin Dome Motel - 3.60 rating


In [60]:
results = search_client.search(search_text="motel", select='HotelId,HotelName,Rating',  order_by='Rating desc', include_total_count=True)
print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: {} - {} rating".format(result["HotelId"], result["HotelName"], result["Rating"]))

Total Documents Matching Query: 2
0.6099695
2: Twin Dome Motel - 3.60 rating
0.25316024
1: Secret Point Motel - 3.60 rating


In [61]:
results = search_client.search(search_text="sublime", search_fields=['HotelName'], select='HotelId,HotelName', include_total_count=True)
print ('Total Documents Matching Query:', results.get_count())

for result in results:
    print(result["@search.score"])
    print("{}: {}".format(result["HotelId"], result["HotelName"]))

Total Documents Matching Query: 1
0.6099695
4: Sublime Cliff Hotel


In [62]:
results = search_client.search(search_text="wifi", include_total_count=True, select='HotelId,HotelName,Tags')
print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: {}: {}".format(result["HotelId"], result["HotelName"], result["Tags"]))

Total Documents Matching Query: 1
0.78019357
2: Twin Dome Motel: ['pool', 'free wifi', 'concierge']


In [64]:
results =  search_client.search(query_type='simple',
    search_text="what hotel has a good restaurant on site" ,
    select='HotelName,HotelId,Description')

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    

2.1393623
None
Sublime Cliff Hotel
1.9309065
None
Twin Dome Motel
1.5589908
None
Triple Landscape Hotel
0.7704947
None
Secret Point Motel


In [65]:
results =  search_client.search(query_type='semantic',semantic_configuration_name='my-semantic-config',
    search_text="what hotel has a good restaurant on site" ,
    select='HotelName,HotelId,Description')

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    

1.5589908
1.8807636499404907
Triple Landscape Hotel
2.1393623
1.527956485748291
Sublime Cliff Hotel
0.7704947
1.3261334896087646
Secret Point Motel
1.9309065
1.0473345518112183
Twin Dome Motel


In [67]:
results =  search_client.search(query_type='simple',
    search_text="what hotel stands out for its gastronomic excellence", 
    select='HotelName,Description,Category')

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])

2.3295155
None
Triple Landscape Hotel
0.9284746
None
Sublime Cliff Hotel
0.51237863
None
Secret Point Motel
0.509938
None
Twin Dome Motel


In [69]:
vectors = [
                0.0023828433,
                -0.02840831,
                -0.007473698,
                -0.03132622,
                -0.0007359287,
                -0.0021935923,
                -0.013109939,
                -0.010611825,
                -0.016984425,
                0.004511058,
                0.028738638,
                0.006775189,
                -0.029949846,
                -0.009235454,
                -0.010432897,
                0.026343754,
                0.0124424,
                0.006155822,
                0.0007002291,
                -0.025352765,
                -0.01703948,
                0.008980825,
                -0.009689656,
                -0.025641803,
                0.010315905,
                0.008340812,
                0.004511058,
                -0.027279686,
                0.010102567,
                0.0013041121,
                0.031601492,
                0.0008645334,
                -0.034904785,
                0.012167125,
                -0.011465175,
                0.0024017685,
                -0.012538745,
                0.02315057,
                0.025848258,
                0.019764697,
                0.015085033,
                -0.02133376,
                0.007081432,
                0.0052130073,
                -0.021691617,
                -0.010350315,
                0.0055536595,
                0.005391936,
                -0.029839737,
                0.010157622,
                0.020191371,
                0.022021946,
                -0.009951167,
                -0.02266884,
                -0.00020064917,
                0.00970342,
                0.0025273624,
                0.017768957,
                -0.004232343,
                -0.0065102377,
                0.0018821881,
                -0.0008189411,
                -0.04255741,
                0.008471567,
                -0.003229312,
                0.00851974,
                -0.027431086,
                0.013997699,
                0.011052264,
                0.0079279,
                0.028298201,
                0.024733398,
                0.015594291,
                0.009682775,
                0.010811399,
                -0.02740356,
                -0.0064345375,
                0.0042289016,
                0.008106829,
                0.004707191,
                0.024182849,
                -0.046658996,
                0.006606584,
                0.00047097716,
                0.007356706,
                -0.0015621817,
                -0.011499585,
                0.01759003,
                -0.014699649,
                0.006018185,
                0.022434857,
                0.020370299,
                0.01966835,
                -0.0046486952,
                -0.036501374,
                0.014286737,
                0.0068543307,
                0.040547907,
                0.00396395,
                0.0061730267,
                0.0017342282,
                -0.0052198893,
                -0.008733078,
                -0.014575776,
                -0.013054885,
                0.0105567705,
                -0.018773708,
                0.0038022266,
                0.025641803,
                -0.0028009163,
                -0.01366737,
                0.011946905,
                0.022765186,
                -0.03751989,
                -0.046989325,
                -0.0035011454,
                0.017810248,
                0.010405369,
                -0.020920848,
                -0.024526943,
                0.01774143,
                0.0034306063,
                0.037327196,
                -0.00351835,
                0.0345194,
                0.0058564614,
                0.000001844795,
                -0.021182358,
                -0.004569554,
                0.0018064878,
                0.048393227,
                0.023769937,
                0.033500884,
                -0.015993439,
                -0.022889059,
                0.0008598021,
                -0.012063897,
                0.04233719,
                -0.020232663,
                -0.009730947,
                0.001959609,
                0.011224311,
                -0.013715543,
                0.018347034,
                -0.0012146479,
                0.0062865773,
                0.014410611,
                0.041208565,
                -0.008788133,
                -0.021650326,
                -0.010721935,
                0.0012086263,
                -0.0008701249,
                0.007941664,
                0.009070289,
                0.022806477,
                0.0077971453,
                0.028435837,
                -0.029894792,
                -0.018828765,
                -0.02085203,
                0.009228571,
                -0.010673762,
                0.0066788434,
                -0.0094969645,
                0.012249707,
                0.0267429,
                0.035290167,
                -0.018828765,
                -0.013020475,
                -0.01511256,
                0.04547532,
                -0.018402088,
                0.0139495265,
                0.013571024,
                -0.002085203,
                0.009662129,
                -0.0141215725,
                0.004511058,
                0.00188907,
                -0.014534485,
                0.01548418,
                0.032922808,
                0.024829743,
                0.013495323,
                0.004022446,
                0.034381762,
                0.012511218,
                -0.017782722,
                -0.03655643,
                0.016433878,
                0.008505977,
                0.00685089,
                0.0015656227,
                -0.63423204,
                -0.016048493,
                -0.026247406,
                0.013412741,
                0.022751423,
                0.03518006,
                0.020865794,
                0.019544477,
                -0.020315245,
                0.0065102377,
                -0.013756834,
                -0.02048041,
                -0.0072328327,
                -0.0026271492,
                0.00022301522,
                -0.040575434,
                0.019489422,
                0.00037914736,
                -0.03443682,
                0.004266752,
                -0.007914137,
                0.032207094,
                -0.014878578,
                0.00010693762,
                0.012992948,
                0.0073498245,
                -0.0067579844,
                -0.023604773,
                -0.020012444,
                -0.013805008,
                -0.016213657,
                0.020342773,
                0.020783212,
                0.010508597,
                0.035868246,
                -0.011843678,
                -0.038125493,
                0.022655077,
                -0.009792884,
                0.031436328,
                -0.00029850058,
                -0.011451412,
                0.028009163,
                0.010907745,
                0.00959331,
                0.01721841,
                0.018787473,
                -0.0067476616,
                0.006069799,
                -0.006159263,
                -0.0135916695,
                -0.00009779765,
                -0.016984425,
                0.01955824,
                0.007363588,
                -0.017768957,
                0.018828765,
                -0.0071502505,
                -0.014506957,
                -0.0024052095,
                0.0036026528,
                0.009324918,
                -0.00073807925,
                -0.048310645,
                -0.005863343,
                0.008230702,
                -0.010157622,
                0.022586258,
                0.019131565,
                0.0007802307,
                -0.016144838,
                0.016846789,
                -0.013715543,
                0.029316716,
                -0.009958048,
                0.021732908,
                0.018938873,
                -0.0068164803,
                0.010226442,
                0.02641257,
                -0.000007963869,
                -0.017837776,
                -0.015002451,
                0.0035235113,
                0.013275105,
                0.023618536,
                -0.03506995,
                0.0038056676,
                0.023315735,
                -0.0011200224,
                0.01548418,
                0.0039673913,
                -0.019461894,
                -0.011671632,
                0.006341632,
                0.0015725045,
                -0.022269692,
                -0.011423884,
                -0.0018942314,
                -0.038841207,
                -0.020810738,
                -0.007914137,
                0.019943625,
                0.015098796,
                0.026082242,
                0.009056525,
                -0.018677363,
                0.0072672423,
                0.010790753,
                -0.018897582,
                -0.0030727496,
                -0.010866454,
                -0.033115502,
                -0.02407274,
                0.01759003,
                -0.036308683,
                0.00020989667,
                0.03138127,
                -0.013688016,
                -0.00095571805,
                0.026192352,
                -0.027582487,
                0.025311474,
                -0.008492213,
                0.0038469587,
                0.0202189,
                0.022916587,
                -0.014534485,
                0.016805498,
                0.0044904123,
                0.021209886,
                0.029426824,
                0.020136317,
                -0.018291978,
                0.016420113,
                0.016089784,
                -0.0009849659,
                -0.014589539,
                0.02626117,
                -0.024664579,
                -0.01844338,
                -0.014087164,
                -0.017947886,
                -0.024127794,
                -0.018883819,
                -0.0094832005,
                0.021127304,
                0.0078797275,
                -0.01837456,
                -0.0018718654,
                -0.0101989135,
                0.012174007,
                -0.0158558,
                0.0059975395,
                0.014410611,
                -0.011238074,
                0.00451794,
                -0.021154832,
                -0.021471398,
                -0.005288708,
                0.009744711,
                0.0038572815,
                -0.02781647,
                -0.02337079,
                0.02981221,
                -0.027086994,
                0.019310493,
                0.0008077581,
                -0.026797956,
                -0.010756345,
                0.009166635,
                -0.03374863,
                0.0043286886,
                0.028009163,
                -0.027995398,
                0.001662829,
                -0.016599042,
                -0.010990327,
                0.020053735,
                0.015800746,
                0.03600588,
                0.0063347504,
                -0.029509407,
                0.014396847,
                0.02389381,
                0.010825163,
                -0.008210056,
                0.003919218,
                -0.021485161,
                0.028766166,
                -0.010350315,
                0.017755194,
                -0.035125002,
                0.019104037,
                -0.0037265262,
                -0.0023312294,
                0.028243145,
                0.002695968,
                0.008155002,
                0.015277725,
                0.015415362,
                0.030803196,
                0.0067614256,
                -0.009944285,
                0.011238074,
                0.010150741,
                0.011829914,
                -0.008299521,
                0.044621967,
                0.012057016,
                -0.00970342,
                -0.011375711,
                -0.010453543,
                -0.033198085,
                0.011403238,
                0.013460915,
                0.008836306,
                0.021416342,
                0.006534324,
                0.004985906,
                0.023618536,
                0.017424865,
                0.0068474486,
                0.037382253,
                0.006844008,
                -0.008086183,
                0.0073842336,
                0.016158603,
                0.015525471,
                0.002711452,
                -0.019764697,
                0.021567743,
                0.012573155,
                0.010350315,
                0.019503186,
                -0.0045798765,
                0.013412741,
                0.0037127624,
                0.0105567705,
                -0.011905614,
                0.044154,
                0.0234809,
                0.013460915,
                -0.023880048,
                0.018911347,
                -0.008382103,
                0.0091253435,
                0.012001961,
                -0.020315245,
                0.0030589858,
                -0.008602323,
                -0.002618547,
                -0.024554469,
                0.030720614,
                -0.014410611,
                0.0023398318,
                -0.018044231,
                0.012931012,
                -0.0042770747,
                0.027169576,
                -0.0071571325,
                0.009152872,
                0.016351296,
                0.0049652606,
                0.029344242,
                -0.0056052734,
                0.0028060777,
                -0.021237414,
                -0.00911158,
                -0.013681134,
                -0.0073773516,
                -0.019916097,
                0.00833393,
                0.0015578807,
                -0.011293129,
                -0.0018168106,
                -0.013220049,
                0.0082375845,
                -0.039391756,
                0.006785512,
                -0.010315905,
                -0.0128621925,
                0.030582977,
                0.008561032,
                0.0026839247,
                0.013130585,
                -0.021127304,
                0.005825493,
                -0.037932802,
                0.042584937,
                0.016984425,
                0.0050512836,
                -0.017603792,
                -0.010735699,
                -0.013935762,
                -0.005866784,
                0.03911648,
                -0.03226215,
                0.0077764997,
                0.0018322947,
                0.024114031,
                -0.011823032,
                0.017920358,
                -0.030582977,
                0.044126473,
                -0.029454352,
                0.0046486952,
                -0.018539725,
                0.008609205,
                -0.03077567,
                -0.0006731317,
                0.01592462,
                -0.0134058595,
                0.00451794,
                0.005567423,
                -0.017094536,
                -0.021485161,
                -0.024348013,
                0.028064217,
                0.00070840126,
                -0.008010482,
                -0.015759455,
                -0.006152381,
                -0.00050323585,
                0.09832799,
                0.0057910834,
                -0.010391606,
                0.010019985,
                0.009345563,
                0.023012934,
                -0.052494813,
                -0.02074192,
                -0.0067614256,
                0.02137505,
                0.013660489,
                -0.00833393,
                0.020948377,
                0.02299917,
                0.017204646,
                0.0045213806,
                0.002792314,
                -0.015800746,
                0.007831555,
                0.011905614,
                -0.018870056,
                -0.0139495265,
                0.015773218,
                0.024306722,
                0.012648855,
                0.014713412,
                0.031546436,
                0.014245446,
                -0.010157622,
                -0.021843018,
                -0.017438628,
                0.0065308833,
                -0.017534975,
                -0.008595441,
                -0.003547598,
                0.0107288165,
                0.010604943,
                -0.023260681,
                0.018787473,
                0.015649345,
                0.012421754,
                -0.011251838,
                -0.01752121,
                -0.006362278,
                -0.0058461386,
                0.004755364,
                -0.011017854,
                0.034244128,
                -0.018663598,
                -0.0054538725,
                -0.006493033,
                0.012676382,
                -0.0031398477,
                0.0015931502,
                -0.022710131,
                -0.007404879,
                -0.014754703,
                -0.027059466,
                -0.019434366,
                -0.014176628,
                -0.013020475,
                -0.01844338,
                0.022765186,
                -0.0070160544,
                -0.04247483,
                -0.009393737,
                -0.01721841,
                -0.02403145,
                -0.0125938,
                -0.020136317,
                -0.0057635563,
                0.0044732075,
                -0.022104528,
                -0.0112862475,
                0.03162902,
                -0.0008266832,
                -0.013564142,
                0.008512858,
                0.009049644,
                0.026729137,
                0.0033789922,
                0.009510728,
                -0.016791733,
                -0.026646554,
                -0.0061730267,
                0.018030468,
                -0.005281826,
                -0.000388825,
                0.003122643,
                0.0049170875,
                0.0016456243,
                0.009751593,
                0.012208416,
                0.0107288165,
                0.009090935,
                -0.017163355,
                0.0073911157,
                0.021017194,
                0.013818771,
                -0.023508428,
                0.0114445295,
                0.008891361,
                -0.022434857,
                -0.021471398,
                -0.0021454194,
                -0.0112862475,
                -0.0072947694,
                -0.0089395335,
                0.0008722755,
                -0.017810248,
                0.005347204,
                0.0032774848,
                0.014176628,
                0.01633753,
                -0.002011223,
                0.042034388,
                0.0017058406,
                0.012834665,
                -0.0030968362,
                -0.0066650794,
                0.0017617557,
                -0.030087484,
                0.009311154,
                0.021911835,
                -0.016612805,
                0.004308043,
                -0.005567423,
                -0.019819751,
                -0.019434366,
                -0.005736029,
                -0.010171386,
                -0.0020972462,
                -0.012476808,
                -0.018677363,
                -0.026343754,
                -0.020122552,
                -0.009400618,
                0.0234809,
                -0.005608714,
                -0.0040018004,
                -0.045365207,
                -0.021664089,
                -0.0061764675,
                0.0047897734,
                -0.002766507,
                -0.030170066,
                -0.007315415,
                -0.00023806927,
                -0.0029970491,
                0.015786983,
                -0.002596181,
                0.0008060376,
                0.0010580857,
                0.033087973,
                -0.005323117,
                -0.021154832,
                -0.00023054224,
                -0.017713903,
                -0.0033463035,
                0.018539725,
                0.019282967,
                -0.009311154,
                0.008843187,
                0.03173913,
                0.010343432,
                -0.000266672,
                -0.010467306,
                0.006214318,
                -0.0048585916,
                0.018250687,
                0.00825823,
                0.0041015875,
                -0.009311154,
                0.034354236,
                -0.023769937,
                0.019076511,
                0.02026019,
                -0.005003111,
                -0.028353255,
                -0.032537427,
                0.014190392,
                -0.02481598,
                0.0024912327,
                -0.018883819,
                -0.0008843188,
                -0.012387345,
                0.0042770747,
                0.019530714,
                0.012476808,
                -0.008636732,
                0.02893133,
                -0.016365059,
                0.018209396,
                -0.020686865,
                0.018457143,
                0.0028215619,
                -0.030170066,
                -0.019585768,
                -0.013543497,
                0.015429126,
                0.013158113,
                0.016048493,
                -0.030830724,
                0.016144838,
                -0.009682775,
                0.018154342,
                0.01892511,
                -0.0078522,
                -0.01299983,
                -0.008698668,
                0.0003914057,
                -0.001240455,
                -0.014823522,
                -0.015979674,
                -0.005467636,
                0.003325658,
                -0.039309174,
                0.03473962,
                -0.036060937,
                -0.028463366,
                0.0039261,
                -0.002203915,
                0.027637541,
                0.010102567,
                0.028009163,
                0.005388495,
                0.01811305,
                -0.02637128,
                0.016736679,
                -0.0035992118,
                -0.029949846,
                -0.0002348434,
                0.02181549,
                0.021540215,
                -0.016062256,
                0.011196783,
                0.008382103,
                -0.02315057,
                -0.030362757,
                0.03366605,
                0.01077699,
                0.0018271334,
                -0.006441419,
                0.005240535,
                -0.024802217,
                0.02299917,
                0.019393075,
                -0.008471567,
                0.0134196235,
                0.0006232383,
                -0.023535954,
                0.03484973,
                -0.006455183,
                0.0029488762,
                -0.002830164,
                -0.037602473,
                -0.004500735,
                -0.026674083,
                -0.0020129434,
                0.022159584,
                -0.0042770747,
                0.035427805,
                0.014396847,
                -0.0075838077,
                -0.021953126,
                0.002255529,
                0.008340812,
                -0.007136487,
                -0.018525962,
                0.021168595,
                -0.028876277,
                -0.008072419,
                -0.0030176947,
                0.0028508096,
                0.009675892,
                -0.026660318,
                -0.014231683,
                -0.006338191,
                -0.015374071,
                -0.01066688,
                0.0049136467,
                0.009944285,
                -0.0056362418,
                0.0049721426,
                0.004218579,
                -0.0011071189,
                -0.01318564,
                -0.0061179716,
                -0.00067700277,
                -0.026646554,
                -0.012924129,
                -0.038235605,
                -0.026729137,
                0.00094797596,
                0.01929673,
                -0.0007656067,
                0.010336551,
                0.016750442,
                -0.02692183,
                -0.004019005,
                -0.0030761906,
                0.03600588,
                -0.025215128,
                0.006210877,
                -0.016874315,
                0.0046899864,
                0.019434366,
                -0.019723406,
                -0.022462385,
                -0.03845582,
                -0.0033738308,
                -0.009152872,
                0.023673592,
                0.018525962,
                0.002670161,
                0.012710792,
                -0.0031157613,
                -0.009152872,
                -0.0061867903,
                0.012242826,
                -0.022682603,
                0.008684905,
                0.03633621,
                0.0064310962,
                0.0141215725,
                0.014768467,
                -0.0023983275,
                -0.025201363,
                -0.0107425805,
                0.015043742,
                -0.011898733,
                0.034216598,
                -0.024485651,
                -0.014506957,
                -0.040272634,
                0.0042529884,
                -0.009318036,
                0.041649006,
                0.0016112151,
                -0.0073016514,
                0.03262001,
                0.032867756,
                -0.00407406,
                0.012600682,
                0.000029032839,
                -0.038428295,
                0.008017365,
                -0.0026684406,
                -0.020562991,
                -0.03796033,
                -0.008540385,
                0.024361778,
                0.008505977,
                -0.023067988,
                -0.011437648,
                0.00014602442,
                -0.022834005,
                -0.008870715,
                -0.016791733,
                0.018126814,
                0.040602963,
                -0.03410649,
                -0.016695388,
                -0.025944605,
                -0.027761415,
                0.021251177,
                -0.005171716,
                0.021953126,
                -0.021196123,
                -0.010439779,
                0.0066650794,
                -0.008010482,
                -0.04096082,
                0.0012137877,
                0.010536125,
                0.004796655,
                0.0070539047,
                0.0148510495,
                -0.038153023,
                0.0091253435,
                -0.0029936084,
                -0.008093065,
                0.014589539,
                0.016420113,
                0.006255609,
                -0.05221954,
                0.007122723,
                0.013316396,
                0.004225461,
                -0.004466326,
                0.0058839885,
                0.009421264,
                0.0048689144,
                0.011361947,
                0.0060766805,
                -0.033390775,
                0.010749462,
                -0.028601002,
                0.0027045703,
                0.0049583786,
                -0.020205135,
                0.014795994,
                -0.015497944,
                0.0004795795,
                -0.0089395335,
                -0.0032912486,
                -0.035290167,
                0.0015011053,
                0.0067545436,
                -0.031876765,
                -0.018016705,
                -0.021760436,
                0.011871205,
                -0.001458954,
                0.013158113,
                0.012483691,
                -0.027431086,
                -0.01848467,
                0.034491874,
                0.0043906253,
                -0.028573474,
                -0.0072328327,
                0.003392756,
                -0.010150741,
                -0.002792314,
                0.00061635644,
                0.0123322895,
                -0.011671632,
                -0.00048043972,
                0.030858252,
                -0.026343754,
                -0.04853086,
                0.016640333,
                -0.008877597,
                -0.01425921,
                0.019104037,
                0.20810738,
                0.02321939,
                -0.035648026,
                0.04252988,
                0.0011931421,
                0.03162902,
                0.03416154,
                0.03540028,
                0.0003647385,
                -0.0069162673,
                -0.015029978,
                0.009565783,
                0.0023639183,
                0.006544647,
                0.007026377,
                -0.0157044,
                -0.038841207,
                -0.03666654,
                0.0071846596,
                0.00028946815,
                0.0018632631,
                -0.031794183,
                -0.017686374,
                0.005137307,
                0.030610505,
                0.019544477,
                0.0058805477,
                0.0035751252,
                0.023315735,
                0.03751989,
                -0.011726686,
                0.0036060936,
                0.0021505808,
                0.005188921,
                0.0021024076,
                -0.0056465645,
                0.018498434,
                -0.0040843827,
                0.006162704,
                0.0026082243,
                -0.015153851,
                -0.02048041,
                0.027541196,
                -0.011341302,
                -0.0007385094,
                -0.016612805,
                0.0024671461,
                0.006465506,
                0.0022830565,
                0.027912816,
                -0.03881368,
                -0.00407406,
                0.0069059446,
                0.026123533,
                0.008829424,
                0.0015811069,
                0.017397337,
                -0.022751423,
                -0.03077567,
                0.003495984,
                -0.01174045,
                0.018057996,
                -0.030197592,
                0.0036611485,
                -0.02359101,
                -0.00911158,
                -0.011843678,
                0.013983936,
                0.016048493,
                -0.00032860873,
                -0.003447811,
                -0.035868246,
                -0.015153851,
                -0.0055571003,
                -0.035703078,
                -0.0054951636,
                0.016764207,
                0.009331799,
                0.01607602,
                0.05285267,
                -0.008327048,
                -0.008327048,
                -0.0024843507,
                0.014479429,
                -0.016048493,
                -0.042722575,
                0.0027819911,
                0.019048983,
                -0.0018099287,
                0.0008133496,
                0.0030624268,
                -0.0004881818,
                -0.021182358,
                0.0054469905,
                -0.005374731,
                0.009524492,
                0.0055364547,
                0.01566311,
                0.0031088793,
                -0.013158113,
                -0.017479919,
                -0.006782071,
                0.0010013103,
                -0.002303702,
                -0.024939854,
                -0.021044722,
                0.0027991957,
                0.01085269,
                -0.012187771,
                -0.010247087,
                -0.007914137,
                -0.023563482,
                -0.005240535,
                0.0023415522,
                0.0018684245,
                -0.012710792,
                -0.013832535,
                -0.013068648,
                0.018677363,
                -0.014919869,
                0.01984728,
                -0.01240799,
                -0.016901843,
                0.013082412,
                -0.016543986,
                -0.02759625,
                0.010983446,
                -0.006011303,
                0.014011463,
                -0.037822694,
                0.002271013,
                -0.02315057,
                0.005742911,
                -0.007700799,
                -0.031491384,
                -0.0054814,
                -0.01229788,
                -0.004751923,
                -0.02277895,
                -0.013158113,
                0.014672121,
                0.013756834,
                0.008196293,
                -0.005584628,
                0.0041532014,
                -0.03204193,
                -0.004442239,
                0.01940684,
                -0.027830234,
                -0.020865794,
                0.019255439,
                -0.007817791,
                0.0022813359,
                0.0014228241,
                0.02766507,
                -0.036308683,
                -0.024375541,
                -0.010432897,
                0.013034239,
                -0.009331799,
                -0.034023907,
                -0.0010305583,
                0.04451186,
                -0.0027991957,
                -0.031188581,
                0.024733398,
                -0.17705645,
                0.020604283,
                0.013385214,
                -0.028215619,
                0.01651646,
                0.0029351125,
                0.022283457,
                0.022283457,
                0.004903324,
                0.002985006,
                0.0068095988,
                0.0031604932,
                -0.03184924,
                -0.019489422,
                -0.0021127304,
                0.006943795,
                0.013426505,
                -0.022462385,
                0.0013118542,
                -0.019764697,
                0.03215204,
                0.008492213,
                0.0022933793,
                -0.009290508,
                -0.0047897734,
                -0.012352935,
                -0.004992788,
                0.026054714,
                0.0048585916,
                -0.011754214,
                -0.015319016,
                0.0016611085,
                0.019159093,
                0.009834175,
                -0.012793374,
                -0.0007729187,
                -0.021044722,
                -0.012194652,
                -0.002400048,
                0.007618217,
                -0.000503666,
                0.008512858,
                0.0074599343,
                -0.005801406,
                -0.0020473527,
                -0.00021602583,
                0.015263962,
                -0.0083958665,
                0.023205625,
                -0.0070539047,
                -0.014410611,
                -0.014176628,
                0.018980164,
                -0.010150741,
                -0.02685301,
                0.01703948,
                -0.01818187,
                0.0202189,
                -0.0008912006,
                -0.002769948,
                -0.02744485,
                -0.0089532975,
                0.015786983,
                0.00071614335,
                -0.010646234,
                -0.026632791,
                -0.0020731597,
                0.009256099,
                0.0038676043,
                0.003055545,
                -0.013089294,
                -0.02529771,
                -0.00062882976,
                -0.0076870355,
                0.011561521,
                0.026935592,
                0.001914877,
                0.022503676,
                0.002477469,
                -0.020246426,
                -0.006121413,
                0.029316716,
                -0.020439118,
                0.029867264,
                -0.022889059,
                0.0072053056,
                -0.0004920529,
                0.011540876,
                0.018099288,
                -0.025903314,
                0.018815,
                -0.01136883,
                -0.007322297,
                -0.011079792,
                0.015043742,
                0.0020731597,
                0.0112587195,
                0.017755194,
                0.005130425,
                0.0076044532,
                -0.00050968764,
                -0.0023639183,
                -0.029344242,
                0.01007504,
                0.00951761,
                0.012614446,
                0.0034082402,
                0.04630114,
                0.024251668,
                -0.0022675723,
                0.0075838077,
                0.009077171,
                0.032840226,
                0.02085203,
                0.005003111,
                -0.014314265,
                -0.01770014,
                0.007287888,
                0.022985406,
                0.044071417,
                0.014438138,
                0.0056912964,
                -0.0075012255,
                0.00039699717,
                -0.00959331,
                -0.009847939,
                -0.078893624,
                0.0051235436,
                0.013825653,
                0.017190881,
                -0.02626117,
                0.021388816,
                0.00351835,
                0.017424865,
                -0.027059466,
                0.037877746,
                0.005742911,
                -0.04500735,
                0.006499915,
                0.0012189491,
                0.02352219,
                0.010047513,
                0.03410649,
                -0.020907085,
                0.02740356,
                0.002081762,
                0.0068199215,
                -0.015126324,
                0.024100266,
                0.018016705,
                0.0040843827,
                -0.015938383,
                -0.021843018,
                0.010322787,
                0.005374731,
                0.019503186,
                0.026233643,
                -0.016433878,
                -0.007921019,
                -0.005708501,
                -0.015153851,
                0.008616086,
                -0.002129935,
                0.0017824012,
                0.02840831,
                -0.013846299,
                0.0017996059,
                0.021086013,
                0.013247577,
                -0.044456802,
                -0.010494834,
                -0.0041187922,
                -0.0140734,
                0.037795164,
                0.019489422,
                -0.02777518,
                0.004232343,
                -0.0062934593,
                0.004356216,
                0.004314925,
                0.01955824,
                -0.023535954,
                0.003871045,
                0.0034977044,
                -0.0056981784,
                -0.0013841137,
                -0.0014735778,
                -0.013137467,
                -0.014644594,
                -0.008678023,
                0.033087973,
                0.02655021,
                0.0035063068,
                -0.00659282,
                0.0026237085,
                -0.001551859,
                0.007170896,
                -0.00064087304,
                -0.025449112,
                0.005539896,
                -0.0137774795,
                0.014548248,
                -0.025600512,
                -0.021237414,
                -0.0071984236,
                -0.005522691,
                0.0046142857,
                -0.025793204,
                0.016268713,
                -0.012414872,
                0.022421094,
                -0.0000034913332,
                -0.004256429,
                -0.006599702,
                0.030142538,
                -0.045778118,
                -0.0021488601,
                0.008526622,
                0.009042761,
                -0.013103058,
                -0.037024397,
                0.016640333,
                0.008540385,
                0.006995409,
                -0.00024645653,
                -0.027031938,
                -0.028601002,
                -0.008609205,
                -0.076636374,
                0.0031536114,
                0.021801727,
                -0.00025204805,
                0.018732417,
                -0.0028869396,
                0.007625099,
                -0.0049170875,
                0.012167125,
                0.0060009803,
                -0.009634601,
                0.011231192,
                -0.01114861,
                -0.008313284,
                -0.027472377,
                -0.00019839106,
                0.02385252,
                -0.0078522,
                0.023687355,
                0.010673762,
                -0.014094045,
                0.002933392,
                0.024788452,
                -0.0052026846,
                -0.0007845318,
                0.015305253,
                -0.016447641,
                0.017562501,
                -0.018663598,
                -0.016241185,
                0.02240733,
                -0.014245446,
                -0.015305253,
                0.04481466,
                -0.029729627,
                -0.052026846,
                -0.025710622,
                0.0026047833,
                0.035097476,
                -0.01548418,
                0.0061248536,
                -0.043273125,
                0.016144838,
                -0.02389381,
                0.0076388624,
                0.006045712,
                -0.01622742,
                0.007976074,
                -0.002393166,
                -0.011010973,
                0.031986877,
                0.008726196,
                -0.0107425805,
                -0.009565783,
                -0.00685089,
                -0.023673592,
                0.007466816,
                -0.00396395,
                0.016915606,
                -0.03239979,
                0.0141491005,
                0.012951657,
                -0.0009875466,
                0.014617067,
                0.028711112,
                -0.007274124,
                -0.00918728,
                -0.004005241,
                0.010646234,
                -0.03248237,
                -0.012607564,
                0.016901843,
                0.009201044,
                0.024540706,
                0.009214808,
                -0.047319654,
                -0.005237094,
                -0.016199894,
                -0.024513178,
                0.0021488601,
                -0.0059803347,
                -0.009283626,
                -0.014190392,
                0.014176628,
                0.008003601,
                0.0054366677,
                0.002541126,
                -0.011017854,
                0.0058908705,
                0.013942644,
                0.0037678173,
                -0.005718824,
                -0.004139438,
                0.0011871206,
                0.011052264,
                0.012050133,
                -0.012621327,
                -0.016048493,
                0.022572495,
                0.03432671,
                -0.006936913,
                0.0020903645,
                -0.02740356,
                -0.0101851495,
                -0.004029328,
                0.009283626,
                -0.014451902,
                -0.045695536,
                0.008093065,
                0.005584628,
                0.017548738,
                0.02240733,
                0.005137307,
                0.01844338,
                0.021099776,
                -0.016901843,
                -0.0042770747,
                -0.013027357,
                -0.023687355,
                0.0026873655,
                0.011031618,
                0.014369319,
                0.0054504317,
                0.0072672423,
                0.0015544397,
                0.0066650794,
                0.016420113,
                -0.0021557421,
                0.009641483,
                0.004569554,
                -0.03204193,
                0.0025325238,
                -0.0061351764,
                -0.010659998,
                -0.014018345,
                -0.00031312453,
                -0.013447151,
                0.0009565783,
                0.0060732397,
                0.073663406,
                -0.010267733,
                -0.014066518,
                0.011878087,
                0.0013393817,
                0.03410649,
                -0.0019389635,
                -0.022104528,
                -0.021953126,
                -0.022916587,
                0.020810738,
                -0.00037097515,
                0.025600512,
                -0.010790753,
                -0.0014804597,
                -0.020507937,
                -0.00069377734,
                -0.00755628,
                -0.008712432,
                -0.0057807607,
                -0.001696378,
                -0.012951657,
                0.028738638,
                0.0042942795,
                -0.016819261,
                0.0141491005,
                0.0030899544,
                -0.00748058,
                0.0017497124,
                -0.018140579,
                0.006936913,
                0.0062659318,
                -0.042392246,
                -0.006159263,
                0.027554959,
                -0.037051924,
                -0.0027252159,
                0.008155002,
                0.033170555,
                0.01885629,
                0.00091958826,
                0.013749952,
                0.0006516259,
                -0.014685885,
                0.03443682,
                -0.0027166135,
                -0.018718654,
                0.004139438,
                -0.032785174
            ]
vectorized_queries = VectorizedQuery(vector = vectors,k_nearest_neighbors=7,fields="DescriptionVector",exhaustive=True,kind="vector")
results =  search_client.search(query_type='semantic', semantic_configuration_name='my-semantic-config',
    search_text="which restaurant has rating of 4 or over?", 
    select='HotelName,Description,Category', query_caption='extractive',
    vector_queries = [vectorized_queries]
    )

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])

0.03306011110544205
0.9661457538604736
Triple Landscape Hotel
0.01587301678955555
0.31893375515937805
Sublime Cliff Hotel
0.03306011110544205
0.061580806970596313
Twin Dome Motel
0.016129031777381897
0.023284239694476128
Secret Point Motel


In [70]:
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Index hotel-finder-vector-semantic Deleted


In [54]:
try:
    result = admin_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)

() No index with the name 'hotel-finder-semantic' was found in the service 'aisearchmcuksouth'.
Code: 
Message: No index with the name 'hotel-finder-semantic' was found in the service 'aisearchmcuksouth'.
